In [1]:
import pandas as pd
import numpy as np
import os
import re
import requests
from bs4 import BeautifulSoup

import os
os.environ["PGDATABASE"] = 'crsp'
os.environ['PGUSER'] = 'bdcallen'
os.environ['PGPASSWORD'] = 'temp_20180308'
os.getenv("PGHOST", "localhost")

dbname = os.getenv("PGDATABASE")
host = os.getenv("PGHOST", "localhost")
conn_string = "postgresql://" + host + "/" + dbname

from sqlalchemy import create_engine, inspect, types as st
from asxlisting import get_soup, is_valid_issuer_id, make_valid_id_df
from asxlisting_db import make_sql_table_comment, set_table_ownership_access
from asxlisting_db import conn_string

In [2]:
engine = create_engine(conn_string)

In [18]:
indexes_df = pd.read_sql("SELECT * FROM edgar.sc13dg_indexes", engine)

In [19]:
indexes_df

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start,success
0,edgar/data/1306923/0001306923-06-000019.txt,0001306923-06-000019.txt,SC 13G,2213.0,2413.0,False,2.0,2263.0,4480.0,4159.0,4249.0,2.0,6743.0,-1.0,True
1,edgar/data/1364742/0001086364-12-002106.txt,0001086364-12-002106.txt,SC 13G/A,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,edgar/data/872448/0001086364-11-002317.txt,0001086364-11-002317.txt,SC 13G,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,edgar/data/1329919/0001140361-13-004557.txt,0001140361-13-004557.txt,SC 13G,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,edgar/data/1552797/0000357235-13-000062.txt,0000357235-13-000062.txt,SC 13G,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
5,edgar/data/1334933/0001086364-11-000247.txt,0001086364-11-000247.txt,SC 13G/A,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
6,edgar/data/872448/0001085256-11-000013.txt,0001085256-11-000013.txt,SC 13G,1208.0,1514.0,False,2.0,1411.0,6351.0,6240.0,6245.0,5.0,10849.0,-1.0,True
7,edgar/data/902219/0000902219-12-000427.txt,0000902219-12-000427.txt,SC 13G/A,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
8,edgar/data/1364742/0001086364-12-002091.txt,0001086364-12-002091.txt,SC 13G/A,1323.0,1350.0,False,2.0,1323.0,2001.0,1999.0,-1.0,2.0,7228.0,-1.0,True
9,edgar/data/1306923/0001306923-06-000020.txt,0001306923-06-000020.txt,SC 13G,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [ ]:
sql = """SELECT a.file_name, a.document, b.form_type FROM edgar.filing_docs_processed AS a
         INNER JOIN edgar.filings AS b
         USING(file_name)
         WHERE b.form_type IN ('SC 13D', 'SC 13D/A', 'SC 13G', 'SC 13G/A')
         AND a.downloaded
         AND a.file_name ~ a.document
      """

downloaded_df = pd.read_sql(sql, engine)
downloaded_df

In [861]:
sql = """SELECT a.file_name, a.document, b.form_type FROM edgar.filing_docs_processed AS a
         INNER JOIN edgar.filings AS b
         USING(file_name)
         LEFT JOIN edgar.sc13dg_indexes_old AS c
         USING(file_name, document)
         WHERE b.form_type IN ('SC 13D', 'SC 13D/A', 'SC 13G', 'SC 13G/A')
         AND a.downloaded
         AND a.file_name ~ a.document
         AND c.file_name IS NULL
      """

smaller_df = pd.read_sql(sql, engine)
smaller_df 

,file_name,document,downloaded,company_name,form_type,cik,date_filed
0,edgar/data/1000014/0000930661-97-000604.txt,0000930661-97-000604.txt,True,RICE PARTNERS II L P,SC 13D,1000014,1997-03-14
1,edgar/data/1000015/0000080255-02-000148.txt,0000080255-02-000148.txt,True,META GROUP INC,SC 13G/A,1000015,2002-02-11
2,edgar/data/1000015/0000080255-03-000161.txt,0000080255-03-000161.txt,True,META GROUP INC,SC 13G/A,1000015,2003-02-04
3,edgar/data/1000015/0000080255-04-000151.txt,0000080255-04-000151.txt,True,META GROUP INC,SC 13G/A,1000015,2004-02-06
4,edgar/data/1000015/0000080255-05-000266.txt,0000080255-05-000266.txt,True,META GROUP INC,SC 13G/A,1000015,2005-02-11
5,edgar/data/1000015/0000080255-97-000155.txt,0000080255-97-000155.txt,True,META GROUP INC,SC 13G,1000015,1997-02-12
6,edgar/data/1000015/0000080255-98-000237.txt,0000080255-98-000237.txt,True,META GROUP INC,SC 13G/A,1000015,1998-02-10
7,edgar/data/1000015/0000080255-99-000474.txt,0000080255-99-000474.txt,True,META GROUP INC,SC 13G/A,1000015,1999-04-12
8,edgar/data/1000015/0000315066-01-000534.txt,0000315066-01-000534.txt,True,META GROUP INC,SC 13G/A,1000015,2001-02-14
9,edgar/data/1000015/0000315066-98-000797.txt,0000315066-98-000797.txt,True,META GROUP INC,SC 13G,1000015,1998-02-10


In [856]:
indexes_df.shape[0]

286013

In [857]:
smaller_df.shape[0]

1081694

In [858]:
downloaded_df.shape[0]

1367618

In [860]:
downloaded_df.shape[0] - smaller_df.shape[0] 

285924

In [5]:
indexes_df[indexes_df['success'] == False]

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start,success
1,edgar/data/1364742/0001086364-12-002106.txt,0001086364-12-002106.txt,SC 13G/A,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,edgar/data/872448/0001086364-11-002317.txt,0001086364-11-002317.txt,SC 13G,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,edgar/data/1329919/0001140361-13-004557.txt,0001140361-13-004557.txt,SC 13G,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,edgar/data/1552797/0000357235-13-000062.txt,0000357235-13-000062.txt,SC 13G,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
5,edgar/data/1334933/0001086364-11-000247.txt,0001086364-11-000247.txt,SC 13G/A,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
7,edgar/data/902219/0000902219-12-000427.txt,0000902219-12-000427.txt,SC 13G/A,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
9,edgar/data/1306923/0001306923-06-000020.txt,0001306923-06-000020.txt,SC 13G,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
10,edgar/data/1329919/0001172661-13-000444.txt,0001172661-13-000444.txt,SC 13G,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
11,edgar/data/1334933/0001183740-11-000126.txt,0001183740-11-000126.txt,SC 13G/A,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
13,edgar/data/1127439/0000948046-07-000041.txt,0000948046-07-000041.txt,SC 13G/A,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [12]:
get_index_url('edgar/data/1329919/0001140361-13-004557.txt')

'https://www.sec.gov/Archives/edgar/data/1329919/000114036113004557/0001140361-13-004557-index.htm'

In [6]:
def get_file_text_from_directory(file_name, document, directory):
    
    path = directory + '/' + get_file_path(file_name, document)
    f = open(path, 'r')
    txt = f.read()
    f.close()
    return(txt)

In [7]:
def get_file_path(file_name, document):
    
    url = re.sub("(\\d{10})-(\\d{2})-(\\d{6})\\.txt", "\\1\\2\\3", file_name)
    
    path = url + '/' + document
    
    return(path)

In [8]:
def get_index_url(file_name): 
    
    # This is a Python version of the function of the same name in scrape_filing_doc_functions.R
    
    regex = "/(\\d+)/(\\d{10}-\\d{2}-\\d{6})"
    matches = re.findall(regex, file_name)[0]
    
    cik = matches[0]
    acc_no = matches[1]
    
    path = re.sub("[^\\d]", "", acc_no)
    
    url = "https://www.sec.gov/Archives/edgar/data/" + cik + "/" + path + "/" + acc_no + "-index.htm"
    
    return(url)

In [569]:
def get_key_indices(file_name, document, form_type, directory):
    
    key_info = {}
    
    text = get_main_doc_text(file_name, document, directory)
    
    key_info['file_name'] = [file_name]
    key_info['document'] = [document]
    key_info['form_type'] = [form_type]
    key_info['title_page_end_lower_bound'] = [find_title_page_end(text)]
    key_info['cover_page_q1_start'] = [find_orthodox_cover_page_q1_start(text)]
    key_info['is_rep_q_as_items'] = [is_rep_q_as_items(text, key_info['cover_page_q1_start'][0])]
    if(key_info['is_rep_q_as_items'][0]):
        
        key_info['cover_page_q1_start'] = [cover_page_start_is_rep_as_q(text, form_type, \
                                                lower_bound = key_info['title_page_end_lower_bound'][0])]
    key_info['num_cusip_sedol_b_q1'] = [num_cusip_sedol_before_index(text, \
                                                                    key_info['cover_page_q1_start'][0])]
    key_info['cover_page_start'] = [cover_page_start(text, form_type, \
            cover_page_q1 = key_info['cover_page_q1_start'][0], \
                                                     rep_q_as_items = key_info['is_rep_q_as_items'][0])]
    
    
    key_info['item_section_start'] = [get_item_section_start(text, form_type)]
    key_info['cover_page_last_q_end'] = [find_cover_pages_last_question(text, \
                        key_info['is_rep_q_as_items'][0], key_info['cover_page_q1_start'][0], \
                                                                        key_info['item_section_start'][0])]
    key_info['cover_page_end'] = [find_cover_pages_end(text, form_type, key_info['is_rep_q_as_items'][0],\
                                key_info['cover_page_last_q_end'][0], key_info['item_section_start'][0])]
    
    key_info['num_cusip_b_items'] = [num_cusip_sedol_before_index(text, \
                                                                  index = key_info['item_section_start'][0])]
    key_info['signature_start'] = [get_signatures_sec_start(text)]
    
    if(key_info['cover_page_start'][0] != -1 and key_info['item_section_start'][0] != -1\
      and key_info['item_section_start'][0] > key_info['cover_page_start'][0]):
        
        amend_l_bound = key_info['cover_page_start'][0]
        
    else:
        
        amend_l_bound = None
    
    key_info['amendment_statement_start'] = find_amendment_statement_start(text, form_type, \
                    lower_bound = amend_l_bound, upper_bound = key_info['item_section_start'][0])
    
    
    key_info_df = pd.DataFrame(key_info)
    
    return(key_info_df)

In [202]:
def write_indexes_to_table(file_name, document, form_type, directory, engine):
    
    try:
        
        df = get_key_indices(file_name, document, form_type, directory)
        df['success'] = True
        df.to_sql('sc13dg_indexes', engine, schema="edgar", if_exists="append", index=False)
        
    except:
        
        df = pd.DataFrame({'file_name': [file_name], 'document': [document], 'form_type': [form_type], \
              'title_page_end_lower_bound': [-2], 'cover_page_q1_start': [-2], 'is_rep_q_as_items': [-2],\
              'num_cusip_sedol_b_q1': [-2], 'cover_page_start': [-2], 'item_section_start': [-2], \
              'cover_page_last_q_end': [-2], 'cover_page_end': [-2], 'num_cusip_b_items': [-2],\
              'signature_start': [-2], 'amendment_statement_start': [-2], 'success': [False]})
        
        df.to_sql('sc13dg_indexes', engine, schema="edgar", if_exists="append", index=False)    

In [151]:
def is_rep_q_as_items(text, q1_start = None):
    
    # this function works on the basis that the the actual "items" in the real item section does not go beyond
    # Item 10. So by searching for Item 12 or 14 (in SC 13G and SC 13D respectively), we can detect whether
    # a filer has erroneously labelled the questions in the reporting pages as items
    
    if(q1_start is None):
      
        q1_start = find_orthodox_cover_page_q1_start(text)
    
    if(q1_start == -1):
    
        text_raised = text.upper()
        
        regex = '\n\s*ITEM\s*(\.)?\s*[\(]?\s*1[24]\s*[\)]?(\.)?'
        
        search = re.search(regex, text_raised)
        
        if(search):
            return(True)
        else:
            return(False)
            
    else:
      
        return(False)

In [70]:
def find_last_search_start(regex, text):
    
    search = re.search(regex, text)
    
    if(search is None):
        
        return(None)
    
    else:
        
        end = 0
        
        while(search is not None):
            
            start = end + search.start()
            end = end + search.end()
            search = re.search(regex, text[end:])
        
    return(start)
    
    


def find_last_search_end(regex, text):
    
    search = re.search(regex, text)
    
    if(search is None):
        
        return(None)
    
    else:
        
        end = 0
        
        while(search is not None):
            
            end = end + search.end()
            search = re.search(regex, text[end:])
        
    return(end)


In [16]:
def get_title_page_end_lower_bound(text, upper_bound = None):
    
    rep_q_as_items = is_rep_q_as_items(text)
    
    if(upper_bound is None):
        upper_bound = get_title_page_end_upper_bound(text)
    
    text_raised = text.upper()[:upper_bound]
    
    if(not rep_q_as_items):
        
        last_search = find_last_search_end('\((?:NAME|CUSIP|SEDOL|DATE|TITLE|AMENDMENT)(.*?\s)*?' + \
                     '(?:ISSUER|SECURITIES|NUMBER|STATEMENT|SCHEDULE|NO\.?|#)\)', text_raised)
        
        if(last_search):
            
            return(last_search)
        
        else:
            return(0)
        
    else:
        
        return(0)

        
        
def get_title_page_end_upper_bound(text, rep_q_as_items = None):
    
    if(rep_q_as_items is None):
        rep_q_as_items = is_rep_q_as_items(text)

    text_raised = text.upper()

    if(rep_q_as_items):

        end = re.search('ITEM\s*[#\.\:]?\s*[\(]?\s*1[24]\s*[\)]?[\.\:]?', text_raised).start()

        regex = 'ITEM\s*[#\.\:]?\s*[\(]?\s*1\s*[\)]?[\.\:]?'

        search = re.search(regex, text_raised[:end])

        return(search.start())

    else:

        regex = '[\(]?\s*1\s*[\)]?\s*[:\.]{0,1}\s*NAME[S]?\s+(OF\s+(?:REPORTING|ABOVE)\s+PERSON[S]?)?' + \
                '(\s*/\s*)?(S\.S\.\s+)?(\s+(?:OR|AND))\s+I\.R\.S\.\s+IDENTIFICATION\s+NO[S]?\.\s+' + \
                '(OF\s+(?:REPORTING|ABOVE)\s+PERSON[S]?)?'     

        search = re.search(regex, text_raised)

        if(search):

            return(search.start())

        else:
            regex = '(?<!DESCRIBED\sIN)\s*ITEM\s*[#\.\:]?\s*[\(]?\s*[0-9]{1,2}\s*' + \
                    '[\)]?[\.\:]?'
            search = re.search(regex, text_raised)

            return(search.start())

In [159]:
def find_title_page_end(text):
    
    rep_q_as_items = is_rep_q_as_items(text)
    
    upper_bound = get_title_page_end_upper_bound(text, rep_q_as_items)
    
    text_raised = text.upper()[:upper_bound]
    
    regex1_end = find_regex1_end(text)
    
    num_cusip_lines = len(re.findall('(?:CUSIP|SEDOL)', text_raised))
    
    if(regex1_end != -1):
        
        return(regex1_end)
    
    elif(num_cusip_lines < 2):
        
        return(upper_bound)
        
    else:
            
        
        rep_start_re = ['(?:CUSIP|SEDOL)', 'SCHEDULE\s+13[DG](/A)?', 'SC\s+13[DG](/A)?', \
                        'PAGE', '13[DG](/A)?']

        pot_results = []

        for regex in rep_start_re:

            start = find_last_search_start(regex, text_raised)

            if(start):
                pot_results.append(start)
                
        result = min(pot_results)
        
        return(result)
    

In [194]:
def find_orthodox_cover_page_q1_start(text):

    text_raised = text.upper()

    regex0 = '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*(.){0,50}\s*' + \
             'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?\s+(?:REPORTING|ABOVE|REPORT)'
    
    regex1 = '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*(.){0,50}\s*' + \
             'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?(\s+(?:REPORTING|ABOVE|REPORT))?\s+' + \
             'PERSON[\(]?[S]?[\)]?'

    regex2 = '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*(.){0,50}\s*' + \
             'NAME[\(]?[S]?[\)]?\s+(?:AND|OF|OR)(\s+I[\.]?R[\.]?S)?\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?\.'
    
    regex3 = '(NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?(\s+(?:REPORTING|ABOVE|REPORT))?\s+' + \
             'PERSON[\(]?[S]?[\)]?\s+)?[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.\|]{0,1}\s*' + \
             'I\.R\.S\.\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?\.\s+(?:OF|OR)\s+(\s+THE)?ABOVE\s+' + \
             'PERSON[\(]?[S]?[\)]?'
    
    regex4 = '(NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?(\s+(?:REPORTING|ABOVE|REPORT))?\s+' + \
             'PERSON[\(]?[S]?[\)]?\s+)?[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.\|]{0,1}\s*S\.S\.\s+OR\s+' + \
             'I\.R\.S\.\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?\.\s+(?:OF|OR)(\s+THE)?\s+' + \
             'ABOVE\s+PERSON[\(]?[S]?[\)]?'
    
    regex5 = '[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*' + \
             'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?\s+FILING\s+(?:PARTY|PARTIES|PERSON|PERSONS)'
    
    regex6 = 'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?(\s+(?:REPORTING|ABOVE|REPORT))?\s+' + \
             'PERSON[\(]?[S]?[\)]?\s+(S\.S\.\s+OR\s+I\.R\.S\.\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?\.\s+' + \
             '(?:OF|OR)(\s+THE)?\s+ABOVE\s+PERSON[\(]?[S]?[\)]?)?' + \
             '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?'
    
    
    regex7 = '\n\s*[\(\|]?\s1\s?[\)\|]?\s*[:\.]{0,1}([A-Z0-9\s\.\(\)_]?)+?' + \
             '[\(\|]?\s2\s?[\)\|]?\s*[:\.]{0,1}([A-Z0-9\s\.\(\)_]?)+?' + \
             '[\(\|]?\s3\s?[\)\|]?\s*[:\.]{0,1}\s+(.)+\s*\(SEC USE ONLY\)'
    
    regex8 = 'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?\s+(?:REPORTING|ABOVE|REPORT)'
    
    regex9 = 'NAME[\(]?[S]?[\)]?\s+(?:AND|OF|OR)(\s+I[\.]?R[\.]?S)?\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?.'
    
    regex10 = '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*(.){0,50}\s*' + \
              'NAME[\(]?[S]?[\)]?\s+(?:AND|OF|OR)(\s+I[\.]?R[\.]?S)?\s+NUMBER\s+OF\s+REPORTING\s+PERSON[S]?'
    
    
    regex_list = [regex0, regex1, regex2, regex3, regex4, regex5, regex6, regex7, regex8, regex9, regex10]
    
    start = None
    
    for i in range(len(regex_list)):
        
        search = re.search(regex_list[i], text_raised)
        
        if(search):
            
            if(start is None):
                
                start = search.start()
                
            else:
                
                new = search.start()
                
                if(new < start):
                    
                    start = new            

    if(start):
        
        return(start)
    
    else:
        
        return(-1)   

In [201]:
def cover_page_start(text, form_type, cover_page_q1 = None, rep_q_as_items = None):
    
    if(rep_q_as_items is None):
        
        rep_q_as_items = is_rep_q_as_items(text)
    
    if(cover_page_q1 is None):
        
        cover_page_q1 = find_cover_page_q1_start(text, rep_q_as_items)
        
    if(cover_page_q1 == -1):
        
        # No question 1 found, so no cover pages, return -1
        return(-1) 
        
    else:
        
        text_to_search = text[:cover_page_q1]
    
        text_reversed_and_raised = text_to_search[::-1].upper()

        regex = '(PISUC|LODES)\s*\n'

        search = re.search(regex, text_reversed_and_raised)

        if(search):

            num_back = search.end()

        else:

            num_back = 0

        start = cover_page_q1 - num_back

        return(start)


In [200]:
def find_cover_pages_last_question(text, rep_q_as_items = None,  lower_bound = None, upper_bound = None):
    
    if(rep_q_as_items is None):
        
        rep_q_as_items = is_rep_q_as_items(text)
    
    if(lower_bound is not None and lower_bound > 0):
        
        if(upper_bound is not None and upper_bound > 0):
            
            text_raised = text.upper()[lower_bound:upper_bound]
            
        else:
            
            text_raised = text.upper()[lower_bound:]
            
    else:
        
        if(upper_bound is not None and upper_bound > 0):
            
            text_raised = text.upper()[:upper_bound]
            
        else:
            
            text_raised = text.upper() 
    
    if(rep_q_as_items):
        
        regex = 'ITEM\s*[\(]?\s*1[24]\s*[\)]?\s*[\:\.]?'
        
    else:
    
        regex = '[\(]?\s*1[24][\)]?\s*[\:\.]?\s*TYPE[S]?\s+OF\s+REPORTING\s+PERSON[S]?[\*]?\s*(\(SEE\s+INSTRUCTIONS\))?'

    search = re.search(regex, text_raised)

    if(search is None):
        return(None)
    
    end = 0

    while(search is not None):
        
        end = end + search.end()
        search = re.search(regex, text_raised[end:])

    if(lower_bound is not None and lower_bound > 0):
        
        end = end + lower_bound
        
    return(end)       
        

    
    
def find_cover_pages_end(text, rep_q_as_items = None, lower_bound = None, upper_bound = None):
    
    
    end = find_cover_pages_last_question(text, rep_q_as_items, lower_bound, upper_bound)
        
    answer_regex = '\s*([A-Z0]{1}[;, \.]?[A-Z0]{1}[;, \.]?)*([A-Z0]{1}[;, \.]?[A-Z0]{1}[;, \.]?)\s*\n?'
    
    if(upper_bound is not None):
    
        last_answer = re.search(answer_regex, text[end:upper_bound])
        
    else:
        
        last_answer = re.search(answer_regex, text[end:])
    
    if(last_answer is not None):
    
        return(end + last_answer.end())
    
    else:
        
        return(-1)
        
 

def num_cusip_sedol_before_index(text, index = None):

    try:
        if(index):
            text_lowered = text.lower()[:index]
        else:
            text_lowered = text.lower()
        
        num_cusip_sedol = len(re.findall('(cusip|sedol)', text_lowered))
        
        return(num_cusip_sedol)

    except Exception as e:
        print(e)
        return(None)

    
def find_explanatory_statement_start(text, lower_bound = None, upper_bound = None):
    
    # This function is designed to get the start of the explanatory/amendment statement 
    # that is sometimes found between the end of the cover pages and the start of the item section 
    
    

    regex_list = ['\n\s*explanatory\s+note', '\n\s*amendment\s+(number|no)(\.)?\s*[0-9]+\s*\n',\
                  '\n\s*schedule\s+13[dg]\s+amendment\s+(number|no)(\.)?\s*[0-9]+\s*\n',\
                  '\n\s*this\s+amendment\s+(number|no)(\.)?\s*[0-9]+\s+', \
                    '\n\s*pursuant\s+to\s+rule\s+13d-', '\n\s*this\s+amended\s+schedule\s+13[dg]', \
'\n\s*the\s+undersigned(\s+reporting\s+person(s)?)?\s+hereby\s+amend\s+(the|their)\s+schedule\s+13[dg]',\
                  '\n\s*the\s+schedule\s+13[dg]\s+was\s+initially\s+filed',\
                 '\n(.)+hereby\s+amends\s+the\s+schedule\s+13[dg]',\
                  '\n\s*the\s+filing\s+of\s+this\s+schedule\s+13[dg]',\
                 '\n\s*the\s+filing\s+of\s+this\s+statement\s+on\s+schedule\s+13[dg]',\
                 '\n\s*the\s+reporting\s+person(s)?\s+listed\s+']

    if(lower_bound):

        if(upper_bound):

            text_lowered = text.lower()[lower_bound:upper_bound]

        else:

            text_lowered = text.lower()[lower_bound:]

    else:

        if(upper_bound):

            text_lowered = text.lower()[:upper_bound]

        else:

            text_lowered = text.lower()

    search_starts = []

    for pattern in regex_list:

        search = re.search(pattern, text_lowered)

        if(search):

            search_starts.append(search.start())

    if(len(search_starts)):

        if(lower_bound):

            return(min(search_starts) + lower_bound)

        else:

            return(min(search_starts))

    else:

        return(-1)
        
    

In [842]:
i = 2000
get_key_indices(sc_13d_df.loc[i, 'file_name'], sc_13d_df.loc[i, 'document'], \
                sc_13d_df.loc[i, 'form_type'], directory)

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,exhibit_start,main_text_length,amendment_statement_start,has_jumbled_order,has_exhibit_break
0,edgar/data/1056947/0001140361-13-024732.txt,0001140361-13-024732.txt,SC 13D,1940,1940,False,1,1940,6012,5715,5780,3,23819,24639,28555,-1,False,True


In [635]:
find_orthodox_cover_page_q1_start(t)

1871

In [689]:
find_all_orthodox_q1_starts(t)

[1669, 5796]

In [840]:
t[100:-1]

'.C. 20549\n\nSCHEDULE 13G\n\nUnder the Securities Exchange Act of 1934\n\n(Amendment No:  1)\n\nJACOBS ENGINEERING GROUP INC\n--------------------------------------------------------\n(Name of Issuer)\n\nCommon Stock\n--------------------------------------------------------\n(Title of Class of Securities)\n\n469814107\n--------------------------------------------------------\n(CUSIP Number)\n\nDecember 30, 2011\n--------------------------------------------------------\n(Date of Event Which Requires Filing of this Statement)\n\nCheck the appropriate box to designate the rule pursuant to\nwhich this Schedule is filed:\n\n[X] Rule 13d-1(b)\n[ ] Rule 13d-1(c)\n[ ] Rule 13d-1(d)\n\n\n*The remainder of this cover page shall be filled out\nfor a reporting person\'s initial filing on this form with\nrespect to the subject class of securities, and for any\nsubsequent amendment containing information which\nwould alter the disclosures provided in a prior cover page.\n\nThe information required 

In [841]:
t[100:]

'.C. 20549\n\nSCHEDULE 13G\n\nUnder the Securities Exchange Act of 1934\n\n(Amendment No:  1)\n\nJACOBS ENGINEERING GROUP INC\n--------------------------------------------------------\n(Name of Issuer)\n\nCommon Stock\n--------------------------------------------------------\n(Title of Class of Securities)\n\n469814107\n--------------------------------------------------------\n(CUSIP Number)\n\nDecember 30, 2011\n--------------------------------------------------------\n(Date of Event Which Requires Filing of this Statement)\n\nCheck the appropriate box to designate the rule pursuant to\nwhich this Schedule is filed:\n\n[X] Rule 13d-1(b)\n[ ] Rule 13d-1(c)\n[ ] Rule 13d-1(d)\n\n\n*The remainder of this cover page shall be filled out\nfor a reporting person\'s initial filing on this form with\nrespect to the subject class of securities, and for any\nsubsequent amendment containing information which\nwould alter the disclosures provided in a prior cover page.\n\nThe information required 

In [653]:
indexes_df.loc[indexes_df['file_name'] == 'edgar/data/1106939/0000892303-00-000015.txt']

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start,success
9168,edgar/data/1106939/0000892303-00-000015.txt,0000892303-00-000015.txt,SC 13G,1666.0,1666.0,False,1.0,1666.0,6671.0,6669.0,-1.0,2.0,5403.0,-1.0,True


In [844]:
def get_key_indices(file_name, document, form_type, directory):
    
    key_info = {}
    
    text = get_main_doc_text(file_name, document, directory)
    
    key_info['file_name'] = [file_name]
    key_info['document'] = [document]
    key_info['form_type'] = [form_type]
    key_info['title_page_end_lower_bound'] = [find_title_page_end(text)]
    key_info['cover_page_q1_start'] = [find_orthodox_cover_page_q1_start(text)]
    key_info['is_rep_q_as_items'] = [is_rep_q_as_items(text, key_info['cover_page_q1_start'][0])]
    if(key_info['is_rep_q_as_items'][0]):
        
        key_info['cover_page_q1_start'] = [cover_page_start_is_rep_as_q(text, form_type, \
                                                lower_bound = key_info['title_page_end_lower_bound'][0])]
    
    key_info['is_schedule_to'] = [is_schedule_to(text)]
    key_info['has_table_of_contents'] = [has_table_of_contents(text)]
    
    key_info['num_cusip_sedol_b_q1'] = [num_cusip_sedol_before_index(text, \
                                                                    key_info['cover_page_q1_start'][0])]
    key_info['cover_page_start'] = [cover_page_start(text, form_type, \
            cover_page_q1 = key_info['cover_page_q1_start'][0], \
                                                     rep_q_as_items = key_info['is_rep_q_as_items'][0])]
    
    
    key_info['item_section_start'] = [get_item_section_start(text, form_type)]
    key_info['cover_page_last_q_end'] = [find_cover_pages_last_question(text, \
                        key_info['is_rep_q_as_items'][0], key_info['cover_page_q1_start'][0], \
                                                                        key_info['item_section_start'][0])]
    key_info['cover_page_end'] = [find_cover_pages_end(text, form_type, key_info['is_rep_q_as_items'][0],\
                                key_info['cover_page_last_q_end'][0], key_info['item_section_start'][0])]
    
    key_info['num_cusip_b_items'] = [num_cusip_sedol_before_index(text, \
                                                                  index = key_info['item_section_start'][0])]
    key_info['signature_start'] = [get_signatures_sec_start(text, key_info['item_section_start'][0])]
    key_info['exhibit_start'] = [get_exhibit_sec_start(text, key_info['signature_start'][0])]
    key_info['main_text_length'] = [len(text)]
    
    if(key_info['cover_page_start'][0] != -1 and key_info['item_section_start'][0] != -1\
      and key_info['item_section_start'][0] > key_info['cover_page_start'][0]):
        
        amend_l_bound = key_info['cover_page_start'][0]
        
    else:
        
        amend_l_bound = None
    
    key_info['amendment_statement_start'] = find_amendment_statement_start(text, form_type, \
                    lower_bound = amend_l_bound, upper_bound = key_info['item_section_start'][0])
    
    key_info['has_jumbled_order'] = [has_jumbled_order(text, form_type, key_info['exhibit_start'][0])]
    
    key_info['has_exhibit_break'] = [has_exhibit_break(text, key_info['cover_page_start'][0],\
                                                       key_info['signature_start'][0])]
    

    
    key_info_df = pd.DataFrame(key_info)
    
    return(key_info_df)
    
    

In [ ]:
has_table_of_contents(text)

In [831]:
indexes_df[indexes_df['document'] == '0000928464-12-000024.txt']

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start,success
22849,edgar/data/1376139/0000928464-12-000024.txt,0000928464-12-000024.txt,SC 13D/A,7459.0,-1.0,True,1.0,-1.0,NaN,10241.0,10427.0,1.0,10378.0,3358.0,True
42119,edgar/data/921669/0000928464-12-000024.txt,0000928464-12-000024.txt,SC 13D/A,7459.0,-1.0,True,1.0,-1.0,NaN,10241.0,10427.0,1.0,10378.0,3358.0,True


In [836]:
def is_schedule_to(text, lower_bound = None, upper_bound = None):
    
    text_raised = get_bounded_text(text, lower_bound, upper_bound).upper()
    
    if(re.search('\n\s*(?:SCHEDULE|SC)\s+TO(/A)?\s*\n', text_raised)):
        
        return(True)
    
    else:
        
        return(False)

In [839]:
def has_table_of_contents(text, lower_bound = None, upper_bound = None):
    
    text_raised = get_bounded_text(text, lower_bound, upper_bound).upper()
    
    if(re.search('\n\s*TABLE\s+OF\s+CONTENTS\s*\n')):
        
        return(True)
    
    else:
        
        return(False)

In [837]:
i = 22849
is_schedule_to(get_main_doc_text(indexes_df.loc[i, 'file_name'], indexes_df.loc[i, 'document'], directory))

True

In [838]:
i = 9168
is_schedule_to(get_main_doc_text(indexes_df.loc[i, 'file_name'], indexes_df.loc[i, 'document'], directory))

False

In [ ]:
has_table_of_contents(text)

In [845]:
i = 9168
get_key_indices(indexes_df.loc[i, 'file_name'], indexes_df.loc[i, 'document'], \
                indexes_df.loc[i, 'form_type'], directory)

TypeError: search() missing 1 required positional argument: 'string'

In [825]:
i = 1
get_key_indices(indexes_df.loc[i, 'file_name'], indexes_df.loc[i, 'document'], \
                indexes_df.loc[i, 'form_type'], directory)

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,exhibit_start,main_text_length,amendment_statement_start,has_jumbled_order,has_exhibit_break
0,edgar/data/1364742/0001086364-12-002106.txt,0001086364-12-002106.txt,SC 13G/A,1355,1382,False,2,1355,2039,2031,2039,2,7294,8390,11545,-1,False,False


In [821]:
get_index_url(indexes_df.loc[i, 'file_name'])

'https://www.sec.gov/Archives/edgar/data/872448/000031506611002274/0000315066-11-002274-index.htm'

In [827]:
print(t[8390:])




Exhibit A




Subsidiary

BlackRock Japan Co. Ltd.
BlackRock Advisors (UK) Limited
BlackRock Institutional Trust Company, N.A.
BlackRock Fund Advisors
BlackRock Asset Management Canada Limited
BlackRock Asset Management Australia Limited
BlackRock Advisors, LLC
BlackRock Investment Management, LLC
BlackRock Investment Management (Australia) Limited
BlackRock (Luxembourg) S.A.
BlackRock (Netherlands) B.V.
BlackRock Fund Managers Limited
BlackRock Asset Management Ireland Limited
BlackRock International Limited
BlackRock Investment Management (UK) Limited



*Entity beneficially owns 5% or greater of the outstanding
shares of the security class being reported on this
Schedule 13G.
Exhibit B


POWER OF ATTORNEY

The undersigned, BLACKROCK, INC., a corporation duly organized
under the laws of the State of Delaware, United States (the
"Company"), does hereby make, constitute and appoint each of Robert
Connolly, Howard Surloff, Edward Baer, Bartholomew Battista,
Daniel Waltcher, Karen Cla

In [820]:
print(t[588:3433])

Item 1(a).	Name of Issuer:

		Atmel Corporation

Item 1(b).	Name of Issuer's Principal Executive Offices:

		2325 Orchard Parkway
		San Jose, CA  95131


Item 2(a).	Name of Person Filing:

		FMR LLC

Item 2(b).	Address or Principal Business Office or, if None,
Residence:

		82 Devonshire Street, Boston,
Massachusetts  02109

Item 2(c).	Citizenship:

		Not applicable

Item 2(d).	Title of Class of Securities:

		Common Stock

Item 2(e).	CUSIP Number:

		049513104

Item 3.	This statement is filed pursuant to Rule 13d-1(b) or 13d-2(b)
and the person filing, FMR LLC, is a parent holding company
in accordance with Section 240.13d-1(b)(ii)(G).  (Note:  See
Item 7).

Item 4.	Ownership

	(a)	Amount Beneficially Owned:	23,492,785

	(b)	Percent of Class:	5.126%

	(c)	Number of shares as to which such
person has:

	(i)	sole power to vote or to direct
the vote:	1,749,751

	(ii)	shared power to vote or to
direct the vote:	0

	(iii)	sole power to dispose or to
direct the disposition of:	23,492,785

	

In [826]:
t = get_main_doc_text(indexes_df.loc[i, 'file_name'], indexes_df.loc[i, 'document'], directory)

In [781]:
get_index_url('edgar/data/814133/0000814133-13-000028.txt')

'https://www.sec.gov/Archives/edgar/data/814133/000081413313000028/0000814133-13-000028-index.htm'

In [848]:
def is_schedule_to(text, lower_bound = None, upper_bound = None):
    
    text_raised = get_bounded_text(text, lower_bound, upper_bound).upper()
    
    if(re.search('\n\s*(?:SCHEDULE|SC)\s+TO(/A)?\s*\n', text_raised)):
        
        return(True)
    
    else:
        
        return(False)
        
        
        
def has_table_of_contents(text, lower_bound = None, upper_bound = None):
    
    text_raised = get_bounded_text(text, lower_bound, upper_bound).upper()
    
    if(re.search('\n\s*TABLE\s+OF\s+CONTENTS\s*\n', text_raised)):
        
        return(True)
    
    else:
        
        return(False)

In [853]:
i = 100000
eg = get_key_indices(indexes_df.loc[i, 'file_name'], indexes_df.loc[i, 'document'], \
                indexes_df.loc[i, 'form_type'], directory)

eg

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,is_schedule_to,has_table_of_contents,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,exhibit_start,main_text_length,amendment_statement_start,has_jumbled_order,has_exhibit_break
0,edgar/data/1184765/0001445546-13-003273.txt,0001445546-13-003273.txt,SC 13G/A,1824,1891,False,False,False,2,1869,3965,3770,3830,2,11106,-1,12207,-1,False,False


In [851]:
eg.columns

Index(['file_name', 'document', 'form_type', 'title_page_end_lower_bound',
       'cover_page_q1_start', 'is_rep_q_as_items', 'is_schedule_to',
       'has_table_of_contents', 'num_cusip_sedol_b_q1', 'cover_page_start',
       'item_section_start', 'cover_page_last_q_end', 'cover_page_end',
       'num_cusip_b_items', 'signature_start', 'exhibit_start',
       'main_text_length', 'amendment_statement_start', 'has_jumbled_order',
       'has_exhibit_break'],
      dtype='object')

In [852]:
def write_indexes_to_table(file_name, document, form_type, directory, engine):
    
    try:
        
        df = get_key_indices(file_name, document, form_type, directory)
        df['success'] = True
        df.to_sql('sc13dg_indexes', engine, schema="edgar", if_exists="append", index=False)
        
    except:
        
        df = pd.DataFrame({'file_name': [file_name], 'document': [document], 'form_type': [form_type], \
              'title_page_end_lower_bound': [-2], 'cover_page_q1_start': [-2], 'is_rep_q_as_items': [-2],\
              'is_schedule_to': [-2],  'has_table_of_contents': [-2], 'num_cusip_sedol_b_q1': [-2],\
              'cover_page_start': [-2], 'item_section_start': [-2], 'cover_page_last_q_end': [-2],\
              'cover_page_end': [-2], 'num_cusip_b_items': [-2], 'signature_start': [-2], \
              'exhibit_start': [-2], 'main_text_length': [-2], 'amendment_statement_start': [-2],\
              'has_jumbled_order': [-2], 'has_exhibit_break': [-2], 'success': [False]})
        
        df.to_sql('sc13dg_indexes', engine, schema="edgar", if_exists="append", index=False)    

In [815]:
print(re.sub('\n\s+\n', '\n', t[25875:]))


SIGNATURES
     After reasonable inquiry and to the best of my knowledge and belief, I certify that the
information set forth in this statement is true, complete and correct.
     Dated: August 15, 2011
STRATFORD CAPITAL PARTNERS, L.P.
By:
Stratford Capital GP Associates, L.P., its general partner
By:
Stratford Capital Corporation, its general partner
By:
/s/ David W. Knickel
David W. Knickel, Vice President
STRATFORD CAPITAL GP ASSOCIATES, L.P.
By:
Stratford Capital Corporation, its general partner
By:
/s/ David W. Knickel
David W. Knickel, Vice President
STRATFORD CAPITAL CORPORATION
By:
/s/ David W. Knickel
David W. Knickel, Vice President
RETAIL & RESTAURANT GROWTH CAPITAL, L.P.
By:
Retail & Restaurant Growth Partners, L.P., its general partner
By: Retail & Restaurant Growth Management, Inc., its general partner
By:
/s/ Raymond C. Hemmig
Raymond C. Hemmig, Chairman and CEO
RETAIL & RESTAURANT GROWTH PARTNERS, L.P.
By: Retail & Restaurant Growth Management, Inc., its general partne

In [790]:
get_index_url(indexes_df.loc[i, 'file_name'])

'https://www.sec.gov/Archives/edgar/data/1742550/000121390018012112/0001213900-18-012112-index.htm'

In [804]:
t = get_main_doc_text(indexes_df.loc[i, 'file_name'], indexes_df.loc[i, 'document'], directory)

In [736]:
find_orthodox_cover_page_q1_start_end(t)

(-1, -1)

In [793]:
t[15471:16200]

'AMENDMENT NO. 3 TO SCHEDULE 13D\n\n\n\n\n\n\n\xa0\xa0\xa0\xa0\xa0This Amendment No.\xa03 to Schedule\xa013D (this “\nThird Amendment\n”) amends and supplements the\nSchedule\xa013D originally filed on August\xa021, 2006, as amended by Amendment No.\xa01 on August\xa028, 2006,\nand by Amendment No.\xa02 on September\xa024, 2010 (the “\nSecond Amendment\n”) (collectively, the “\nSchedule\n13D\n”), which related to the common stock, par value $0.001 per share (“\nCommon Stock\n”), of Teletouch\nCommunications, Inc., a Delaware corporation (the “\nIssuer\n”). Capitalized terms used herein and not\notherwise defined have the meaning assigned to such terms in the Second Amendment. All items or\nresponses not described herein remain as previously reported in the Second Amendment.\n\n\n\n\n\n\n\n'

In [776]:
t[1922:7212]

'\n\n\n\n\n\n\n\n    CUSIP NO. 91529Y106           13G                        Page 2 of 10 Pages\n\n  1. NAME OF REPORTING PERSON\n     S.S. OR I.R.S. IDENTIFICATION NO. OF ABOVE PERSON\n\n        AXA Assurances I.A.R.D. Mutuelle\n\n  2. CHECK THE APPROPRIATE BOX IF A MEMBER OF A GROUP *                (A) [ ]\n                                                                       (B) [ ]\n\n  3. SEC USE ONLY\n\n  4. CITIZENSHIP OR PLACE OF ORGANIZATION\n     France\n\n\n           NUMBER OF SHARES   5. SOLE VOTING POWER                           0\n               BENEFICIALLY\n                OWNED AS OF   6. SHARED VOTING POWER                    38,369\n          December 31, 2010\n                    BY EACH   7. SOLE DISPOSITIVE POWER                      0\n                  REPORTING\n               PERSON WITH:   8. SHARED DISPOSITIVE POWER               56,017\n\n\n  9. AGGREGATE AMOUNT BENEFICIALLY OWNED BY EACH                        56,017\n     REPORTING PERSON\n     (Not 

In [733]:
get_index_url('edgar/data/888002/0000888002-11-000169.txt')

'https://www.sec.gov/Archives/edgar/data/888002/000088800211000169/0000888002-11-000169-index.htm'

In [699]:
get_exhibit_sec_start(text, lower_bound = 1669, upper_bound = 6676)

-1

In [795]:
def get_exhibit_sec_start(text, lower_bound = None, upper_bound = None):
    
    text_upper = get_bounded_text(text, lower_bound, upper_bound).upper()
    
    regex0 = '\n\s*EXHIBIT\s+INDEX\s*\n'
    
    regex1 = '\n\s*EXHIBIT\s+[0-9A-Z](:)?\s*\n'
    
    regex2 = '\n\s*APPENDIX\s+[0-9A-Z](:)?\s*\n'
    
    regex_list = [regex0, regex1, regex2]
    
    regex_search = None
    
    for i in range(len(regex_list)):
        
        regex_search = re.search(regex_list[i], text_upper)
        
        if(regex_search):
            
            break
        
    if(regex_search):
        
        if(lower_bound is not None):
        
            return(regex_search.start() + lower_bound)
        
        else:
            
            return(regex_search.start())
    
    else:
        return(-1)

In [ ]:
find_all_orthodox_q1_starts(text, exhibit_start)

In [734]:
regex0 = '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*(.){0,50}\s*' + \
         'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?\s+(?:REPORTING|ABOVE|REPORT)'

In [737]:
re.search(regex0, t)

In [741]:
t[1000:]

"------------------\n            (Date of event which requires filing of this Statement)\n\n  Check the appropriate box to designate the rule pursuant to which\n  this Schedule is filed:\n\n    X  Rule 13d-1(b)\n       Rule 13d-1(c)\n       Rule 13d-1(d)\n\n  * The remainder of this cover page shall be filled out for a reporting\n  person's initial filing on this form with respect to the subject class\n  of securities, and for any subsequent amendment containing information\n  which would alter the disclosures provided in a prior cover page.\n\n  The information required in the remainder of this cover page shall not\n  be deemed to be 'filed' for the purpose of Section 18 of the Securities\n  Exchange Act of 1934 ('Act') or otherwise subject to the liabilities\n  of that section of the Act but shall be subject to all other provisions\n  of the Act (however, see the Notes).\n\n                       (CONTINUED ON FOLLOWING PAGE(S))\n\n\n\n\n\n\n\n                                   13G  

In [742]:
page = requests.get('https://www.sec.gov/Archives/edgar/data/5513/000088800211000169/0000888002-11-000169.txt')
page

<Response [200]>

In [743]:
page.content

b"-----BEGIN PRIVACY-ENHANCED MESSAGE-----\nProc-Type: 2001,MIC-CLEAR\nOriginator-Name: webmaster@www.sec.gov\nOriginator-Key-Asymmetric:\n MFgwCgYEVQgBAQICAf8DSgAwRwJAW2sNKK9AVtBzYZmr6aGjlWyK3XmZv3dTINen\n TWSM7vrzLADbmYQaionwg5sDW3P6oaM5D3tdezXMm7z1T+B+twIDAQAB\nMIC-Info: RSA-MD5,RSA,\n F9OBTNxQEgyxwS0QYqvNcTdxBQbMQ9JPA9jDfjshs7957Aj1yE03B4Q7pk1W+ijm\n 1aZtYaIYyQYFVRn/Y7N2Gg==\n\n<SEC-DOCUMENT>0000888002-11-000169.txt : 20110209\n<SEC-HEADER>0000888002-11-000169.hdr.sgml : 20110209\n<ACCEPTANCE-DATETIME>20110209083424\nACCESSION NUMBER:\t\t0000888002-11-000169\nCONFORMED SUBMISSION TYPE:\tSC 13G/A\nPUBLIC DOCUMENT COUNT:\t\t2\nFILED AS OF DATE:\t\t20110209\nDATE AS OF CHANGE:\t\t20110209\nGROUP MEMBERS:\t\tAXA\nGROUP MEMBERS:\t\tAXA ASSURANCES I.A.R.D. MUTUELLE\nGROUP MEMBERS:\t\tAXA ASSURANCES VIE MUTUELLE\nGROUP MEMBERS:\t\tAXA-EQUITABLE LIFE INSURANCE COMPANY\n\nSUBJECT COMPANY:\t\n\n\tCOMPANY DATA:\t\n\t\tCOMPANY CONFORMED NAME:\t\t\tUnum Group\n\t\tCENTRAL INDEX KEY:\t\t\t000000

In [748]:
re.search(b'<[a-zA-Z0-9\-]+\s+', page.content)

<re.Match object; span=(4397, 4408), match=b'<PAGE\n\n    '>

In [753]:
re.search(b'<PAGE\s+', page.content)

<re.Match object; span=(4397, 4408), match=b'<PAGE\n\n    '>

In [757]:
re.search(b'>', page.content[4408:])

<re.Match object; span=(6287, 6288), match=b'>'>

In [764]:
6288 + 4408

10696

In [765]:
page.content[4397:10696]

b"<PAGE\n\n    CUSIP NO. 91529Y106           13G                        Page 2 of 10 Pages\n\n  1. NAME OF REPORTING PERSON\n     S.S. OR I.R.S. IDENTIFICATION NO. OF ABOVE PERSON\n\n        AXA Assurances I.A.R.D. Mutuelle\n\n  2. CHECK THE APPROPRIATE BOX IF A MEMBER OF A GROUP *                (A) [ ]\n                                                                       (B) [ ]\n\n  3. SEC USE ONLY\n\n  4. CITIZENSHIP OR PLACE OF ORGANIZATION\n     France\n\n\n           NUMBER OF SHARES   5. SOLE VOTING POWER                           0\n               BENEFICIALLY\n                OWNED AS OF   6. SHARED VOTING POWER                    38,369\n          December 31, 2010\n                    BY EACH   7. SOLE DISPOSITIVE POWER                      0\n                  REPORTING\n               PERSON WITH:   8. SHARED DISPOSITIVE POWER               56,017\n\n\n  9. AGGREGATE AMOUNT BENEFICIALLY OWNED BY EACH                        56,017\n     REPORTING PERSON\n     (Not to be 

In [754]:
re.findall(b'<PAGE\s+', page.content)

[b'<PAGE\n\n    ', b'<PAGE\n\n', b'<PAGE\n\n', b'<PAGE\n\n', b'<PAGE\n']

In [755]:
re.findall(b'</PAGE\s+', page.content)

[]

In [768]:
re.findall(b'<PAGE[^>]', page.content)

[b'<PAGE\n', b'<PAGE\n', b'<PAGE\n', b'<PAGE\n', b'<PAGE\n']

In [756]:
'page'.encode('utf-8')

b'page'

In [772]:
print(fix_unmatched_page_tags(page.content))

b"-----BEGIN PRIVACY-ENHANCED MESSAGE-----\nProc-Type: 2001,MIC-CLEAR\nOriginator-Name: webmaster@www.sec.gov\nOriginator-Key-Asymmetric:\n MFgwCgYEVQgBAQICAf8DSgAwRwJAW2sNKK9AVtBzYZmr6aGjlWyK3XmZv3dTINen\n TWSM7vrzLADbmYQaionwg5sDW3P6oaM5D3tdezXMm7z1T+B+twIDAQAB\nMIC-Info: RSA-MD5,RSA,\n F9OBTNxQEgyxwS0QYqvNcTdxBQbMQ9JPA9jDfjshs7957Aj1yE03B4Q7pk1W+ijm\n 1aZtYaIYyQYFVRn/Y7N2Gg==\n\n<SEC-DOCUMENT>0000888002-11-000169.txt : 20110209\n<SEC-HEADER>0000888002-11-000169.hdr.sgml : 20110209\n<ACCEPTANCE-DATETIME>20110209083424\nACCESSION NUMBER:\t\t0000888002-11-000169\nCONFORMED SUBMISSION TYPE:\tSC 13G/A\nPUBLIC DOCUMENT COUNT:\t\t2\nFILED AS OF DATE:\t\t20110209\nDATE AS OF CHANGE:\t\t20110209\nGROUP MEMBERS:\t\tAXA\nGROUP MEMBERS:\t\tAXA ASSURANCES I.A.R.D. MUTUELLE\nGROUP MEMBERS:\t\tAXA ASSURANCES VIE MUTUELLE\nGROUP MEMBERS:\t\tAXA-EQUITABLE LIFE INSURANCE COMPANY\n\nSUBJECT COMPANY:\t\n\n\tCOMPANY DATA:\t\n\t\tCOMPANY CONFORMED NAME:\t\t\tUnum Group\n\t\tCENTRAL INDEX KEY:\t\t\t000000

In [766]:
re.search('A[^B]', 'ACD')

<re.Match object; span=(0, 2), match='AC'>

In [767]:
re.search('A[^B]', 'ABCD')

In [773]:
def fix_unmatched_page_tags(text):
    
    # First fix missing right dels
    
    start_tag_r_regex = '<PAGE[^>]' 
    end_tag_r_regex = '</PAGE[^>]'
    
    corrected_r_start = '<PAGE>\n' # Introduce newline in case newline replace by right del
    corrected_r_end = '</PAGE>\n'
    
    fixed_content = re.sub(start_tag_r_regex, corrected_r_start, text)
    fixed_content = re.sub(end_tag_r_regex, corrected_r_end, fixed_content)
    
    # Now fix missing left dels
    
    start_tag_l_regex = '[^<]PAGE>' 
    end_tag_l_regex = '[^<]/PAGE>'
    
    corrected_l_start = '\n<PAGE>' # Introduce newline in case newline replace by left del
    corrected_l_end = '\n</PAGE>'
    
    fixed_content = re.sub(start_tag_l_regex, corrected_l_start, fixed_content)
    fixed_content = re.sub(end_tag_l_regex, corrected_l_end, fixed_content)
    
    return(fixed_content)


def clean_text(text):
    
    # this function gets rid of unwanted characters and spaces
    
    result = fix_unmatched_page_tags(text) # First fix tags which are missing delimeters
    
    char_list = ['\xa0', '&#160;', '&#\n160;']
    
    for c in char_list:
        
        result = re.sub(c, ' ', result) 
        
    return(result)

In [769]:
def fix_unmatched_page_tags(page_content):
    
    # First fix missing right dels
    
    start_tag_r_regex = '<PAGE[^>]' 
    end_tag_r_regex = '</PAGE[^>]'
    
    corrected_r_start = '<PAGE>\n' # Introduce newline in case newline replace by right del
    corrected_r_end = '</PAGE>\n'
    
    fixed_content = re.sub(start_tag_r_regex.encode('utf-8'), corrected_r_start.encode('utf-8'), page.content)
    fixed_content = re.sub(end_tag_r_regex.encode('utf-8'), corrected_r_end.encode('utf-8'), fixed_content)
    
    # Now fix missing left dels
    
    start_tag_l_regex = '[^<]PAGE>' 
    end_tag_l_regex = '[^<]/PAGE>'
    
    corrected_l_start = '\n<PAGE>' # Introduce newline in case newline replace by left del
    corrected_l_end = '\n</PAGE>'
    
    fixed_content = re.sub(start_tag_l_regex.encode('utf-8'), corrected_r_start.encode('utf-8'), fixed_content)
    fixed_content = re.sub(end_tag_l_regex.encode('utf-8'), corrected_r_end.encode('utf-8'), fixed_content)
    
    return(fixed_content)

In [751]:
re.search(b'<[a-zA-Z0-9\-\/]+\s+', page.content).group(0)

b'<PAGE\n\n    '

In [744]:
soup = BeautifulSoup(page.content, 'html.parser')

In [749]:
soup.find('PAGE')

In [700]:
def has_exhibit_break(text, cover_page_start, signature_start):
    
    exhibit_st = get_exhibit_sec_start(text, cover_page_start, signature_start)
    
    if(exhibit_st != -1):
        
        return(True)
    
    else:
        
        return(False)

In [ ]:
def has_jumbled_order(text, form_type, has_table_of_contents = None, exhibit_start):
    
    # This function is defined to be true if there are item sections or signature sections in the parts of the
    # text which are before the last instance of an orthodox q1 of a cover page (hence "jumbled")
    # this can include cases where for instance, an order goes cover_page -> items -> signatures -> 
    # cover_page -> items -> signatures 
    
    
    result = False # Assume false till proven otherwise
    
    q1_starts = find_all_orthodox_q1_starts(text, exhibit_start)

    for i in range(len(q1_starts)):

        if(i == 0):
            
            if(has_table_of_contents is None):
                
                has_table_of_contents = has_table_of_contents(text, 0, q1_starts[i])
            

            segment = get_bounded_text(text, 0, q1_starts[i])
            
            if(has_table_of_contents):
                
                table_of_conts_index
            
            has_items = (get_item_section_start(segment, form_type) != -1)
            has_signatures = (get_signatures_sec_start(segment) != -1)

        else:

            segment = get_bounded_text(text, q1_starts[i-1], q1_starts[i])

            has_items = (get_item_section_start(segment, form_type) != -1)
            has_signatures = (get_signatures_sec_start(segment) != -1)

        if(has_items or has_signatures):

            result = True
            break

    return(result)

In [655]:
get_index_url('edgar/data/1106939/0000892303-00-000015.txt')

'https://www.sec.gov/Archives/edgar/data/1106939/000089230300000015/0000892303-00-000015-index.htm'

In [709]:
def find_all_orthodox_q1_starts(text, exhibit_start = None):
    
    # Finds all orthodox q1s BEFORE the exhibit section if it exists in the main text
    # (we do not want cover pages from exhibit parts)
    
    lst = []
    
    start, end = find_orthodox_cover_page_q1_start_end(text)
    
    while(start != -1):
        
        lst.append(start)
        p_start, p_end = find_orthodox_cover_page_q1_start_end(text, end)
        
        if(p_start != -1):
            
            start = end + p_start
            end = end + p_end
            
        else:
            
            start = -1
            
    if(exhibit_start is not None and exhibit_start != -1):
        
        result = [i for i in lst if i < exhibit_start]
        return(result)

    else:
        
        return(lst)
        

In [720]:
def get_item_section_start(text, form_type):    
    
    
    if(re.search('SC\s+13D', form_type)):
        
        re_list = ['\n\s*[#\.\;\:]?\s*1[#\.\;\:]?\s*security\s+and\s+issuer',
                   '\n\s*[#\.\;\:]?\s*2[#\.\;\:]?\s*identity\s+and\s+background',
                   '\n\s*[#\.\;\:]?\s*3[#\.\;\:]?\s*source\s+and\s+amount\s+of\s+funds',
                   '\n\s*[#\.\;\:]?\s*4[#\.\;\:]?\s*purpose\s+of\s+transaction',
                   '\n\s*[#\.\;\:]?\s*5[#\.\;\:]?\s*interest\s+in\s+securities\s+of\s+the\s+issuer',
                   '\n\s*[#\.\;\:]?\s*6[#\.\;\:]?\s*contracts,\s+arrangements,\s+understandings',
                   '\n\s*[#\.\;\:]?\s*7[#\.\;\:]?\s*material\s+to\s+be\s+filed\s+as\s+exhibits'
                  ]
                       
        
    elif(re.search('SC\s+13G', form_type)):
        
        re_list = ['\n\s*[#\.\;\:]?\s*1[#\.\;\:]?[\(]?\s*a[\)]?[#\.\;\:]?\s*name\s+of\s+issuer',
                   '\n\s*[#\.\;\:]?\s*1[#\.\;\:]?[\(]?\s*b[\)]?[#\.\;\:]?\s*address\s+of\s+issuer',
                   '\n\s*[#\.\;\:]?\s*2[#\.\;\:]?[\(]?\s*a[\)]?[#\.\;\:]?\s*name[s]?\s+of\s+person[s]?\s+filing',
                   '\n\s*[#\.\;\:]?\s*2[#\.\;\:]?[\(]?\s*b[\)]?[#\.\;\:]?\s*address\s+of\s+principle',
                   '\n\s*[#\.\;\:]?\s*2[#\.\;\:]?[\(]?\s*c[\)]?[#\.\;\:]?\s*citizenship',
                   '\n\s*[#\.\;\:]?\s*2[#\.\;\:]?[\(]?\s*d[\)]?[#\.\;\:]?\s*title\s+of\s+class',
                   '\n\s*[#\.\;\:]?\s*2[#\.\;\:]?[\(]?\s*e[\)]?[#\.\;\:]?\s*cusip\s+number',
                   '\n\s*[#\.\;\:]?\s*3[#\.\;\:]?\s*if\s+this\s+statement\s+is\s+filed\s+pursuant',
                   '\n\s*[#\.\;\:]?\s*4[#\.\;\:]?\s*ownership',
                   '\n\s*[#\.\;\:]?\s*5[#\.\;\:]?\s*ownership\s+of\s+five',
                   '\n\s*[#\.\;\:]?\s*6[#\.\;\:]?\s*ownership\s+of\s+more',
                   '\n\s*[#\.\;\:]?\s*7[#\.\;\:]?\s*identification\s+and\s+classification',
                   '\n\s*[#\.\;\:]?\s*8[#\.\;\:]?\s*identification\s+and\s+classification',
                   '\n\s*[#\.\;\:]?\s*9[#\.\;\:]?\s*notice\s+of\s+dissolution',
                   '\n\s*[#\.\;\:]?\s*10[#\.\;\:]?\s*certification'
                  ]
        
    else:
        
        # Not a Schedule 13D(/A) or 13G(/A)
        return(None)
   
    cover_pages_end = find_cover_pages_last_question(text)
    
    if(cover_pages_end):

        text_lowered = text[cover_pages_end:].lower()

        search = re.search('\n\s*item\s*[#\.\;\:]?\s*(?:[0-9])[#\.\;\:]?', text_lowered)
        
        if(search):
            
            search_str = search.group(0)
            
            num_to_item = re.search('item', search_str).start()

            return(cover_pages_end + search.start() + num_to_item)  
        
        else:
            return(-1) # ie. assume no Item section (this probably happens with 13D/A and 13G/A)
    
    else:
        
        text_lowered = text.lower() # Assume NO REPORTING PAGES (this happens, especially with 13D/A and 13G/A)

        search = re.search('\n\s*item\s*[#\.\;\:]?\s*[0-9][#\.\;\:]?', text_lowered)

        if(search):

            search_str = search.group(0)
            
            num_to_item = re.search('item', search_str).start()

            return(search.start() + num_to_item)  
        
        else: 
             
            result = -1 # ie. assume no Item section (this probably happens with 13D/A and 13G/A) if no matches in
                          # re_list
                
            for i in range(len(re_list)):
                
                search = re.search(re_list[i], text_lowered)
                
                if(search):
                    
                    search_str = search.group(0)
            
                    num_to_item = re.search('item', search_str).start()
                    
                    result = search.start() + num_to_item
                    break
            
            
            return(result)     

In [717]:
def get_signatures_sec_start(text, lower_bound = None, upper_bound = None):
    
    text_lowered = get_bounded_text(text, lower_bound, upper_bound).lower()
    
    regex1 = '\n\s*(signature|signatures)\s*(\.)?\n'
    
    regex2 = 'the\s+following\s+certification\s+shall\s+be\s+included\s+if\s+the\s+statement\s+is\s+filed\s+' + \
             'pursuant to rule 13d-1\(b\)'
    
    regex3 = 'by\s+signing\s+below\s+i\s+certify\s+that,\s+to\s+the\s+best\s+of\s+my\s+knowledge'

    regex4 = 'after\s+reasonable\s+inquiry\s+'
    
    regex_list = [regex1, regex2, regex3, regex4]
    
    regex_search = None
    
    for i in range(len(regex_list)):
        
        regex_search = re.search(regex_list[i], text_lowered)
        
        if(regex_search):
            
            break
        
    if(regex_search):
        
        if(lower_bound is not None):
        
            return(regex_search.start() + lower_bound)
        
        else:
            
            return(regex_search.start())
    
    else:
      
        return(-1)

In [710]:
def has_jumbled_order(text, form_type, exhibit_start):
    
    # This function is defined to be true if there are item sections or signature sections in the parts of the
    # text which are before the last instance of an orthodox q1 of a cover page (hence "jumbled")
    # this can include cases where for instance, an order goes cover_page -> items -> signatures -> 
    # cover_page -> items -> signatures 
    
    result = False # Assume false till proven otherwise
    
    q1_starts = find_all_orthodox_q1_starts(text, exhibit_start)

    for i in range(len(q1_starts)):

        if(i == 0):

            segment = get_bounded_text(text, 0, q1_starts[i])

        else:

            segment = get_bounded_text(text, q1_starts[i-1], q1_starts[i])

        has_items = (get_item_section_start(segment, form_type) != -1)
        has_signatures = (get_signatures_sec_start(segment) != -1)

        if(has_items or has_signatures):

            result = True
            break

    return(result)

In [711]:
has_jumbled_order(t, 'SC 13G', -1)

True

In [706]:
m = 0
for i in range(m):
    
    print('Hello')

In [ ]:
get_exhibit_sec_start(text, lower_bound = None, upper_bound = None)

In [675]:
def cover_page_start(text, lower_bound = None, cover_page_q1 = None, rep_q_as_items = None):
    
    if(rep_q_as_items is None):
        
        rep_q_as_items = is_rep_q_as_items(text, cover_page_q1)
    
    if(cover_page_q1 is None):
        
        cover_page_q1 = find_orthodox_cover_page_q1_start(text, rep_q_as_items)
        
    if(cover_page_q1 == -1):
        
        # No question 1 found, so no cover pages, return -1
        return(-1) 
        
    else:
        
        text_to_search = text[:cover_page_q1]
    
        text_reversed_and_raised = text_to_search[::-1].upper()

        regex = '(PISUC|LODES)\s*\n'

        search = re.search(regex, text_reversed_and_raised)

        if(search):

            num_back = search.end()

        else:

            num_back = 0

        start = cover_page_q1 - num_back

        return(start)

In [687]:
def cover_page_start_is_rep_as_q(text, form_type, lower_bound = None, upper_bound = None, rep_q_as_items = None):
    # this function is for cases where is_rep_q_as_items is True
    
    
    text_raised = get_bounded_text(text, lower_bound, upper_bound).upper()
        
    search = None
    
    if(rep_q_as_items is None):
        
        rep_q_as_items = is_rep_q_as_items(text)

    if(re.search('SC 13D', form_type)):
        
        if(rep_q_as_items):
            
            search = re.search('\n\s*ITEM\s+[\(\|]?\s?1\s?[\)\|]?\s*[:\.]{0,1}(.)+?' + \
                  '\n\s*ITEM\s+[\(\|]?\s?14\s?[\)\|]?\s*[:\.]{0,1}', text_raised, flags = re.DOTALL)
            
        else:
            return(-1)
        
    if(re.search('SC 13G', form_type)):
        
        if(rep_q_as_items):
            
            search = re.search('\n\s*ITEM\s+[\(\|]?\s?1\s?[\)\|]?\s*[:\.]{0,1}(.)+?' + \
                  '\n\s*ITEM\s+[\(\|]?\s?12\s?[\)\|]?\s*[:\.]{0,1}', text_raised, flags = re.DOTALL)
            
        else:
            return(-1)
            
    if(search):
        
        return(search.start() + lower_bound)
    
    else:
        
        return(-1)

In [676]:
q1_starts = find_all_orthodox_q1_starts(t)
q1_starts

[1669, 5796]

In [697]:
get_exhibit_sec_start(t, lower_bound = 5796, upper_bound = None)

-1

In [694]:
get_cover_page_starts(t, 'SC 13G', q1_starts)

[1669, 5796]

In [693]:
t

'\n\nSC 13G\n\n1\n\n\n\n\n\n               SECURITIES AND EXCHANGE COMMISSION\n                      WASHINGTON, D.C 20549\n\n                          SCHEDULE 13G\n\n            Under the Securities Exchange Act of 1934\n                      (Amendment No.     )*\n                                     ----\n\n\n                   THERMOVIEW INDUSTRIES, INC.\n     -------------------------------------------------------\n                        (Name of Issuer)\n\n                          COMMON STOCK\n     -------------------------------------------------------\n                 (Title of Class of Securities)\n\n                           883671 50 5\n     -------------------------------------------------------\n                         (CUSIP Number)\n\n                           12/31/1999\n     -------------------------------------------------------\n     (Date of Event Which Requires Filing of this Statement)\n\n     Check the appropriate box to designate the rule pursuant to\nwh

In [688]:
def find_cover_page_q1_start(text, form_type, lower_bound = None, upper_bound = None, rep_q_as_items = None):
    
    if(rep_q_as_items is None):
        
         rep_q_as_items = is_rep_q_as_items(get_bounded_text(text, lower_bound, upper_bound))
            
    if(rep_q_as_items):
        
        result = cover_page_start_is_rep_as_q(text, form_type, lower_bound, upper_bound, rep_q_as_items)
        
    else:
        
        result = find_orthodox_cover_page_q1_start(text, lower_bound, upper_bound)
        
    return(result)

In [692]:
def get_cover_page_starts(text, form_type, q1_starts = None):
    
    cover_page_starts = []
    
    if(q1_starts is not None and len(q1_starts) > 0):
        
        for i in range(len(q1_starts)):
            
            if(i == 0):
                
                start = cover_page_start(text, form_type, lower_bound = 0, cover_page_q1 = q1_starts[i],\
                                         rep_q_as_items = False)
                                                                
                cover_page_starts.append(start)
                
            else:
                
                start = cover_page_start(text, form_type, lower_bound = q1_starts[i-1], \
                                         cover_page_q1 = q1_starts[i], rep_q_as_items = False)
                                                                
                cover_page_starts.append(start)
        
        return(cover_page_starts)
    
    else:
        
        return(None)
      

In [ ]:
get_cover_page_starts(text, form_type, q1_starts = None)

In [690]:
def find_cover_page_q1_start(text, form_type, lower_bound = None, upper_bound = None, rep_q_as_items = None):
    
    if(rep_q_as_items is None):
        
         rep_q_as_items = is_rep_q_as_items(get_bounded_text(text, lower_bound, upper_bound))
            
    if(rep_q_as_items):
        
        result = cover_page_start_is_rep_as_q(text, form_type, lower_bound, upper_bound, rep_q_as_items)
        
    else:
        
        result = find_orthodox_cover_page_q1_start(text, lower_bound, upper_bound)
        
    return(result)

    
        
def cover_page_start(text, form_type, lower_bound = None, cover_page_q1 = None, rep_q_as_items = None):
    
    if(rep_q_as_items is None):
        
        rep_q_as_items = is_rep_q_as_items(text, cover_page_q1)
    
    if(cover_page_q1 is None):
        
        cover_page_q1 = find_cover_page_q1_start(text, form_type, lower_bound, cover_page_q1, rep_q_as_items)
        
    if(cover_page_q1 == -1):
        
        # No question 1 found, so no cover pages, return -1
        return(-1) 
        
    else:
        
        text_to_search = text[:cover_page_q1]
    
        text_reversed_and_raised = text_to_search[::-1].upper()

        regex = '(PISUC|LODES)\s*\n'

        search = re.search(regex, text_reversed_and_raised)

        if(search):

            num_back = search.end()

        else:

            num_back = 0

        start = cover_page_q1 - num_back

        return(start)

In [685]:
get_index_url('edgar/data/911631/0001193125-07-031855.txt')

'https://www.sec.gov/Archives/edgar/data/911631/000119312507031855/0001193125-07-031855-index.htm'

In [678]:
t[:1669]

'\n\nSC 13G\n\n1\n\n\n\n\n\n               SECURITIES AND EXCHANGE COMMISSION\n                      WASHINGTON, D.C 20549\n\n                          SCHEDULE 13G\n\n            Under the Securities Exchange Act of 1934\n                      (Amendment No.     )*\n                                     ----\n\n\n                   THERMOVIEW INDUSTRIES, INC.\n     -------------------------------------------------------\n                        (Name of Issuer)\n\n                          COMMON STOCK\n     -------------------------------------------------------\n                 (Title of Class of Securities)\n\n                           883671 50 5\n     -------------------------------------------------------\n                         (CUSIP Number)\n\n                           12/31/1999\n     -------------------------------------------------------\n     (Date of Event Which Requires Filing of this Statement)\n\n     Check the appropriate box to designate the rule pursuant to\nwh

In [679]:
t[1669:5796]

"\n\n\n\n1.   NAME OF REPORTING PERSONS I.R.S. IDENTIFICATION NO. OF ABOVE\n     PERSONS (ENTITIES ONLY)\n          Robert E. Anderson\n\n2.   CHECK THE APPROPRIATE BOX IF A MEMBER OF A GROUP.\n          (a) [x]\n          (b) [ ]\n\n3.   SEC USE ONLY\n\n4.   CITIZENSHIP OR PLACE OF ORGANIZATION\n\n          U.S.A.\n\n\nNUMBER OF                5.   SOLE VOTING POWER: 1,667\n SHARES\nBENEFICIALLY             6.   SHARED VOTING POWER:  0\n OWNED BY\nEACH REPORTING           7.   SOLE DISPOSITIVE POWER: 1,667\n PERSON WITH\n                         8.   SHARED DISPOSITIVE POWER: 0\n\n9.   AGGREGATE AMOUNT BENEFICIALLY OWNED BY EACH REPORTING PERSON\n\n          1,667 Shares\n\n\n10.  CHECK IF THE AGGREGATE AMOUNT IN ROW (9) EXCLUDES CERTAIN\n     SHARES\n     [x] Excludes 455,215 held by the Robert E. Anderson Trust\n     dtd 4/15/98\n\n\n11.  PERCENT OF CLASS REPRESENTED BY AMOUNT IN ROW (9)\n\n          .02\n\n12.  TYPE OF REPORTING PERSON\n\n          IN\n\n\nItem 1(a) Name of Issuer:

In [ ]:
def find_all_cover_page_starts(text, q1_starts = None, )

In [ ]:
def is_rep_q_as_items(text, q1_start = None):
    
    # this function works on the basis that the the actual "items" in the real item section does not go beyond
    # Item 10. So by searching for Item 12 or 14 (in SC 13G and SC 13D respectively), we can detect whether
    # a filer has erroneously labelled the questions in the reporting pages as items
    
    if(q1_start is None):
      
        q1_start = find_orthodox_cover_page_q1_start(text)
    
    if(q1_start == -1):
    
        text_raised = text.upper()
        
        regex = '\n\s*ITEM\s*(\.)?\s*[\(]?\s*1[24]\s*[\)]?(\.)?'
        
        search = re.search(regex, text_raised)
        
        if(search):
            return(True)
        else:
            return(False)
            
    else:
      
        return(False)

In [674]:
t[1669:5796]

"\n\n\n\n1.   NAME OF REPORTING PERSONS I.R.S. IDENTIFICATION NO. OF ABOVE\n     PERSONS (ENTITIES ONLY)\n          Robert E. Anderson\n\n2.   CHECK THE APPROPRIATE BOX IF A MEMBER OF A GROUP.\n          (a) [x]\n          (b) [ ]\n\n3.   SEC USE ONLY\n\n4.   CITIZENSHIP OR PLACE OF ORGANIZATION\n\n          U.S.A.\n\n\nNUMBER OF                5.   SOLE VOTING POWER: 1,667\n SHARES\nBENEFICIALLY             6.   SHARED VOTING POWER:  0\n OWNED BY\nEACH REPORTING           7.   SOLE DISPOSITIVE POWER: 1,667\n PERSON WITH\n                         8.   SHARED DISPOSITIVE POWER: 0\n\n9.   AGGREGATE AMOUNT BENEFICIALLY OWNED BY EACH REPORTING PERSON\n\n          1,667 Shares\n\n\n10.  CHECK IF THE AGGREGATE AMOUNT IN ROW (9) EXCLUDES CERTAIN\n     SHARES\n     [x] Excludes 455,215 held by the Robert E. Anderson Trust\n     dtd 4/15/98\n\n\n11.  PERCENT OF CLASS REPRESENTED BY AMOUNT IN ROW (9)\n\n          .02\n\n12.  TYPE OF REPORTING PERSON\n\n          IN\n\n\nItem 1(a) Name of Issuer:

In [667]:
start, end = find_orthodox_cover_page_q1_start_end(t)

In [668]:
end

1695

In [671]:
find_orthodox_cover_page_q1_start_end(text, 1695)

(805, 854)

In [662]:
def find_orthodox_cover_page_q1_start_end(text, lower_bound = None, upper_bound = None):

    text_raised = get_bounded_text(text, lower_bound, upper_bound).upper()

    regex0 = '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*(.){0,50}\s*' + \
             'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?\s+(?:REPORTING|ABOVE|REPORT)'
    
    regex1 = '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*(.){0,50}\s*' + \
             'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?(\s+(?:REPORTING|ABOVE|REPORT))?\s+' + \
             'PERSON[\(]?[S]?[\)]?'

    regex2 = '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*(.){0,50}\s*' + \
             'NAME[\(]?[S]?[\)]?\s+(?:AND|OF|OR)(\s+I[\.]?R[\.]?S)?\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?\.'
    
    regex3 = '(NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?(\s+(?:REPORTING|ABOVE|REPORT))?\s+' + \
             'PERSON[\(]?[S]?[\)]?\s+)?[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.\|]{0,1}\s*' + \
             'I\.R\.S\.\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?\.\s+(?:OF|OR)\s+(\s+THE)?ABOVE\s+' + \
             'PERSON[\(]?[S]?[\)]?'
    
    regex4 = '(NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?(\s+(?:REPORTING|ABOVE|REPORT))?\s+' + \
             'PERSON[\(]?[S]?[\)]?\s+)?[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.\|]{0,1}\s*S\.S\.\s+OR\s+' + \
             'I\.R\.S\.\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?\.\s+(?:OF|OR)(\s+THE)?\s+' + \
             'ABOVE\s+PERSON[\(]?[S]?[\)]?'
    
    regex5 = '[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*' + \
             'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?\s+FILING\s+(?:PARTY|PARTIES|PERSON|PERSONS)'
    
    regex6 = 'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?(\s+(?:REPORTING|ABOVE|REPORT))?\s+' + \
             'PERSON[\(]?[S]?[\)]?\s+(S\.S\.\s+OR\s+I\.R\.S\.\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?\.\s+' + \
             '(?:OF|OR)(\s+THE)?\s+ABOVE\s+PERSON[\(]?[S]?[\)]?)?' + \
             '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?'
    
    
    regex7 = '\n\s*[\(\|]?\s1\s?[\)\|]?\s*[:\.]{0,1}([A-Z0-9\s\.\(\)_]?)+?' + \
             '[\(\|]?\s2\s?[\)\|]?\s*[:\.]{0,1}([A-Z0-9\s\.\(\)_]?)+?' + \
             '[\(\|]?\s3\s?[\)\|]?\s*[:\.]{0,1}\s+(.)+\s*\(SEC USE ONLY\)'
    
    regex8 = 'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?\s+(?:REPORTING|ABOVE|REPORT)'
    
    regex9 = 'NAME[\(]?[S]?[\)]?\s+(?:AND|OF|OR)(\s+I[\.]?R[\.]?S)?\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?.'
    
    regex10 = '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*(.){0,50}\s*' + \
              'NAME[\(]?[S]?[\)]?\s+(?:AND|OF|OR)(\s+I[\.]?R[\.]?S)?\s+NUMBER\s+OF\s+REPORTING\s+PERSON[S]?'
    
    
    regex_list = [regex0, regex1, regex2, regex3, regex4, regex5, regex6, regex7, regex8, regex9, regex10]
    
    
    for i in range(len(regex_list)):
        
        search = re.search(regex_list[i], text_raised)
        
        if(search):
            
            break

    if(search):
        
        return(search.start(), search.end())
    
    else:
        
        return(-1, -1)   

In [656]:
t = get_main_doc_text(indexes_df.loc[i, 'file_name'], indexes_df.loc[i, 'document'], directory)

In [657]:
t[1669:6676]

"\n\n\n\n1.   NAME OF REPORTING PERSONS I.R.S. IDENTIFICATION NO. OF ABOVE\n     PERSONS (ENTITIES ONLY)\n          Robert E. Anderson\n\n2.   CHECK THE APPROPRIATE BOX IF A MEMBER OF A GROUP.\n          (a) [x]\n          (b) [ ]\n\n3.   SEC USE ONLY\n\n4.   CITIZENSHIP OR PLACE OF ORGANIZATION\n\n          U.S.A.\n\n\nNUMBER OF                5.   SOLE VOTING POWER: 1,667\n SHARES\nBENEFICIALLY             6.   SHARED VOTING POWER:  0\n OWNED BY\nEACH REPORTING           7.   SOLE DISPOSITIVE POWER: 1,667\n PERSON WITH\n                         8.   SHARED DISPOSITIVE POWER: 0\n\n9.   AGGREGATE AMOUNT BENEFICIALLY OWNED BY EACH REPORTING PERSON\n\n          1,667 Shares\n\n\n10.  CHECK IF THE AGGREGATE AMOUNT IN ROW (9) EXCLUDES CERTAIN\n     SHARES\n     [x] Excludes 455,215 held by the Robert E. Anderson Trust\n     dtd 4/15/98\n\n\n11.  PERCENT OF CLASS REPRESENTED BY AMOUNT IN ROW (9)\n\n          .02\n\n12.  TYPE OF REPORTING PERSON\n\n          IN\n\n\nItem 1(a) Name of Issuer:

In [632]:
def get_main_doc_text(file_name, document, directory):
  
    full_text = clean_text(get_file_text_from_directory(file_name, document, directory))
    soup = BeautifulSoup(full_text, 'html5lib')
    main_doc_text = soup.find(["document", "DOCUMENT"]).get_text('\n')
    
    return(main_doc_text)

In [633]:
get_index_url(sc_13d_df.loc[i, 'file_name'])

'https://www.sec.gov/Archives/edgar/data/317540/000002134413000014/0000021344-13-000014-index.htm'

In [646]:
t = get_main_doc_text(sc_13d_df.loc[i, 'file_name'], sc_13d_df.loc[i, 'document'], directory)

In [625]:
t[22955:]

'\n\n\n\n\n \n\nAPPENDIX A\n\n \n\nName, business address, present principal occupation or employment and place of citizenship of the directors and executive officers of\n\n \n\nFONDS STRATÉGIQUE D’INVESTISSEMENT\n\n \n\nThe name, business address and present principal occupation or employment of each of the directors and executive officers of Fonds Stratégique d’Investissement are set forth below. The business address of each director and executive officer is Fonds Stratégique d’Investissement c/o 56, rue de Lille, 75007 Paris, France. Unless otherwise indicated, each director and executive officer is a citizen of France.\n\n \n\nDIRECTORS\n\n \n\n\n\n\nName\n\n\nPresent Principal Occupation or Employment\n\n\n\n\n \n\n\n\n\n\nJEAN-PIERRE JOUYET\n\n\nChairman of the Board of Directors\n\n\n\n\n \n\n\n\n\n\nJEAN-YVES GILET\n\n\nChief Executive Officer\n\n\n\n\n \n\n\n\n\n\nPHILIPPE BRAIDY\n\n\nSenior Vice President\n\n\n\n\n \n\n\n\n\n\nDAVID AZEMA\n\n\nDirector\n\n\n\n\n \n\n\n\n\n\nF

In [620]:
t[11654:]

'\n\xa0\n\n\n\n\n\n\n\n Exhibit\n\xa0\xa0\n Description of Exhibit\n\n\n\n\n99.1\n\xa0\xa0\nJoint Filing Agreement.\n\n\n\n\n99.2\n\xa0\xa0\nPower of Attorney.\n \xa0 Page 9 of 30\n\n\n\n\n\n\n SCHEDULE I  EXECUTIVE OFFICERS AND DIRECTORS OF  REPORTING PERSONS \nThe following sets forth the name and present principal occupation of each executive officer and director of Bank of America Corporation\nand Bank of America, N.A. The business address of each of the executive officers and directors of Bank of America Corporation is Bank of America Corporate Center, 100 North Tryon Street, Charlotte, North Carolina 28255. The business address of each\nof the executive officers and directors of Bank of America, N.A. is 101 South Tryon Street, Charlotte, North Carolina 28255.  \xa0\n\n\n\n\n\n\n\n\n\n Name\n\xa0\xa0\n Position with Bank of\nAmerica Corporation\n\xa0\xa0\n Principal Occupation\n\nBrian T. Moynihan\n\xa0\xa0\nChief Executive Officer, President and Director\n\xa0\xa0\nChief Executiv

In [616]:
def get_key_indices(file_name, document, form_type, directory):
    
    key_info = {}
    
    text = get_main_doc_text(file_name, document, directory)
    
    key_info['file_name'] = [file_name]
    key_info['document'] = [document]
    key_info['form_type'] = [form_type]
    key_info['title_page_end_lower_bound'] = [find_title_page_end(text)]
    key_info['cover_page_q1_start'] = [find_orthodox_cover_page_q1_start(text)]
    key_info['is_rep_q_as_items'] = [is_rep_q_as_items(text, key_info['cover_page_q1_start'][0])]
    if(key_info['is_rep_q_as_items'][0]):
        
        key_info['cover_page_q1_start'] = [cover_page_start_is_rep_as_q(text, form_type, \
                                                lower_bound = key_info['title_page_end_lower_bound'][0])]
    key_info['num_cusip_sedol_b_q1'] = [num_cusip_sedol_before_index(text, \
                                                                    key_info['cover_page_q1_start'][0])]
    key_info['cover_page_start'] = [cover_page_start(text, form_type, \
            cover_page_q1 = key_info['cover_page_q1_start'][0], \
                                                     rep_q_as_items = key_info['is_rep_q_as_items'][0])]
    
    
    key_info['item_section_start'] = [get_item_section_start(text, form_type)]
    key_info['cover_page_last_q_end'] = [find_cover_pages_last_question(text, \
                        key_info['is_rep_q_as_items'][0], key_info['cover_page_q1_start'][0], \
                                                                        key_info['item_section_start'][0])]
    key_info['cover_page_end'] = [find_cover_pages_end(text, form_type, key_info['is_rep_q_as_items'][0],\
                                key_info['cover_page_last_q_end'][0], key_info['item_section_start'][0])]
    
    key_info['num_cusip_b_items'] = [num_cusip_sedol_before_index(text, \
                                                                  index = key_info['item_section_start'][0])]
    key_info['signature_start'] = [get_signatures_sec_start(text, key_info['item_section_start'][0])]
    key_info['exhibit_start'] = [get_exhibit_sec_start(text, key_info['signature_start'][0])]
    key_info['main_text_length'] = [len(text)]
    
    if(key_info['cover_page_start'][0] != -1 and key_info['item_section_start'][0] != -1\
      and key_info['item_section_start'][0] > key_info['cover_page_start'][0]):
        
        amend_l_bound = key_info['cover_page_start'][0]
        
    else:
        
        amend_l_bound = None
    
    key_info['amendment_statement_start'] = find_amendment_statement_start(text, form_type, \
                    lower_bound = amend_l_bound, upper_bound = key_info['item_section_start'][0])

    
    
    key_info_df = pd.DataFrame(key_info)
    
    return(key_info_df)

In [576]:
get_index_url(sc_13d_df.loc[i, 'file_name'])

'https://www.sec.gov/Archives/edgar/data/1363679/000126645406000277/0001266454-06-000277-index.htm'

In [582]:
find_title_page_end(t)

KeyboardInterrupt: 

In [577]:
sc_13d_df.loc[i, 'document']

'0001266454-06-000277.txt'

In [586]:
text_raised = t.upper()

regex0 = '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*(.){0,50}\s*' + \
         'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?\s+(?:REPORTING|ABOVE|REPORT)'

regex1 = '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*(.){0,50}\s*' + \
         'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?(\s+(?:REPORTING|ABOVE|REPORT))?\s+' + \
         'PERSON[\(]?[S]?[\)]?'

regex2 = '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*(.){0,50}\s*' + \
         'NAME[\(]?[S]?[\)]?\s+(?:AND|OF|OR)(\s+I[\.]?R[\.]?S)?\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?\.'

regex3 = '(NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?(\s+(?:REPORTING|ABOVE|REPORT))?\s+' + \
         'PERSON[\(]?[S]?[\)]?\s+)?[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.\|]{0,1}\s*' + \
         'I\.R\.S\.\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?\.\s+(?:OF|OR)\s+(\s+THE)?ABOVE\s+' + \
         'PERSON[\(]?[S]?[\)]?'

regex4 = '(NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?(\s+(?:REPORTING|ABOVE|REPORT))?\s+' + \
         'PERSON[\(]?[S]?[\)]?\s+)?[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.\|]{0,1}\s*S\.S\.\s+OR\s+' + \
         'I\.R\.S\.\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?\.\s+(?:OF|OR)(\s+THE)?\s+' + \
         'ABOVE\s+PERSON[\(]?[S]?[\)]?'

regex5 = '[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*' + \
         'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?\s+FILING\s+(?:PARTY|PARTIES|PERSON|PERSONS)'

regex6 = 'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?(\s+(?:REPORTING|ABOVE|REPORT))?\s+' + \
         'PERSON[\(]?[S]?[\)]?\s+(S\.S\.\s+OR\s+I\.R\.S\.\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?\.\s+' + \
         '(?:OF|OR)(\s+THE)?\s+ABOVE\s+PERSON[\(]?[S]?[\)]?)?' + \
         '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?'


regex7 = '\n\s*[\(\|]?\s1\s?[\)\|]?\s*[:\.]{0,1}([A-Z0-9\s\.\(\)_]?)+?' + \
         '[\(\|]?\s2\s?[\)\|]?\s*[:\.]{0,1}([A-Z0-9\s\.\(\)_]?)+?' + \
         '[\(\|]?\s3\s?[\)\|]?\s*[:\.]{0,1}\s+(.)+\s*\(SEC USE ONLY\)'

regex8 = 'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?\s+(?:REPORTING|ABOVE|REPORT)'

regex9 = 'NAME[\(]?[S]?[\)]?\s+(?:AND|OF|OR)(\s+I[\.]?R[\.]?S)?\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?.'

regex10 = '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*(.){0,50}\s*' + \
          'NAME[\(]?[S]?[\)]?\s+(?:AND|OF|OR)(\s+I[\.]?R[\.]?S)?\s+NUMBER\s+OF\s+REPORTING\s+PERSON[S]?'

In [587]:
re.search(regex0, text_raised)

<re.Match object; span=(4920, 5198), match='\n            \n          \n\n      \n    \n    \>

In [588]:
regex_list = [regex0, regex1, regex2, regex3, regex4, regex5, regex6, regex7, regex8, regex9, regex10]
    
start = None

for i in range(len(regex_list)):

    search = re.search(regex_list[i], text_raised)

    if(search):

        if(start is None):

            start = search.start()

        else:

            new = search.start()

            if(new < start):

                start = new            

KeyboardInterrupt: 

In [589]:
def find_orthodox_cover_page_q1_start(text):

    text_raised = text.upper()

    regex0 = '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*(.){0,50}\s*' + \
             'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?\s+(?:REPORTING|ABOVE|REPORT)'
    
    regex1 = '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*(.){0,50}\s*' + \
             'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?(\s+(?:REPORTING|ABOVE|REPORT))?\s+' + \
             'PERSON[\(]?[S]?[\)]?'

    regex2 = '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*(.){0,50}\s*' + \
             'NAME[\(]?[S]?[\)]?\s+(?:AND|OF|OR)(\s+I[\.]?R[\.]?S)?\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?\.'
    
    regex3 = '(NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?(\s+(?:REPORTING|ABOVE|REPORT))?\s+' + \
             'PERSON[\(]?[S]?[\)]?\s+)?[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.\|]{0,1}\s*' + \
             'I\.R\.S\.\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?\.\s+(?:OF|OR)\s+(\s+THE)?ABOVE\s+' + \
             'PERSON[\(]?[S]?[\)]?'
    
    regex4 = '(NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?(\s+(?:REPORTING|ABOVE|REPORT))?\s+' + \
             'PERSON[\(]?[S]?[\)]?\s+)?[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.\|]{0,1}\s*S\.S\.\s+OR\s+' + \
             'I\.R\.S\.\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?\.\s+(?:OF|OR)(\s+THE)?\s+' + \
             'ABOVE\s+PERSON[\(]?[S]?[\)]?'
    
    regex5 = '[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*' + \
             'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?\s+FILING\s+(?:PARTY|PARTIES|PERSON|PERSONS)'
    
    regex6 = 'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?(\s+(?:REPORTING|ABOVE|REPORT))?\s+' + \
             'PERSON[\(]?[S]?[\)]?\s+(S\.S\.\s+OR\s+I\.R\.S\.\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?\.\s+' + \
             '(?:OF|OR)(\s+THE)?\s+ABOVE\s+PERSON[\(]?[S]?[\)]?)?' + \
             '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?'
    
    
    regex7 = '\n\s*[\(\|]?\s1\s?[\)\|]?\s*[:\.]{0,1}([A-Z0-9\s\.\(\)_]?)+?' + \
             '[\(\|]?\s2\s?[\)\|]?\s*[:\.]{0,1}([A-Z0-9\s\.\(\)_]?)+?' + \
             '[\(\|]?\s3\s?[\)\|]?\s*[:\.]{0,1}\s+(.)+\s*\(SEC USE ONLY\)'
    
    regex8 = 'NAME[\(]?[S]?[\)]?\s+(?:OF|OR)(\s+THE)?\s+(?:REPORTING|ABOVE|REPORT)'
    
    regex9 = 'NAME[\(]?[S]?[\)]?\s+(?:AND|OF|OR)(\s+I[\.]?R[\.]?S)?\s+IDENTIFICATION\s+NO[\(]?[S]?[\)]?.'
    
    regex10 = '\s+[\(\|]?\s*[1L]\s*[\)\|]?\s*[:\.]{0,1}(\s*[\(\|]\s*A\s*[\)\|])?\s*(.){0,50}\s*' + \
              'NAME[\(]?[S]?[\)]?\s+(?:AND|OF|OR)(\s+I[\.]?R[\.]?S)?\s+NUMBER\s+OF\s+REPORTING\s+PERSON[S]?'
    
    
    regex_list = [regex0, regex1, regex2, regex3, regex4, regex5, regex6, regex7, regex8, regex9, regex10]
    
    
    for i in range(len(regex_list)):
        
        search = re.search(regex_list[i], text_raised)
        
        if(search):
            
            break

    if(search):
        
        return(search.start())
    
    else:
        
        return(-1)   

In [585]:
find_orthodox_cover_page_q1_start(t)

KeyboardInterrupt: 

In [578]:
t = get_main_doc_text(sc_13d_df.loc[i, 'file_name'], sc_13d_df.loc[i, 'document'], directory)

In [580]:
print(t)


SC 13D
1
broadcaster_sc13d-061206.htm
SCHEDULE 13-D


  
    
      Schedule 13-D






  
    
      

          
            
              UNITED
                STATES
              SECURITIES
                AND EXCHANGE COMMISSION
              Washington,
                D.C. 20549
            
          

      
    
    
    SCHEDULE
      13D
    
    Under
      the Securities Exchange Act of 1934
    (Amendment
      No. _____)*
     
     
    
      

          
            
              BROADCASTER,
                INC.
              (formerly
                International Microcomputer Software, Inc.)
            
          
          
            
              (Name
                of Issuer)
            
          
          
             
          
          
            
              Common
                Stock, no par value
            
          
          
            
              (Title
                of Class of Securities)
            
          
    

In [573]:
t[4169:4402]

'____________________________________________________________________________\n\n\n\n------------------------------------------------------------------------\n\n\nCUSIP NO.  20037A201                                             Page 2 of 4\n\n'

In [547]:
t[34441:]

'\n\n\xa0\n\n\xa0\n\nExhibit\n  A\n\nJOINT FILING UNDERTAKING\n\n\xa0\n\nThe\n  undersigned, being duly authorized thereunto, hereby execute this agreement\n  as an exhibit to this Schedule 13D to evidence the agreement of the below-named\n  parties, in accordance with the rules promulgated pursuant to the Securities\n  Exchange Act of 1934, to file this Schedule jointly on behalf of each such party.\n\n\xa0\n\nDated:  May 19, 2006\n\n\xa0\n\nPrides Capital Partners, L.L.C.\n\n\xa0\n\n\n\n    \n        \n            By: \n        \n            /s/ Murray A. Indick  \n\n------------------------ \n\nMurray A. Indick\n\nManaging Member\n\n\xa0\n\n\xa0\n\n\n\n    \n        \n            Kevin A. Richardson, II  \n\n\xa0\n\n\n\n    \n        \n            /s/ Murray A. Indick   \n        \n            By: \n        \n            /s/ Murray A. Indick \n        \n            \xa0 \n    \n        \n            ------------------------   \n        \n            ------------------------  \n    \

In [558]:
def find_cover_pages_last_question(text, rep_q_as_items = None,  lower_bound = None, upper_bound = None):
    
    if(rep_q_as_items is None):
        
        rep_q_as_items = is_rep_q_as_items(text)
    
    if(lower_bound is not None and lower_bound > 0):
        
        if(upper_bound is not None and upper_bound > 0):
            
            text_raised = text.upper()[lower_bound:upper_bound]
            
        else:
            
            text_raised = text.upper()[lower_bound:]
            
    else:
        
        if(upper_bound is not None and upper_bound > 0):
            
            text_raised = text.upper()[:upper_bound]
            
        else:
            
            text_raised = text.upper() 
    
    if(rep_q_as_items):
        
        regex = 'ITEM\s*[\(\|]?\s*1[24]\s*[\)\|]?\s*[\:\.]?'
        
    else:
    
        regex = '[\(\|]?\s*1[24]\s*[\)\|]?\s*[\:\.]?\s*TYPE[S]?\s+OF\s+REPORTING\s+PERSON[S]?[\*]?\s*(\(SEE\s+INSTRUCTIONS\))?'

    search = re.search(regex, text_raised)

    if(search is None):
        return(None)
    
    end = 0

    while(search is not None):
        
        end = end + search.end()
        search = re.search(regex, text_raised[end:])

    if(lower_bound is not None and lower_bound > 0):
        
        end = end + lower_bound
        
    return(end)       

In [509]:
indexes

NameError: name 'indexes' is not defined

In [554]:
t = get_main_doc_text(sc_13d_df.loc[i, 'file_name'], sc_13d_df.loc[i, 'document'], directory)

In [534]:
print(t[3514:])



       Signature

       After reasonable inquiry and to the best of my knowledge and
       belief, I certify that the information set forth in this
       statement is true, complete and correct.

                 02/15/2010
                 Date

                 /s/ Dale M. Brown
                 Signature

                 Dale M. Brown, President
                 Name/Title






In [196]:
i = 1112
get_key_indices(indexes_df.loc[i, 'file_name'], indexes_df.loc[i, 'document'], \
                indexes_df.loc[i, 'form_type'], directory)

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start
0,edgar/data/101507/0001051042-11-000006.txt,0001051042-11-000006.txt,SC 13G,1598,1716,False,2,1660,2576,2516,2525,3,5727,-1


In [195]:
indexes_df[indexes_df['file_name'] == 'edgar/data/101507/0001051042-11-000006.txt']

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start,success
1112,edgar/data/101507/0001051042-11-000006.txt,0001051042-11-000006.txt,SC 13G,1598.0,-1.0,False,5.0,-1.0,2576.0,2516.0,2529.0,3.0,5727.0,-1.0,True


In [197]:
no_cover_start = pd.read_sql("SELECT * FROM edgar.sc13dg_indexes WHERE cover_page_q1_start = -1 AND cover_page_start = -1", engine)
no_cover_start

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start,success
0,edgar/data/101507/0001051042-11-000006.txt,0001051042-11-000006.txt,SC 13G,1598,-1,False,5,-1,2576.0,2516,2529,3,5727.0,-1,True
1,edgar/data/741561/0000741561-00-000002.txt,0000741561-00-000002.txt,SC 13G,1414,-1,False,2,-1,1414.0,1412,-1,1,3671.0,-1,True
2,edgar/data/1051042/0001051042-13-000001.txt,0001051042-13-000001.txt,SC 13G,1596,-1,False,5,-1,2580.0,2520,2533,3,5714.0,-1,True
3,edgar/data/1051042/0001051042-13-000004.txt,0001051042-13-000004.txt,SC 13G,1590,-1,False,5,-1,2574.0,2514,2527,3,5705.0,-1,True
4,edgar/data/1051042/0001051042-13-000006.txt,0001051042-13-000006.txt,SC 13G,1595,-1,False,5,-1,2579.0,2519,2532,3,5713.0,-1,True
5,edgar/data/1051042/0001051042-13-000007.txt,0001051042-13-000007.txt,SC 13G,1595,-1,False,5,-1,2575.0,2515,2528,3,5701.0,-1,True
6,edgar/data/1051042/0001051042-13-000008.txt,0001051042-13-000008.txt,SC 13G,1589,-1,False,5,-1,2573.0,2513,2526,3,5704.0,-1,True
7,edgar/data/1051042/0001051042-13-000009.txt,0001051042-13-000009.txt,SC 13G,1598,-1,False,5,-1,2592.0,2532,2545,3,5759.0,-1,True
8,edgar/data/1051042/0001051042-13-000010.txt,0001051042-13-000010.txt,SC 13G,1602,-1,False,5,-1,2590.0,2529,2542,3,5739.0,-1,True
9,edgar/data/1051042/0001051042-13-000011.txt,0001051042-13-000011.txt,SC 13G,1594,-1,False,5,-1,2572.0,2512,2525,3,5695.0,-1,True


In [510]:
indexes_df[indexes_df['document'] == '0000950123-10-013272.txt']

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start,success
488,edgar/data/1201633/0000950123-10-013272.txt,0000950123-10-013272.txt,SC 13G/A,1230.0,1476.0,False,2.0,1400.0,9937.0,8653.0,8683.0,7.0,1368.0,-1.0,True


In [521]:
indexes_df[indexes_df['document'] == '0000892303-00-000015.txt']

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start,success
9168,edgar/data/1106939/0000892303-00-000015.txt,0000892303-00-000015.txt,SC 13G,1666.0,1666.0,False,1.0,1666.0,6671.0,6669.0,-1.0,2.0,5403.0,-1.0,True


In [512]:
i = 488
get_key_indices(indexes_df.loc[i, 'file_name'], indexes_df.loc[i, 'document'], \
                indexes_df.loc[i, 'form_type'], directory)

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start
0,edgar/data/1201633/0000950123-10-013272.txt,0000950123-10-013272.txt,SC 13G/A,1344,1625,False,2,1542,10555,9190,9241,7,1510,-1


In [513]:
t = get_main_doc_text(indexes_df.loc[i, 'file_name'], indexes_df.loc[i, 'document'], directory)

In [516]:
t[:1542]

'\nSC 13G/A\n1\na55129sc13gza.htm\nSC 13G/A\n\n\n\nsc13gza\n\n\n\nTable of Contents\n\n\n\n\xa0\n\xa0\n\n\n\n\nUNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\n\n\nWashington, D.C. 20549\n\n\nSCHEDULE 13G/A\n\n\nUnder\nthe Securities Exchange Act of 1934\n\xa0\n(Amendment No.\xa06)*\n\nBROADCOM CORPORATION\n\xa0\n\n\n(Name of Issuer)\n\n\nClass\xa0A common stock, par value $0.0001\n\xa0\n(Title of Class of Securities)\n\n111320107\n\xa0\n(CUSIP Number)\n\nDecember\xa031, 2009\n\xa0\n(Date of Event Which Requires Filing of this Statement)\n\nCheck the appropriate box to designate the rule pursuant to which this Schedule is filed:\n\n\n\n\n\n\n\n    \xa0\n\n    o\n    \xa0\n    Rule\xa013d-1 (b)\n\n\n\n    \xa0\n\n    \xa0\n\n    o\n    \xa0\n    Rule\xa013d-1 (c)\n\n\n\n    \xa0\n\n    \xa0\n\n    þ\n    \xa0\n    Rule\xa013d-1 (d)\n\n\n\n\n\n\n\n\n\n    \n    \n    \n\n\n\n    \xa0\n\n\n    *\n    \xa0\n    The remainder of this cover page shall be filled out for a reporting person’s

In [519]:
get_index_url('edgar/data/1429074/0000950123-11-006511.txt')

'https://www.sec.gov/Archives/edgar/data/1429074/000095012311006511/0000950123-11-006511-index.htm'

In [520]:
get_index_url('edgar/data/926660/0000950134-07-014185.txt')

'https://www.sec.gov/Archives/edgar/data/926660/000095013407014185/0000950134-07-014185-index.htm'

In [515]:
print(t[1542:9241])







 
 
 
 
 
 
 
 
 
 
 










CUSIP No.
 
111320107 
 
Page
 
2 
 
of 
 
12 






 



 
 
 
 
 
 





1
 
NAME OF REPORTING PERSON:
I.R.S. Identification Nos. of above persons(entities only):HENRY SAMUELI, PH.D.


 
 
 



 
 
 




2
 
CHECK THE APPROPRIATE BOX IF A MEMBER OF A GROUP:



 
(a)   o



 
(b)   o



 
 
 



3
 
SEC USE ONLY:


 
 


 
 



 
 
 



4
 
CITIZENSHIP OR PLACE OF ORGANIZATION:


 
 


 
United States



 
 
 
 



 
5
 
SOLE VOTING POWER:


 
 
 


NUMBER OF
 
30,556,607



 
 
 
 



SHARES
6
 
SHARED VOTING POWER:


BENEFICIALLY
 
 


OWNED BY
 
-0-



 
 
 
 



EACH
7
 
SOLE DISPOSITIVE POWER:


REPORTING
 
 


PERSON
 
30,556,607



 
 
 
 



WITH:
8
 
SHARED DISPOSITIVE POWER:


 
 
 


 
 
-0-



 
 
 



9
 
AGGREGATE AMOUNT BENEFICIALLY OWNED BY EACH REPORTING PERSON:


 
 


 
30,556,607



 
 
 



10
 
CHECK IF THE AGGREGATE AMOUNT IN ROW (9) EXCLUDES CERTAIN SHARES:


 
 


 
o



 
 
 



11
 
PERCENT OF CLASS REPRESENTED BY AMOUNT

In [511]:
i = 488
get_key_indices(no_cover_start.loc[i, 'file_name'], no_cover_start.loc[i, 'document'], \
                no_cover_start.loc[i, 'form_type'], directory)

KeyError: 'the label [488] is not in the [index]'

In [199]:
get_index_url('edgar/data/741561/0000741561-00-000002.txt')

'https://www.sec.gov/Archives/edgar/data/741561/000074156100000002/0000741561-00-000002-index.htm'

In [180]:
t = get_main_doc_text(sc_13d_df.loc[0, 'file_name'], sc_13d_df.loc[0, 'document'], directory)
t[2657:2700]

'CUSIP No. 233916-20-4\n13D\nPage 3 of 6 Pages'

In [173]:
answer_regex = '\s+([A-Z0]{1}[;,\.]?[A-Z0]{1}[;,\.]?)*([A-Z0]{1}[;,\.]?[A-Z0]{1}[;,\.]?)\s*\n?'

In [174]:
re.search(answer_regex, t[2600:2700])

<re.Match object; span=(17, 23), match='\nTYPE '>

In [157]:
def find_regex1_end(text):
    
    text_lowered = text.lower()

    regex1 = 'but\s+shall\s+be\s+subject\s+to\s+all\s+other\s+provisions\s+of\s+the\s+act\s*' + \
              '(\(however,\s+see\s+the\s+notes\s*\))?(\.)?'

    end = find_last_search_end(regex1, text_lowered)
    
    if(end is not None):
        
        return(end)
    
    else:
        # Use -1 to distinguish cases where regex was not found from other failures
        return(-1)


In [165]:
def get_item_section_start(text, form_type):    
    
    
    if(re.search('SC\s+13D', form_type)):
        
        re_list = ['\n\s*[#\.\;\:]?\s*1[#\.\;\:]?\s*security\s+and\s+issuer',
                   '\n\s*[#\.\;\:]?\s*2[#\.\;\:]?\s*identity\s+and\s+background',
                   '\n\s*[#\.\;\:]?\s*3[#\.\;\:]?\s*source\s+and\s+amount\s+of\s+funds',
                   '\n\s*[#\.\;\:]?\s*4[#\.\;\:]?\s*purpose\s+of\s+transaction',
                   '\n\s*[#\.\;\:]?\s*5[#\.\;\:]?\s*interest\s+in\s+securities\s+of\s+the\s+issuer',
                   '\n\s*[#\.\;\:]?\s*6[#\.\;\:]?\s*contracts,\s+arrangements,\s+understandings',
                   '\n\s*[#\.\;\:]?\s*7[#\.\;\:]?\s*material\s+to\s+be\s+filed\s+as\s+exhibits'
                  ]
                       
        
    elif(re.search('SC\s+13G', form_type)):
        
        re_list = ['\n\s*[#\.\;\:]?\s*1[#\.\;\:]?[\(]?\s*a[\)]?[#\.\;\:]?\s*name\s+of\s+issuer',
                   '\n\s*[#\.\;\:]?\s*1[#\.\;\:]?[\(]?\s*b[\)]?[#\.\;\:]?\s*address\s+of\s+issuer',
                   '\n\s*[#\.\;\:]?\s*2[#\.\;\:]?[\(]?\s*a[\)]?[#\.\;\:]?\s*name[s]?\s+of\s+person[s]?\s+filing',
                   '\n\s*[#\.\;\:]?\s*2[#\.\;\:]?[\(]?\s*b[\)]?[#\.\;\:]?\s*address\s+of\s+principle',
                   '\n\s*[#\.\;\:]?\s*2[#\.\;\:]?[\(]?\s*c[\)]?[#\.\;\:]?\s*citizenship',
                   '\n\s*[#\.\;\:]?\s*2[#\.\;\:]?[\(]?\s*d[\)]?[#\.\;\:]?\s*title\s+of\s+class',
                   '\n\s*[#\.\;\:]?\s*2[#\.\;\:]?[\(]?\s*e[\)]?[#\.\;\:]?\s*cusip\s+number',
                   '\n\s*[#\.\;\:]?\s*3[#\.\;\:]?\s*if\s+this\s+statement\s+is\s+filed\s+pursuant',
                   '\n\s*[#\.\;\:]?\s*4[#\.\;\:]?\s*ownership',
                   '\n\s*[#\.\;\:]?\s*5[#\.\;\:]?\s*ownership\s+of\s+five',
                   '\n\s*[#\.\;\:]?\s*6[#\.\;\:]?\s*ownership\s+of\s+more',
                   '\n\s*[#\.\;\:]?\s*7[#\.\;\:]?\s*identification\s+and\s+classification',
                   '\n\s*[#\.\;\:]?\s*8[#\.\;\:]?\s*identification\s+and\s+classification',
                   '\n\s*[#\.\;\:]?\s*9[#\.\;\:]?\s*notice\s+of\s+dissolution',
                   '\n\s*[#\.\;\:]?\s*10[#\.\;\:]?\s*certification'
                  ]
        
    else:
        
        # Not a Schedule 13D(/A) or 13G(/A)
        return(None)
   
    cover_pages_end = find_cover_pages_last_question(text)
    
    if(cover_pages_end):

        text_lowered = text[cover_pages_end:].lower()

        search = re.search('\n\s*item\s*[#\.\;\:]?\s*(?:[0-9])[#\.\;\:]?', text_lowered)
        
        if(search):

            return(cover_pages_end + search.start())  
        
        else:
            return(None) # ie. assume no Item section (this probably happens with 13D/A and 13G/A)
    
    else:
        
        text_lowered = text.lower() # Assume NO REPORTING PAGES (this happens, especially with 13D/A and 13G/A)

        search = re.search('\n\s*item\s*[#\.\;\:]?\s*[0-9][#\.\;\:]?', text_lowered)

        if(search):

            return(search.start())  
        
        else: 
             
            result = None # ie. assume no Item section (this probably happens with 13D/A and 13G/A) if no matches in
                          # re_list
                
            for i in range(len(re_list)):
                
                search = re.search(re_list[i], text_lowered)
                
                if(search):
                    
                    result = search.start()
                    break
            
            
            return(result) 

In [531]:
def get_signatures_sec_start(text, lower_bound = None, upper_bound = None):
    
    text_lowered = get_bounded_text(text, lower_bound, upper_bound).lower()
    
    regex1 = '\n\s*(signature|signatures)\s*(\.)?\n'
    
    regex2 = 'the\s+following\s+certification\s+shall\s+be\s+included\s+if\s+the\s+statement\s+is\s+filed\s+' + \
             'pursuant to rule 13d-1\(b\)'
    
    regex3 = 'by\s+signing\s+below\s+i\s+certify\s+that,\s+to\s+the\s+best\s+of\s+my\s+knowledge'

    regex4 = 'after\s+reasonable\s+inquiry\s+'
    
    regex_list = [regex1, regex2, regex3, regex4]
    
    regex_search = None
    
    for i in range(len(regex_list)):
        
        regex_search = re.search(regex_list[i], text_lowered)
        
        if(regex_search):
            
            break
        
    if(regex_search):
        
        if(lower_bound is not None):
        
            return(regex_search.start() + lower_bound)
        
        else:
            
            return(regex_search.start())
    
    else:
        return(None)

In [529]:
def get_exhibit_sec_start(text, lower_bound = None, upper_bound = None):
    
    text_upper = get_bounded_text(text, lower_bound, upper_bound).upper()
    
    regex0 = '\n\s*EXHIBIT\s+INDEX'
    
    regex1 = '\n\s*EXHIBIT\s+[0-9A-Z](:)?'
    
    regex2 = '\n\s*APPENDIX\s+[0-9A-Z](:)?'
    
    regex_list = [regex0, regex1, regex2]
    
    regex_search = None
    
    for i in range(len(regex_list)):
        
        regex_search = re.search(regex_list[i], text_upper)
        
        if(regex_search):
            
            break
        
    if(regex_search):
        
        if(lower_bound is not None):
        
            return(regex_search.start() + lower_bound)
        
        else:
            
            return(regex_search.start())
    
    else:
        return(-1)

In [537]:
get_exhibit_sec_start(t, lower_bound = 3514, upper_bound = None)

-1

In [535]:
t[3514:]

'\n\n       Signature\n\n       After reasonable inquiry and to the best of my knowledge and\n       belief, I certify that the information set forth in this\n       statement is true, complete and correct.\n\n                 02/15/2010\n                 Date\n\n                 /s/ Dale M. Brown\n                 Signature\n\n                 Dale M. Brown, President\n                 Name/Title\n\n\n\n'

In [594]:
def get_key_indices(file_name, document, form_type, directory):
    
    key_info = {}
    
    text = get_main_doc_text(file_name, document, directory)
    
    key_info['file_name'] = [file_name]
    key_info['document'] = [document]
    key_info['form_type'] = [form_type]
    key_info['title_page_end_lower_bound'] = [find_title_page_end(text)]
    key_info['cover_page_q1_start'] = [find_orthodox_cover_page_q1_start(text)]
    key_info['is_rep_q_as_items'] = [is_rep_q_as_items(text, key_info['cover_page_q1_start'][0])]
    if(key_info['is_rep_q_as_items'][0]):
        
        key_info['cover_page_q1_start'] = [cover_page_start_is_rep_as_q(text, form_type, \
                                                lower_bound = key_info['title_page_end_lower_bound'][0])]
    key_info['num_cusip_sedol_b_q1'] = [num_cusip_sedol_before_index(text, \
                                                                    key_info['cover_page_q1_start'][0])]
    key_info['cover_page_start'] = [cover_page_start(text, form_type, \
            cover_page_q1 = key_info['cover_page_q1_start'][0], \
                                                     rep_q_as_items = key_info['is_rep_q_as_items'][0])]
    
    
    key_info['item_section_start'] = [get_item_section_start(text, form_type)]
    key_info['cover_page_last_q_end'] = [find_cover_pages_last_question(text, \
                        key_info['is_rep_q_as_items'][0], key_info['cover_page_q1_start'][0], \
                                                                        key_info['item_section_start'][0])]
    key_info['cover_page_end'] = [find_cover_pages_end(text, form_type, key_info['is_rep_q_as_items'][0],\
                                key_info['cover_page_last_q_end'][0], key_info['item_section_start'][0])]
    
    key_info['num_cusip_b_items'] = [num_cusip_sedol_before_index(text, \
                                                                  index = key_info['item_section_start'][0])]
    key_info['signature_start'] = [get_signatures_sec_start(text, key_info['item_section_start'][0])]
    key_info['exhibit_start'] = [get_exhibit_sec_start(text, key_info['signature_start'][0])]
    
    if(key_info['cover_page_start'][0] != -1 and key_info['item_section_start'][0] != -1\
      and key_info['item_section_start'][0] > key_info['cover_page_start'][0]):
        
        amend_l_bound = key_info['cover_page_start'][0]
        
    else:
        
        amend_l_bound = None
    
    key_info['amendment_statement_start'] = find_amendment_statement_start(text, form_type, \
                    lower_bound = amend_l_bound, upper_bound = key_info['item_section_start'][0])
    
    
    key_info_df = pd.DataFrame(key_info)
    
    return(key_info_df)

In [21]:
sc_13d_df = indexes_df[indexes_df['form_type'] == 'SC 13D'].copy().reset_index(drop = True)

In [22]:
sc_13d_df

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start,success
0,edgar/data/1360823/0001116502-06-000894.txt,0001116502-06-000894.txt,SC 13D,1812.0,1863.0,False,2.0,1813.0,2700.0,2644.0,2664.0,3.0,10758.0,-1.0,True
1,edgar/data/1295315/0001123292-06-000307.txt,0001123292-06-000307.txt,SC 13D,1554.0,1684.0,False,2.0,1554.0,14056.0,13904.0,14023.0,8.0,26003.0,-1.0,True
2,edgar/data/1327670/0000921895-06-001277.txt,0000921895-06-001277.txt,SC 13D,2896.0,3229.0,False,2.0,2983.0,32979.0,32479.0,32622.0,12.0,50679.0,-1.0,True
3,edgar/data/884423/0000884423-10-000004.txt,0000884423-10-000004.txt,SC 13D,1125.0,1180.0,False,2.0,1125.0,1848.0,1795.0,1809.0,2.0,3514.0,-1.0,True
4,edgar/data/1358905/0001005477-06-002180.txt,0001005477-06-002180.txt,SC 13D,2524.0,2648.0,False,2.0,2543.0,4633.0,4521.0,4532.0,2.0,10156.0,-1.0,True
5,edgar/data/1301030/0000950005-06-000298.txt,0000950005-06-000298.txt,SC 13D,1788.0,1829.0,False,3.0,1788.0,6306.0,6223.0,6270.0,6.0,12247.0,-1.0,True
6,edgar/data/1337851/0001337851-06-000014.txt,0001337851-06-000014.txt,SC 13D,2129.0,2350.0,False,2.0,2202.0,6804.0,6575.0,6587.0,4.0,17230.0,-1.0,True
7,edgar/data/1326150/0000902664-06-001334.txt,0000902664-06-001334.txt,SC 13D,3828.0,4208.0,False,2.0,3962.0,11602.0,9516.0,9616.0,4.0,14745.0,-1.0,True
8,edgar/data/709878/0000950103-11-000069.txt,0000950103-11-000069.txt,SC 13D,1669.0,1669.0,False,1.0,1669.0,3872.0,2698.0,2707.0,1.0,22488.0,-1.0,True
9,edgar/data/1326150/0000902664-06-001658.txt,0000902664-06-001658.txt,SC 13D,3637.0,4019.0,False,2.0,3773.0,10372.0,9241.0,9372.0,4.0,25694.0,-1.0,True


In [23]:
max(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'])

87277.0

In [24]:
min(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'])

0.0

In [25]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] > 20).sum()

12689

In [26]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] > 40).sum()

11579

In [27]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] > 60).sum()

10391

In [28]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] > 80).sum()

9968

In [29]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] > 100).sum()

9526

In [30]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] > 150).sum()

8457

In [31]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] > 200).sum()

7830

In [32]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] > 250).sum()

7255

In [33]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] > 300).sum()

6775

In [34]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] > 500).sum()

5724

In [35]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] > 750).sum()

4883

In [36]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] > 1000).sum()

4298

In [37]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] > 2000).sum()

3175

In [205]:
filt = np.logical_and(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] > 1000, \
                      sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] < 2000)

sc_13d_df.loc[filt]

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start,success,cover_page_max_f_end
7,edgar/data/1326150/0000902664-06-001334.txt,0000902664-06-001334.txt,SC 13D,3828.0,4208.0,False,2.0,3962.0,11602.0,9516.0,9616.0,4.0,14745.0,-1.0,True,9616.0
8,edgar/data/709878/0000950103-11-000069.txt,0000950103-11-000069.txt,SC 13D,1669.0,1669.0,False,1.0,1669.0,3872.0,2698.0,2707.0,1.0,22488.0,-1.0,True,2707.0
79,edgar/data/947116/0000950144-97-013146.txt,0000950144-97-013146.txt,SC 13D,1909.0,2369.0,False,2.0,2024.0,7507.0,5564.0,5722.0,2.0,15990.0,-1.0,True,5722.0
81,edgar/data/915912/0000898822-98-000328.txt,0000898822-98-000328.txt,SC 13D,1805.0,1483.0,False,1.0,1483.0,3873.0,2624.0,2639.0,1.0,14617.0,-1.0,True,2639.0
88,edgar/data/51103/0000909518-07-000044.txt,0000909518-07-000044.txt,SC 13D,2559.0,3037.0,False,2.0,2768.0,19119.0,17895.0,17934.0,5.0,32576.0,-1.0,True,17934.0
126,edgar/data/1470423/0000950123-11-012934.txt,0000950123-11-012934.txt,SC 13D,26.0,1421.0,False,2.0,1365.0,43591.0,42175.0,42193.0,17.0,43955.0,-1.0,True,42193.0
144,edgar/data/1017475/0001193125-12-006480.txt,0001193125-12-006480.txt,SC 13D,1782.0,1831.0,False,2.0,1785.0,5075.0,3462.0,3483.0,4.0,14307.0,-1.0,True,3483.0
157,edgar/data/1462180/0001462180-12-000001.txt,0001462180-12-000001.txt,SC 13D,1108.0,96.0,False,0.0,96.0,1108.0,1050.0,-1.0,0.0,6164.0,-1.0,True,1050.0
158,edgar/data/1462180/0001462180-12-000020.txt,0001462180-12-000020.txt,SC 13D,1116.0,97.0,False,0.0,97.0,1116.0,1058.0,-1.0,0.0,7812.0,-1.0,True,1058.0
159,edgar/data/1462180/0001462180-12-000023.txt,0001462180-12-000023.txt,SC 13D,1114.0,97.0,False,0.0,97.0,1114.0,1056.0,-1.0,0.0,6747.0,-1.0,True,1056.0


In [211]:
i = 637
get_key_indices(sc_13d_df.loc[i, 'file_name'], sc_13d_df.loc[i, 'document'], sc_13d_df.loc[i, 'form_type'], directory)

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start
0,edgar/data/1571188/0001193125-13-092942.txt,0001193125-13-092942.txt,SC 13D,1240,1349,False,2,1296,3486,2415,2426,3,9611,-1


In [348]:
i = 17185
idx = get_key_indices(sc_13d_df.loc[i, 'file_name'], sc_13d_df.loc[i, 'document'], sc_13d_df.loc[i, 'form_type'], directory)
idx

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start
0,edgar/data/1500308/0001193125-14-150502.txt,0001193125-14-150502.txt,SC 13D,2225,2288,False,3,2230,40565,38472,39708,19,85585,-1


In [349]:
t = get_main_doc_text(sc_13d_df.loc[i, 'file_name'], sc_13d_df.loc[i, 'document'], directory)

In [350]:
t[idx.loc[0, 'cover_page_end']:idx.loc[0, 'item_section_start']]

'This Schedule 13D (i)\xa0represents the initial statement on Schedule 13D jointly filed by Tianwen Liu, Yong Feng, Xiaosong Zhang, Junhe Che, Ying Huang, Qiang Peng, Li Wang, Xiaohui Zhu, Yen-wen Kang, Li Huang, Miao Du, Yan Zhou,\nBenson Tam, Jiadong Qu, BENO Group Limited, Jinyuan Development (Hong Kong) Company Limited and Wuxi Jinyuan Industry Investment\xa0& Development Co. Ltd. (collectively, the “Reporting Persons”) with respect to iSoftStone Holdings\nLimited (the “Company” or the “Issuer”), and (ii)\xa0amends and supplements the statement on Schedule 13D filed on July\xa026, 2013, by Mr.\xa0Tianwen Liu with respect to the Issuer with the United States Securities and\nExchange Commission (the “SEC”), as amended and/or supplemented by Amendment No.1 to the Schedule 13D filed on November\xa04, 2013 and Amendment No.2 to the Schedule 13D filed on March\xa03, 2014.'

In [351]:
t[idx.loc[0, 'cover_page_last_q_end']:idx.loc[0, 'cover_page_end']]

'OO\n \xa0\n\n\n(1)\nIncludes Ordinary Shares (as defined below), including Ordinary Shares represented by the ADS (as defined below), indirectly held by Wuxi Jinyuan Industry Investment\n& Development Co. Ltd. through Jinyuan Development (Hong Kong) Company Limited. See Item 5. \n\n\n(2)\nWuxi Jinyuan Industry Investment & Development Co. Ltd. and Jinyuan Development (Hong Kong) Company Limited may also be deemed to beneficially own (i) 103,151,758\nOrdinary Shares held by the other Reporting Persons and (ii) 36,731,389 Ordinary Shares of the Issuer beneficially owned by the Everbright 13D Reporting Persons (as defined below) by reason of the agreements described in Item 4, which are excluded\nfrom the above share amounts and percentages.\xa0See Items 2, 4 and 6. \n\n\n(3)\nBased on a total of 598,049,084 Ordinary Shares, including (i) 582,504,751 Ordinary Shares outstanding as of September 30, 2013, as reported in Exhibit 99.1 to the\nIssuer’s Form 6-K filed with the SEC (as defined 

In [322]:
find_last_footnote_index_end(t, lower_bound = 21481, upper_bound = 22878)

21508

In [354]:
get_index_url('edgar/data/1004039/0000899140-97-000151.txt')

'https://www.sec.gov/Archives/edgar/data/1004039/000089914097000151/0000899140-97-000151-index.htm'

In [346]:
t[21481:22877]

'IN       \n\n\n\n\n\n\n    \n\n\xa0\n\n* Based on an aggregate of 3,746,890,087 Ordinary Shares (comprised of (i)\xa02,380,077,324 Class\xa0A Shares and 659,561,893 Class\xa0B Shares issued and outstanding as of April\xa028, 2014 as disclosed in the Investment Agreement and (ii)\xa0707,250,870 Class\xa0A Shares newly issued by the Issuer to Ali YK under the Investment Agreement).  Each Class\xa0B Share is convertible at the option of the holder into one Class\xa0A Share, and Class\xa0A Shares are not convertible into Class\xa0B Shares.  Based on the foregoing, and pursuant to the SPSA Closing, the percentage of the Ordinary Shares beneficially owned by Yu Feng will equal 2.1%.  Yu Feng is reporting his beneficial ownership of 77,959,008 Class\xa0A Shares as a result of a veto right to be held by YF Venus following the SPSA Closing with respect to any transfer by Ali YK of such 77,959,008 Class\xa0A Shares (subject to certain exceptions).  Yu Feng does not and, following the SPSA Closi

In [347]:
t[22877:]

'9\n\n\n\n\n\n\n\n\xa0\nItem 1.                     Security and Issuer\n\xa0\nThis statement on Schedule 13D (this “Schedule 13D”) relates to the Class\xa0A ordinary shares, par value US$0.00001 per share (the “Class\xa0A Shares”), of Youku Tudou Inc., an exempted company with limited liability incorporated under the laws of the Cayman Islands (the “Issuer”).  The Issuer’s principal place of business is 11/F SinoSteel Plaza, 8 Haidian Street Beijing 100080, People’s Republic of China.\n\xa0\nItem 2.                     Identity and Background\n\xa0\nThis Schedule 13D is being filed jointly, pursuant to Rule\xa013d-1 under the Securities Exchange Act of 1934, as amended (the “Exchange Act”), by (a)\xa0Ali YK Investment Holding Limited, an exempted company with limited liability incorporated under the laws of the Cayman Islands (“Ali YK”), (b)\xa0Alibaba Investment Limited, a company organized under the laws of the British Virgin Islands (“AIL”), (c)\xa0Alibaba Group Holding Limited, an

In [307]:
re.search('\n\s*\n', t[21508:])

<re.Match object; span=(1366, 1369), match='\n\xa0\n'>

In [315]:
text = t
end = find_cover_pages_last_question(text, False)

answer_regex = '\s*([A-Z0]{1}[;, \.]?[A-Z0]{1}[;, \.]?)*([A-Z0]{1}[;, \.]?[A-Z0]{1}[;, \.]?)\s*\n?'

if(upper_bound is not None):

    last_answer = re.search(answer_regex, text[end:upper_bound])

else:

    last_answer = re.search(answer_regex, text[end:])

if(last_answer is not None):

    end = end + last_answer.end()

# Now check for footnotes

last_footnote_start = find_last_footnote_index_end(text, lower_bound = end, upper_bound = upper_bound)

if(last_footnote_start is not None):

    end = last_footnote_start

# Now, check for first multi-newlines

search = re.search('\n\s*\n', text[end:])

if(search is not None):

    end = end + search.end()

    

   

In [327]:
text = t
end = find_cover_pages_last_question(text, False)

answer_regex = '\s*([A-Z0]{1}[;, \.]?[A-Z0]{1}[;, \.]?)*([A-Z0]{1}[;, \.]?[A-Z0]{1}[;, \.]?)\s*\n?'

if(upper_bound is not None):

    last_answer = re.search(answer_regex, text[end:upper_bound])

else:

    last_answer = re.search(answer_regex, text[end:])

if(last_answer is not None):

    end = end + last_answer.end()

In [328]:
end

21506

In [317]:
find_cover_pages_last_question(text, False)

21481

In [323]:
t == text

True

In [451]:
def get_item_section_start(text, form_type):    
    
    
    if(re.search('SC\s+13D', form_type)):
        
        re_list = ['\n\s*[#\.\;\:]?\s*1[#\.\;\:]?\s*security\s+and\s+issuer',
                   '\n\s*[#\.\;\:]?\s*2[#\.\;\:]?\s*identity\s+and\s+background',
                   '\n\s*[#\.\;\:]?\s*3[#\.\;\:]?\s*source\s+and\s+amount\s+of\s+funds',
                   '\n\s*[#\.\;\:]?\s*4[#\.\;\:]?\s*purpose\s+of\s+transaction',
                   '\n\s*[#\.\;\:]?\s*5[#\.\;\:]?\s*interest\s+in\s+securities\s+of\s+the\s+issuer',
                   '\n\s*[#\.\;\:]?\s*6[#\.\;\:]?\s*contracts,\s+arrangements,\s+understandings',
                   '\n\s*[#\.\;\:]?\s*7[#\.\;\:]?\s*material\s+to\s+be\s+filed\s+as\s+exhibits'
                  ]
                       
        
    elif(re.search('SC\s+13G', form_type)):
        
        re_list = ['\n\s*[#\.\;\:]?\s*1[#\.\;\:]?[\(]?\s*a[\)]?[#\.\;\:]?\s*name\s+of\s+issuer',
                   '\n\s*[#\.\;\:]?\s*1[#\.\;\:]?[\(]?\s*b[\)]?[#\.\;\:]?\s*address\s+of\s+issuer',
                   '\n\s*[#\.\;\:]?\s*2[#\.\;\:]?[\(]?\s*a[\)]?[#\.\;\:]?\s*name[s]?\s+of\s+person[s]?\s+filing',
                   '\n\s*[#\.\;\:]?\s*2[#\.\;\:]?[\(]?\s*b[\)]?[#\.\;\:]?\s*address\s+of\s+principle',
                   '\n\s*[#\.\;\:]?\s*2[#\.\;\:]?[\(]?\s*c[\)]?[#\.\;\:]?\s*citizenship',
                   '\n\s*[#\.\;\:]?\s*2[#\.\;\:]?[\(]?\s*d[\)]?[#\.\;\:]?\s*title\s+of\s+class',
                   '\n\s*[#\.\;\:]?\s*2[#\.\;\:]?[\(]?\s*e[\)]?[#\.\;\:]?\s*cusip\s+number',
                   '\n\s*[#\.\;\:]?\s*3[#\.\;\:]?\s*if\s+this\s+statement\s+is\s+filed\s+pursuant',
                   '\n\s*[#\.\;\:]?\s*4[#\.\;\:]?\s*ownership',
                   '\n\s*[#\.\;\:]?\s*5[#\.\;\:]?\s*ownership\s+of\s+five',
                   '\n\s*[#\.\;\:]?\s*6[#\.\;\:]?\s*ownership\s+of\s+more',
                   '\n\s*[#\.\;\:]?\s*7[#\.\;\:]?\s*identification\s+and\s+classification',
                   '\n\s*[#\.\;\:]?\s*8[#\.\;\:]?\s*identification\s+and\s+classification',
                   '\n\s*[#\.\;\:]?\s*9[#\.\;\:]?\s*notice\s+of\s+dissolution',
                   '\n\s*[#\.\;\:]?\s*10[#\.\;\:]?\s*certification'
                  ]
        
    else:
        
        # Not a Schedule 13D(/A) or 13G(/A)
        return(None)
   
    cover_pages_end = find_cover_pages_last_question(text)
    
    if(cover_pages_end):

        text_lowered = text[cover_pages_end:].lower()

        search = re.search('\n\s*item\s*[#\.\;\:]?\s*(?:[0-9])[#\.\;\:]?', text_lowered)
        
        if(search):
            
            search_str = search.group(0)
            
            num_to_item = re.search('item', search_str).start()

            return(cover_pages_end + search.start() + num_to_item)  
        
        else:
            return(None) # ie. assume no Item section (this probably happens with 13D/A and 13G/A)
    
    else:
        
        text_lowered = text.lower() # Assume NO REPORTING PAGES (this happens, especially with 13D/A and 13G/A)

        search = re.search('\n\s*item\s*[#\.\;\:]?\s*[0-9][#\.\;\:]?', text_lowered)

        if(search):

            search_str = search.group(0)
            
            num_to_item = re.search('item', search_str).start()

            return(search.start() + num_to_item)  
        
        else: 
             
            result = None # ie. assume no Item section (this probably happens with 13D/A and 13G/A) if no matches in
                          # re_list
                
            for i in range(len(re_list)):
                
                search = re.search(re_list[i], text_lowered)
                
                if(search):
                    
                    search_str = search.group(0)
            
                    num_to_item = re.search('item', search_str).start()
                    
                    result = search.start() + num_to_item
                    break
            
            
            return(result)     

In [325]:
last_answer

<re.Match object; span=(0, 25), match='IN       \n\n\n\n\n\n\n    \n\n\xa0\n\n'>

In [326]:
end

-1

In [330]:
t[21506:22878]

'* Based on an aggregate of 3,746,890,087 Ordinary Shares (comprised of (i)\xa02,380,077,324 Class\xa0A Shares and 659,561,893 Class\xa0B Shares issued and outstanding as of April\xa028, 2014 as disclosed in the Investment Agreement and (ii)\xa0707,250,870 Class\xa0A Shares newly issued by the Issuer to Ali YK under the Investment Agreement).  Each Class\xa0B Share is convertible at the option of the holder into one Class\xa0A Share, and Class\xa0A Shares are not convertible into Class\xa0B Shares.  Based on the foregoing, and pursuant to the SPSA Closing, the percentage of the Ordinary Shares beneficially owned by Yu Feng will equal 2.1%.  Yu Feng is reporting his beneficial ownership of 77,959,008 Class\xa0A Shares as a result of a veto right to be held by YF Venus following the SPSA Closing with respect to any transfer by Ali YK of such 77,959,008 Class\xa0A Shares (subject to certain exceptions).  Yu Feng does not and, following the SPSA Closing, will not have shared dispositive po

In [329]:
find_last_footnote_index_end(text,21506,  22878)

-1

In [314]:
last_answer

<re.Match object; span=(0, 4), match='\nSC '>

In [316]:
end

-1

In [295]:
t = get_main_doc_text(sc_13d_df.loc[i, 'file_name'], sc_13d_df.loc[i, 'document'], directory)

In [297]:
t[idx.loc[0, 'cover_page_end']:idx.loc[0, 'item_section_start']]

'* Based on an aggregate of 3,746,890,087 Ordinary Shares (comprised of (i)\xa02,380,077,324 Class\xa0A Shares and 659,561,893 Class\xa0B Shares issued and outstanding as of April\xa028, 2014 as disclosed in the Investment Agreement and (ii)\xa0707,250,870 Class\xa0A Shares newly issued by the Issuer to Ali YK under the Investment Agreement).  Each Class\xa0B Share is convertible at the option of the holder into one Class\xa0A Share, and Class\xa0A Shares are not convertible into Class\xa0B Shares.  Based on the foregoing, and pursuant to the SPSA Closing, the percentage of the Ordinary Shares beneficially owned by Yu Feng will equal 2.1%.  Yu Feng is reporting his beneficial ownership of 77,959,008 Class\xa0A Shares as a result of a veto right to be held by YF Venus following the SPSA Closing with respect to any transfer by Ali YK of such 77,959,008 Class\xa0A Shares (subject to certain exceptions).  Yu Feng does not and, following the SPSA Closing, will not have shared dispositive po

In [299]:
find_last_footnote_start(t, lower_bound = idx.loc[0, 'cover_page_last_q_end'],\
                         upper_bound = idx.loc[0, 'item_section_start'])

21490

In [298]:
t[idx.loc[0, 'cover_page_last_q_end']:idx.loc[0, 'cover_page_end']]

'IN       \n\n\n\n\n\n\n    \n\n\xa0\n\n'

In [300]:
t[21490:]

'\n\n\n\n\n\n\n    \n\n\xa0\n\n* Based on an aggregate of 3,746,890,087 Ordinary Shares (comprised of (i)\xa02,380,077,324 Class\xa0A Shares and 659,561,893 Class\xa0B Shares issued and outstanding as of April\xa028, 2014 as disclosed in the Investment Agreement and (ii)\xa0707,250,870 Class\xa0A Shares newly issued by the Issuer to Ali YK under the Investment Agreement).  Each Class\xa0B Share is convertible at the option of the holder into one Class\xa0A Share, and Class\xa0A Shares are not convertible into Class\xa0B Shares.  Based on the foregoing, and pursuant to the SPSA Closing, the percentage of the Ordinary Shares beneficially owned by Yu Feng will equal 2.1%.  Yu Feng is reporting his beneficial ownership of 77,959,008 Class\xa0A Shares as a result of a veto right to be held by YF Venus following the SPSA Closing with respect to any transfer by Ali YK of such 77,959,008 Class\xa0A Shares (subject to certain exceptions).  Yu Feng does not and, following the SPSA Closing, will 

In [478]:
i = 925
idx = get_key_indices(sc_13d_df.loc[i, 'file_name'], sc_13d_df.loc[i, 'document'], sc_13d_df.loc[i, 'form_type'], directory)
idx

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start
0,edgar/data/1546758/0001193125-12-158467.txt,0001193125-12-158467.txt,SC 13D,1458,1511,False,2,1459,2496,2461,2485,2,6002,-1


In [439]:
find_last_footnote_index_end(t, lower_bound = 4548, upper_bound = 4898)

4812

In [400]:
find_cover_pages_end(t, 'SC 13D', False, 13160, 13336)

-1

In [462]:
get_index_url(sc_13d_df.loc[i, 'file_name'])

'https://www.sec.gov/Archives/edgar/data/1546758/000119312512158467/0001193125-12-158467-index.htm'

In [479]:
t[2461:2485]

'IN\n \xa0\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\n\n '

In [480]:
t[2485:2496]

'3\n of 5 \xa0\n '

In [481]:
t[2496:2600]

'Item\xa01. Security and Issuer.  The class of equity securities to which this Schedule 13D relates is the c'

In [455]:
t[5112:5118]

'      '

In [453]:
t[5118:]

'Item\n        1. Security and Issuer\n       \n      This\n        statement relates to the common stock $0.001 par value, of Best Care Inc.,\n        a\n        Nevada Corporation (the “Issuer"). The principal offices of the Issuer are\n        located at 811 Chicago Avenue, Suite 803, Evanston, Illinois 60022.\n      \n      Item\n        2. Identity and Background\n       \n      (a)\n        The\n        name of the person filing this statement: Chaim Limor (the "Reporting\n        Person").\n      \n      (b)\n        The\n        business address of the Reporting Person is: 811 Chicago Avenue, Suite 803,\n        Evanston, Illinois 60022.\n      \n      (c)\n        The\n        present principal occupation of the Reporting Person is serving as secretary\n        and\n        director of the Issuer. The name, principal business, and address of the\n        organization in which such employment is conducted are as follows: Best Care,\n        Inc. 811 Chicago Avenue, Suite 803, E

In [463]:
t = get_main_doc_text(sc_13d_df.loc[i, 'file_name'], sc_13d_df.loc[i, 'document'], directory)

In [475]:
find_last_footnote_index_end(t, 2458, 2496)

In [474]:
re.search('Type of reporting\nperson', t)

<re.Match object; span=(2434, 2458), match='Type of reporting\nperson'>

In [467]:
t[:5362]

'\nSC 13D\n1\nd333556dsc13d.htm\nSCHEDULE 13D\n\n\nSchedule 13D\n\n \n\n \xa0\n\xa0 SECURITIES AND EXCHANGE\nCOMMISSION  Washington, D.C. 20549 \n\xa0 \xa0\nSCHEDULE 13D  Under the Securities Exchange Act of 1934  \xa0\n\xa0 Community\nShores Bank Corporation  (Name of Issuer) \n\xa0 \xa0\nCommon Stock \n(Title of Class of Securities)  204046 10 6  (CUSIP Number) \nJerome M. Schwartz  Dickinson Wright PLLC  2600 West Big Beaver Road, Suite 300\n Troy, Michigan 48084-3312  313-223-3628  (Name, Address and Telephone Number of Person Authorized to\nReceive Notices and Communications)  December\xa030, 2011 \n(Date of Event which Requires Filing of this Statement) \n\xa0 \xa0 If the filing person has previously filed a statement on Schedule 13G to report the acquisition which is the subject of this Schedule 13D, and is filing this schedule because of Rule 13d-1(e), 13d-1(f) or\n13d-1(g), check the following box\xa0\xa0¨.  The remainder of this cover page shall be\nfilled out for a reporting

In [458]:
t[idx.loc[0, 'cover_page_last_q_end']:idx.loc[0, 'cover_page_end']]

'IN\n- -------- -----------------------------------------------------------------------\n\n*    Includes 6,000 shares owned by spouse and 1,335,550 shares held by the John\n     A. & Alexandra McMillan Living Trust.\n\n**   The calculation of the foregoing percentage is based on 38,624,272 shares\n     of common stock outstanding as of December 31, 2006, and includes shares\n     underlying options to purchase 125,000 shares of common stock which are\n     exercisable within 60 days of the filing of this Schedule 13D.\n\n\n'

In [459]:
t[idx.loc[0, 'cover_page_end']:idx.loc[0, 'item_section_start']]

'                               Page 2 of 6 Pages\n\n\n\n\n'

In [435]:
t[4600:5200]

'*The\n      Reporting Person received 800,000 shares of the Issuer’s common stock in\n      consideration for services previously rendered to the Issued in the amount\n      of\n      $800. \n    \n    **Based\n      on 4,250,000 shares of the Issuer’s common stock outstanding as of January 9,\n      2007\n    \n      \n        \n      \n      \n        \n          \n          \n        \n        \n          \n          \n          \n            \n          \n        \n        \n          \n          \n        \n      \n    \n     \n    \n      Item\n        1. Security and Issuer\n       \n      This\n        statement relates t'

In [420]:
get_index_url('edgar/data/941179/0000930413-07-001364.txt')

'https://www.sec.gov/Archives/edgar/data/941179/000093041307001364/0000930413-07-001364-index.htm'

In [401]:
find_last_footnote_index_end(t, 13160, 13336)

13197

In [409]:
t[13186:]

'9\n\n\n\n\n\n\n\n\xa0\nResponses to each item of this Statement on Schedule 13D are incorporated by reference into the response to each other item, as applicable.\n\xa0\n\n\n\nItem 1.    \n\nSecurity and Issuer         \n\xa0\nThis Statement on Schedule 13D relates to the shares of common stock, par value $0.01 (the “Common Stock”), of Great Wolf Resorts,\xa0Inc. (the “Issuer”).  The principal executive offices of the Issuer are located at 525 Junction Road, Suite\xa06000 South, Madison, Wisconsin 53717.\n\xa0\n\n\n\nItem 2.    \n\nIdentity and Background         \n\xa0\nThis Statement on Schedule 13D is filed jointly by (i)\xa0K-9 Holdings,\xa0Inc., a Delaware corporation (“K-9 Holdings”), (ii)\xa0K-9 Investors, L.P., a Delaware limited partnership (“K-9 Investors”), (iii)\xa0Apollo Management VII, L.P., a Delaware limited partnership (“Management\xa0VII”), (iv)\xa0AIF VII Management LLC, a Delaware limited liability company (“AIF VII Management”), (v)\xa0Apollo Management, L.P., a D

In [402]:
t[13197:13336]

'Responses to each item of this Statement on Schedule 13D are incorporated by reference into the response to each other item, as applicable.'

In [391]:
t = get_main_doc_text(sc_13d_df.loc[i, 'file_name'], sc_13d_df.loc[i, 'document'], directory)

In [406]:
def find_cover_pages_end(text, form_type, rep_q_as_items = None, lower_bound = None, upper_bound = None):
    
    if(upper_bound is None):
        
        upper_bound = get_item_section_start(text, form_type)
    
    if(lower_bound is None):
        
        lower_bound = find_cover_pages_last_question(text, rep_q_as_items, None, upper_bound)
    
    end = None
        
    # First check for footnotes
    
    last_footnote_start = find_last_footnote_index_end(text, lower_bound, upper_bound)
    
    if(last_footnote_start != -1):
        
        end = last_footnote_start
        
        search = re.search('\n\s*\n', text[end:upper_bound])

        if(search is not None):

            end = end + search.end()

            return(end)
        
 
    answer_regex = '\s*([A-Z0]{1}[;, \.]?[A-Z0]{1}[;, \.]?)*([A-Z0]{1}[;, \.]?[A-Z0]{1}[;, \.]?)\s*\n?'

    last_answer = re.search(answer_regex, text[lower_bound:upper_bound])

    if(last_answer is not None):

        end = lower_bound + last_answer.end()
        return(end)

    # Finally, try finding a key stopword in all caps, such as SCHEDULE 13D, CUSIP, etc...

    end = first_header_word_after_cover_page(text, form_type, lower_bound, \
                                       upper_bound, rep_q_as_items)

    if(end != -1):

        return(end)

    else:

        return(-1)

            

    

In [395]:
t[13330:]

'cable.\n\xa0\n\n\n\nItem 1.    \n\nSecurity and Issuer         \n\xa0\nThis Statement on Schedule 13D relates to the shares of common stock, par value $0.01 (the “Common Stock”), of Great Wolf Resorts,\xa0Inc. (the “Issuer”).  The principal executive offices of the Issuer are located at 525 Junction Road, Suite\xa06000 South, Madison, Wisconsin 53717.\n\xa0\n\n\n\nItem 2.    \n\nIdentity and Background         \n\xa0\nThis Statement on Schedule 13D is filed jointly by (i)\xa0K-9 Holdings,\xa0Inc., a Delaware corporation (“K-9 Holdings”), (ii)\xa0K-9 Investors, L.P., a Delaware limited partnership (“K-9 Investors”), (iii)\xa0Apollo Management VII, L.P., a Delaware limited partnership (“Management\xa0VII”), (iv)\xa0AIF VII Management LLC, a Delaware limited liability company (“AIF VII Management”), (v)\xa0Apollo Management, L.P., a Delaware limited partnership (“Apollo Management”), (vi)\xa0Apollo Management GP, LLC, a Delaware limited liability company (“Management GP”), (vii)\xa0Apoll

In [ ]:
find_last_search_start(regex, text)

In [215]:
def get_bounded_text(text, lower_bound = None, upper_bound = None):
    
    if(lower_bound is not None):
        
        if(upper_bound is not None):
            
            if(lower_bound >= upper_bound):
                
                raise ValueError("upper_bound not more than lower_bound")
                
            else:
                
                return(text[lower_bound:upper_bound])
            
        else:
            
            return(text[lower_bound:])
        
    else:
        
        if(upper_bound is not None):
            
            return(text[lower_bound:upper_bound])
        
        else:
            
            return(text)
    

In [433]:
def find_last_footnote_index_end(text, lower_bound = None, upper_bound = None):
    
    
    bounded_text = get_bounded_text(text, lower_bound, upper_bound)
        
    
    last_footnote_ind_end = find_last_search_end('\n\s*[\(\|]?\s*([*]+|[0-9]{1,3}|[a-zA-Z]{1})\s*[\)\|]?\s*[a-zA-Z]',\
                                                 bounded_text)
    
    if(last_footnote_ind_end is not None):
        
        if(lower_bound is not None):
            
            result = last_footnote_ind_end + lower_bound
            
        else: 
            
            result = last_footnote_ind_end
            
        return(result)
            
    else:
        
        return(-1)
    

In [476]:
def find_last_footnote_index_end(text, lower_bound = None, upper_bound = None):
    
    
    bounded_text = get_bounded_text(text, lower_bound, upper_bound)
        
    regex1 = '\n\s*[\(\|]?\s*[*]+\s*[\)\|]?\s*[a-zA-Z]'
    regex2 = '\n\s*[\(\|]\s*([0-9]{1,3}|[a-zA-Z]{1})\s*[\)\|]\s*[a-zA-Z]'   
    
    # first try regex1
    
    last_footnote_ind_end  = find_last_search_end(regex1, bounded_text)
    
    if(last_footnote_ind_end is not None):
        
        if(lower_bound is not None):
            
            result = last_footnote_ind_end + lower_bound
            
        else: 
            
            result = last_footnote_ind_end
            
        return(result)
            
    else:
        
        # Try regex2
        
        last_footnote_ind_end = find_last_search_end(regex2, bounded_text)
    
        if(last_footnote_ind_end is not None):

            if(lower_bound is not None):

                result = last_footnote_ind_end + lower_bound

            else: 

                result = last_footnote_ind_end

            return(result)
        
        else:
            
            return(-1)

In [477]:
def find_last_footnote_start(text, lower_bound = None, upper_bound = None):
    
    
    bounded_text = get_bounded_text(text, lower_bound, upper_bound)
        
    regex1 = '\n\s*[\(\|]?\s*[*]+\s*[\)\|]?\s*[a-zA-Z]'
    regex2 = '\n\s*[\(\|]\s*([0-9]{1,3}|[a-zA-Z]{1})\s*[\)\|]\s*[a-zA-Z]'   
    
    # first try regex1
    
    last_footnote_start = find_last_search_start(regex1, bounded_text)
    
    if(last_footnote_start is not None):
        
        if(lower_bound is not None):
            
            result = last_footnote_start + lower_bound
            
        else: 
            
            result = last_footnote_start
            
        return(result)
            
    else:
        
        # Try regex2
        
        last_footnote_start = find_last_search_start(regex2, bounded_text)
    
        if(last_footnote_start is not None):

            if(lower_bound is not None):

                result = last_footnote_start + lower_bound

            else: 

                result = last_footnote_start

            return(result)
        
        else:
            
            return(-1)

In [275]:
def first_header_word_after_cover_page(text, form_type, cover_page_last_q = None, upper_bound = None, rep_q_as_items = None):
    
    if(rep_q_as_items is None):
        
        rep_q_as_items = is_rep_q_as_items(text)
    
    if(cover_page_last_q is None):
        
        cover_page_last_q = find_cover_pages_last_question(text, rep_q_as_items,  lower_bound, upper_bound)
        
    if(cover_page_last_q == -1):
        
        # No question 1 found, so no cover pages, return -1
        return(-1) 
    
    else:
        
        text_to_search = get_bounded_text(text, cover_page_last_q, upper_bound)
        
        text_raised = text_to_search.upper()

        regex = '\n\s*(CUSIP|SEDOL|SCHEDULE\s+13[DG]|PAGE|13[DG])'

        search = re.search(regex, text_raised)

        if(search):

            num_forward = search.start()
            
            start = cover_page_last_q + num_forward

            return(start)

        else:

            return(-1)

In [246]:
first_header_word_after_cover_page(t, sc_13d_df.loc[i, 'form_type'], \
                                   cover_page_last_q = 2415, upper_bound = 3486, rep_q_as_items = False)

3433

In [247]:
t[3433:3486]

'\n\n\n\n\n\n\n\n\n\n\n\n\n\nCUSIP No. Q62163110\n\xa0\xa0\n Page\n 3\n of 5 \xa0'

In [290]:
t = get_main_doc_text(sc_13d_df.loc[i, 'file_name'], sc_13d_df.loc[i, 'document'], directory)

In [254]:
find_last_footnote_start(t, lower_bound = 2415, upper_bound = 3433)

2942

In [376]:
def find_cover_pages_end(text, form_type, rep_q_as_items = None, lower_bound = None, upper_bound = None):
    
    if(upper_bound is None):
        
        upper_bound = get_item_section_start(text, form_type)
    
    if(lower_bound is None):
        
        lower_bound = find_cover_pages_last_question(text, rep_q_as_items, None, upper_bound)
    
    end = None
        
    # First check for footnotes
    
    last_footnote_start = find_last_footnote_index_end(text, lower_bound, upper_bound)
    
    if(last_footnote_start != -1):
        
        end = last_footnote_start
        
        search = re.search('\n\s*\n', text[end:])

        if(search is not None):

            end = end + search.end()

            return(end)


        else:

            return(-1)
        
    else:
        
        answer_regex = '\s*([A-Z0]{1}[;, \.]?[A-Z0]{1}[;, \.]?)*([A-Z0]{1}[;, \.]?[A-Z0]{1}[;, \.]?)\s*\n?'
    
        last_answer = re.search(answer_regex, text[lower_bound:upper_bound])

        if(last_answer is not None):

            end = lower_bound + last_answer.end()
            return(end)
            
        else:

            # Finally, try finding a key stopword in all caps, such as SCHEDULE 13D, CUSIP, etc...

            end = first_header_word_after_cover_page(text, form_type, lower_bound, \
                                               upper_bound, rep_q_as_items)
            
            if(end != -1):
                
                return(end)
            
            else:
                
                return(-1)
            
            

    

In [383]:
find_last_footnote_index_end(t, lower_bound = 2516, upper_bound = 2576)

-1

In [374]:
answer_regex = '\s*([A-Z0]{1}[;, \.]?[A-Z0]{1}[;, \.]?)*([A-Z0]{1}[;, \.]?[A-Z0]{1}[;, \.]?)\s*\n?'
    
last_answer = re.search(answer_regex, t[2516:2576])

In [375]:
last_answer

<re.Match object; span=(0, 9), match='IA\n\n\n\n   '>

In [378]:
last_answer.end()

9

In [387]:
find_cover_pages_end(t, 'SC 13G', rep_q_as_items = False, lower_bound = 2516, upper_bound = 2576)

2525

In [379]:
t[2525:2576]

'CUSIP  902952100    SCHEDULE 13G        PAGE 3 OF 4'

In [389]:
get_key_indices('edgar/data/101507/0001051042-11-000006.txt', '0001051042-11-000006.txt', 'SC 13G', directory)

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start
0,edgar/data/101507/0001051042-11-000006.txt,0001051042-11-000006.txt,SC 13G,1598,1716,False,2,1660,2576,2516,2525,3,5727,-1


In [388]:
def get_key_indices(file_name, document, form_type, directory):
    
    key_info = {}
    
    text = get_main_doc_text(file_name, document, directory)
    
    key_info['file_name'] = [file_name]
    key_info['document'] = [document]
    key_info['form_type'] = [form_type]
    key_info['title_page_end_lower_bound'] = [find_title_page_end(text)]
    key_info['cover_page_q1_start'] = [find_orthodox_cover_page_q1_start(text)]
    key_info['is_rep_q_as_items'] = [is_rep_q_as_items(text, key_info['cover_page_q1_start'][0])]
    if(key_info['is_rep_q_as_items'][0]):
        
        key_info['cover_page_q1_start'] = [cover_page_start_is_rep_as_q(text, form_type, \
                                                lower_bound = key_info['title_page_end_lower_bound'][0])]
    key_info['num_cusip_sedol_b_q1'] = [num_cusip_sedol_before_index(text, \
                                                                    key_info['cover_page_q1_start'][0])]
    key_info['cover_page_start'] = [cover_page_start(text, form_type, \
            cover_page_q1 = key_info['cover_page_q1_start'][0], \
                                                     rep_q_as_items = key_info['is_rep_q_as_items'][0])]
    
    
    key_info['item_section_start'] = [get_item_section_start(text, form_type)]
    key_info['cover_page_last_q_end'] = [find_cover_pages_last_question(text, \
                        key_info['is_rep_q_as_items'][0], key_info['cover_page_q1_start'][0], \
                                                                        key_info['item_section_start'][0])]
    key_info['cover_page_end'] = [find_cover_pages_end(text, form_type, key_info['is_rep_q_as_items'][0],\
                                key_info['cover_page_last_q_end'][0], key_info['item_section_start'][0])]
    
    key_info['num_cusip_b_items'] = [num_cusip_sedol_before_index(text, \
                                                                  index = key_info['item_section_start'][0])]
    key_info['signature_start'] = [get_signatures_sec_start(text)]
    
    if(key_info['cover_page_start'][0] != -1 and key_info['item_section_start'][0] != -1\
      and key_info['item_section_start'][0] > key_info['cover_page_start'][0]):
        
        amend_l_bound = key_info['cover_page_start'][0]
        
    else:
        
        amend_l_bound = None
    
    key_info['amendment_statement_start'] = find_amendment_statement_start(text, form_type, \
                    lower_bound = amend_l_bound, upper_bound = key_info['item_section_start'][0])
    
    
    key_info_df = pd.DataFrame(key_info)
    
    return(key_info_df)

In [360]:
answer_regex = '\s*([A-Z0]{1}[;, \.]?[A-Z0]{1}[;, \.]?)*([A-Z0]{1}[;, \.]?[A-Z0]{1}[;, \.]?)\s*\n?'
    
last_answer = re.search(answer_regex, t[2516:2576])


In [361]:
last_answer

<re.Match object; span=(0, 9), match='IA\n\n\n\n   '>

In [358]:
t = get_main_doc_text('edgar/data/101507/0001051042-11-000006.txt', '0001051042-11-000006.txt', directory)

In [341]:
get_index_url('edgar/data/902219/0000902219-13-000427.txt')

'https://www.sec.gov/Archives/edgar/data/902219/000090221913000427/0000902219-13-000427-index.htm'

In [359]:
t[2501:]

'               IA\n\n\n\n   CUSIP  902952100    SCHEDULE 13G        PAGE 3 OF 4\n\n   ITEM 1 (A)    NAME OF ISSUER\n\n\t         U.S. GLOBAL INVESTORS FUND\n\n   ITEM 1 (B)    ADDRESS OF ISSUER\n\n \t         7900 CALLAGHAN RD.\n\t\t PO BOX 781234\n\t\t SAN ANTONIO, TX  78229\n\n\n   ITEM 2 (A)    NAME OF PERSON FILING\n\n                 FINANCIAL & INVESTMENT MANAGEMENT GROUP, LTD\n\n   ITEM 2 (B)    ADDRESS OF PERSON FILING\n\n                 111 CASS ST.\n                 TRAVERSE CITY, MI 49684\n\n   ITEM 2 (C)    CITIZENSHIP\n\n                 MICHIGAN\n\n   ITEM 2 (D)    TITLE OF CLASS OF SECURITIES\n\n                 CLASS A COMMON STOCK\n\n   ITEM 2 (E)    CUSIP NO.\n\n\t\t 902952100\n\n   ITEM 3        THIS STATEMENT IS BEING FILED BY AN INVESTMENT ADVISOR IN\n                 ACCORDANCE WITH RULE 13D-1(B)(1)(ii)(E).\n\n   CUSIP 902952100     SCHEDULE 13G           PAGE 4 OF 4\n\n   OWNERSHIP\n   ITEM 4 (A)    AMOUNT BENEFICIALLY OWNED\n\n                 881,700   * SEE 

In [282]:
t[2516:2525]

'IA\n\n\n\n   '

In [283]:
t[2525:]

'CUSIP  902952100    SCHEDULE 13G        PAGE 3 OF 4\n\n   ITEM 1 (A)    NAME OF ISSUER\n\n\t         U.S. GLOBAL INVESTORS FUND\n\n   ITEM 1 (B)    ADDRESS OF ISSUER\n\n \t         7900 CALLAGHAN RD.\n\t\t PO BOX 781234\n\t\t SAN ANTONIO, TX  78229\n\n\n   ITEM 2 (A)    NAME OF PERSON FILING\n\n                 FINANCIAL & INVESTMENT MANAGEMENT GROUP, LTD\n\n   ITEM 2 (B)    ADDRESS OF PERSON FILING\n\n                 111 CASS ST.\n                 TRAVERSE CITY, MI 49684\n\n   ITEM 2 (C)    CITIZENSHIP\n\n                 MICHIGAN\n\n   ITEM 2 (D)    TITLE OF CLASS OF SECURITIES\n\n                 CLASS A COMMON STOCK\n\n   ITEM 2 (E)    CUSIP NO.\n\n\t\t 902952100\n\n   ITEM 3        THIS STATEMENT IS BEING FILED BY AN INVESTMENT ADVISOR IN\n                 ACCORDANCE WITH RULE 13D-1(B)(1)(ii)(E).\n\n   CUSIP 902952100     SCHEDULE 13G           PAGE 4 OF 4\n\n   OWNERSHIP\n   ITEM 4 (A)    AMOUNT BENEFICIALLY OWNED\n\n                 881,700   * SEE NOTE 1 *\n\n   ITEM 4 (B)   

In [276]:
get_index_url('edgar/data/101507/0001051042-11-000006.txt')

'https://www.sec.gov/Archives/edgar/data/101507/000105104211000006/0001051042-11-000006-index.htm'

In [278]:
get_index_url(' edgar/data/1379593/0000909143-06-000105.txt')

'https://www.sec.gov/Archives/edgar/data/1379593/000090914306000105/0000909143-06-000105-index.htm'

In [255]:
t[2942:]

'\n\n\n**\nWestcliff Trust may be deemed to beneficially own 71.6% of the Issuer’s issued and outstanding Ordinary Shares, based upon 38,252,399 Ordinary Shares outstanding,\nwhich is the sum of (i) 10,870,275 Ordinary Shares issued and outstanding as reported by the Issuer on its Form 6-K, filed with the US Securities Exchange Commission (the “SEC”) on November 26, 2012, plus (ii) the 27,382,054 Ordinary\nShares that would be issuable to Westcliff Trust upon conversion of the Series\xa03 Notes. \n\n\n\n\n\n\n\n\n\n\n\n\n\nCUSIP No. Q62163110\n\xa0\xa0\n Page\n 3\n of 5 \xa0\n ITEM\xa01. SECURITY AND ISSUER. \nThis Statement of Beneficial Ownership on Schedule 13D (this “Statement”) relates to the Ordinary Shares of Mission NewEnergy\nLimited, an Australian corporation (the “Issuer”) and is being filed pursuant to Rule 13d-2 under the Securities Exchange Act of 1934, as amended (the “Exchange Act”). The address of the principal executive offices of the Issuer\nis Unit 2B, 431 Roberts R

In [235]:
t[3433:3486]

'\n\n\n\n\n\n\n\n\n\n\n\n\n\nCUSIP No. Q62163110\n\xa0\xa0\n Page\n 3\n of 5 \xa0'

In [220]:
re.search('\n\s*[\(\|]?\s*[*0-9a-z]+\s*[\)\|]?', t[2426:3486])

<re.Match object; span=(7, 15), match='\n \xa0\n\n\n*\n'>

In [210]:
get_index_url('edgar/data/1508554/0001193125-10-291892.txt')

'https://www.sec.gov/Archives/edgar/data/1508554/000119312510291892/0001193125-10-291892-index.htm'

In [38]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] > 4000).sum()

1672

In [39]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] > 10000).sum()

302

In [71]:
sc_13d_df.loc[sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] > 10000]

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start,success
35,edgar/data/911092/0000898431-06-000059.txt,0000898431-06-000059.txt,SC 13D,2698.0,2932.0,False,2.0,2858.0,17337.0,17225.0,-1.0,7.0,31259.0,-1.0,True
71,edgar/data/882289/0000950148-97-002504.txt,0000950148-97-002504.txt,SC 13D,2133.0,2463.0,False,2.0,2297.0,23004.0,22906.0,-1.0,9.0,35161.0,-1.0,True
141,edgar/data/1504459/0001144204-12-003504.txt,0001144204-12-003504.txt,SC 13D,1890.0,1992.0,False,2.0,1890.0,24308.0,24276.0,-1.0,20.0,62332.0,-1.0,True
216,edgar/data/831529/0000914248-98-000092.txt,0000914248-98-000092.txt,SC 13D,2187.0,2354.0,False,2.0,2229.0,13848.0,13694.0,-1.0,6.0,19567.0,-1.0,True
397,edgar/data/89041/0001019056-06-000408.txt,0001019056-06-000408.txt,SC 13D,2467.0,2922.0,False,2.0,2681.0,10980.0,10895.0,-1.0,4.0,15732.0,-1.0,True
398,edgar/data/89041/0001019056-06-000414.txt,0001019056-06-000414.txt,SC 13D,2513.0,2924.0,False,2.0,2680.0,13676.0,13591.0,-1.0,5.0,19410.0,-1.0,True
407,edgar/data/1140474/0001013594-06-000383.txt,0001013594-06-000383.txt,SC 13D,1.0,1936.0,False,2.0,1750.0,19221.0,19137.0,-1.0,2.0,27195.0,-1.0,True
423,edgar/data/1000742/0000921530-06-000260.txt,0000921530-06-000260.txt,SC 13D,2488.0,2656.0,False,1.0,2656.0,13717.0,13630.0,-1.0,1.0,25718.0,-1.0,True
491,edgar/data/1008598/0001024739-98-000078.txt,0001024739-98-000078.txt,SC 13D,2425.0,2808.0,False,2.0,2562.0,21600.0,21517.0,-1.0,8.0,32747.0,-1.0,True
495,edgar/data/1283843/0001193125-13-087742.txt,0001193125-13-087742.txt,SC 13D,1688.0,1780.0,False,2.0,1723.0,11188.0,10927.0,-1.0,9.0,30315.0,-1.0,True


In [263]:
get_index_url('edgar/data/33619/0001167674-13-000101.txt')

'https://www.sec.gov/Archives/edgar/data/33619/000116767413000101/0001167674-13-000101-index.htm'

In [284]:

get_key_indices('edgar/data/33619/0001167674-13-000101.txt', '0001167674-13-000101.txt', \
                'SC 13G/A', directory)

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start
0,edgar/data/33619/0001167674-13-000101.txt,0001167674-13-000101.txt,SC 13G/A,1268,1324,False,2,1269,2974,2937,2974,2,9354,-1


In [285]:
get_key_indices('edgar/data/1042441/0001104659-12-009966.txt', '0001104659-12-009966.txt', 'SC 13D/A', directory)

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start
0,edgar/data/1042441/0001104659-12-009966.txt,0001104659-12-009966.txt,SC 13D/A,1706,1741,False,2,1706,32151,32095,32122,21,36167,-1


In [266]:
get_index_url('edgar/data/1042441/0001104659-12-009966.txt')

'https://www.sec.gov/Archives/edgar/data/1042441/000110465912009966/0001104659-12-009966-index.htm'

In [286]:
t = get_main_doc_text('edgar/data/1042441/0001104659-12-009966.txt', '0001104659-12-009966.txt', directory)

In [288]:
t[32095:32122]

'   IN       \n\n\n\n\n\n\n    \n\n\xa0\n'

In [74]:
get_index_url('edgar/data/1051470/0001193125-06-207740.txt')

'https://www.sec.gov/Archives/edgar/data/1051470/000119312506207740/0001193125-06-207740-index.htm'

In [75]:
get_index_url('edgar/data/1173514/0001193125-12-418696.txt')

'https://www.sec.gov/Archives/edgar/data/1173514/000119312512418696/0001193125-12-418696-index.htm'

In [271]:
def get_signatures_sec_start(text):
    
    text_lowered = text.lower()
    
    
    regex1 = 'the\s+following\s+certification\s+shall\s+be\s+included\s+if\s+the\s+statement\s+is\s+filed\s+' + \
             'pursuant to rule 13d-1\(b\)'
    
    regex2 = 'by\s+signing\s+below\s+i\s+certify\s+that,\s+to\s+the\s+best\s+of\s+my\s+knowledge'
    
    regex3 = '\n\s*(signature|signatures)\s*(\.)?\n'

    regex4 = 'after\s+reasonable\s+inquiry\s+'
    
    regex5 = '\n\s*each\s+of\s+the\s+undersigned\s+'
    
    regex6 = '\n\s*(date|dated)\:\s*'
    
    regex_list = [regex1, regex2, regex3, regex4, regex5, regex6]
    
    regex_search = None
    
    for i in range(len(regex_list)):
        
        regex_search = re.search(regex_list[i], text_lowered)
        
        if(regex_search):
            
            break
        
    if(regex_search):
        
        return(regex_search.start())
    
    else:
        return(None)

In [ ]:
Each of the undersigned hereby certifies after reasonable inquiry, that to the best of his/her knowledge and belief, the information set forth in this statement is true, complete and correct

In [79]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_last_q_end'] > 1000).sum()

1757

In [80]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_last_q_end'] > 500).sum()

3577

In [81]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_last_q_end'] > 250).sum()

5923

In [82]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_last_q_end'] > 100).sum()

8992

In [83]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_last_q_end'] > 50).sum()

11590

In [84]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_last_q_end'] > 20).sum()

13339

In [85]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_last_q_end'] > 10).sum()

14245

In [86]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_last_q_end'] > 5).sum()

14909

In [87]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_last_q_end'] > 1).sum()

16318

In [88]:
sc_13d_df.shape[0]

17504

In [ ]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_last_q_end'] > 100).sum()

In [90]:
sc_13d_df['cover_page_max_f_end'] = sc_13d_df.apply(lambda x: max(x['cover_page_last_q_end'], \
                                                                 x['cover_page_end']), axis = 1)

In [91]:
sc_13d_df.loc[sc_13d_df['item_section_start'] - sc_13d_df['cover_page_last_q_end'] > 5000]

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start,success,cover_page_max_f_end
725,edgar/data/1384652/0001384652-07-000005.txt,0001384652-07-000005.txt,SC 13D,2035.0,2316.0,False,2.0,2217.0,14027.0,6885.0,8503.0,2.0,21325.0,-1.0,True,8503.0
906,edgar/data/1142512/0001193125-13-129336.txt,0001193125-13-129336.txt,SC 13D,1777.0,1835.0,False,2.0,1806.0,12460.0,6827.0,6870.0,6.0,15013.0,-1.0,True,6870.0
952,edgar/data/1464420/0001019687-13-000810.txt,0001019687-13-000810.txt,SC 13D,1524.0,1574.0,False,2.0,1550.0,13351.0,7970.0,8220.0,4.0,19589.0,-1.0,True,8220.0
1359,edgar/data/1003565/0001193125-06-241009.txt,0001193125-06-241009.txt,SC 13D,1720.0,1744.0,False,2.0,1721.0,18467.0,12587.0,12612.0,2.0,33008.0,-1.0,True,12612.0
1440,edgar/data/1100969/0001193125-13-219579.txt,0001193125-13-219579.txt,SC 13D,2374.0,2419.0,False,2.0,2396.0,27349.0,15618.0,15629.0,8.0,44918.0,-1.0,True,15629.0
1454,edgar/data/1490054/0001078782-13-001074.txt,0001078782-13-001074.txt,SC 13D,1566.0,1607.0,False,2.0,1584.0,10702.0,5321.0,5575.0,2.0,15556.0,-1.0,True,5575.0
1538,edgar/data/823277/0000950137-06-014005.txt,0000950137-06-014005.txt,SC 13D,1782.0,2010.0,False,2.0,1950.0,11453.0,6018.0,6284.0,2.0,20821.0,-1.0,True,6284.0
1705,edgar/data/865911/0001193125-11-024491.txt,0001193125-11-024491.txt,SC 13D,1798.0,1798.0,False,1.0,1798.0,16371.0,9500.0,9516.0,1.0,27842.0,-1.0,True,9516.0
1747,edgar/data/1475233/0001086380-11-000001.txt,0001086380-11-000001.txt,SC 13D,3156.0,3466.0,False,2.0,3364.0,14385.0,8595.0,8932.0,2.0,23651.0,-1.0,True,8932.0
1848,edgar/data/1051470/0001193125-06-207740.txt,0001193125-06-207740.txt,SC 13D,1698.0,1761.0,False,2.0,1716.0,14003.0,2942.0,6692.0,2.0,18254.0,-1.0,True,6692.0


In [92]:
(sc_13d_df['cover_page_max_f_end'] == -1).sum()

0

In [93]:
sc_13d_df.loc[sc_13d_df['item_section_start'] - sc_13d_df['cover_page_max_f_end'] > 5000]

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start,success,cover_page_max_f_end
725,edgar/data/1384652/0001384652-07-000005.txt,0001384652-07-000005.txt,SC 13D,2035.0,2316.0,False,2.0,2217.0,14027.0,6885.0,8503.0,2.0,21325.0,-1.0,True,8503.0
906,edgar/data/1142512/0001193125-13-129336.txt,0001193125-13-129336.txt,SC 13D,1777.0,1835.0,False,2.0,1806.0,12460.0,6827.0,6870.0,6.0,15013.0,-1.0,True,6870.0
952,edgar/data/1464420/0001019687-13-000810.txt,0001019687-13-000810.txt,SC 13D,1524.0,1574.0,False,2.0,1550.0,13351.0,7970.0,8220.0,4.0,19589.0,-1.0,True,8220.0
1359,edgar/data/1003565/0001193125-06-241009.txt,0001193125-06-241009.txt,SC 13D,1720.0,1744.0,False,2.0,1721.0,18467.0,12587.0,12612.0,2.0,33008.0,-1.0,True,12612.0
1440,edgar/data/1100969/0001193125-13-219579.txt,0001193125-13-219579.txt,SC 13D,2374.0,2419.0,False,2.0,2396.0,27349.0,15618.0,15629.0,8.0,44918.0,-1.0,True,15629.0
1454,edgar/data/1490054/0001078782-13-001074.txt,0001078782-13-001074.txt,SC 13D,1566.0,1607.0,False,2.0,1584.0,10702.0,5321.0,5575.0,2.0,15556.0,-1.0,True,5575.0
1538,edgar/data/823277/0000950137-06-014005.txt,0000950137-06-014005.txt,SC 13D,1782.0,2010.0,False,2.0,1950.0,11453.0,6018.0,6284.0,2.0,20821.0,-1.0,True,6284.0
1705,edgar/data/865911/0001193125-11-024491.txt,0001193125-11-024491.txt,SC 13D,1798.0,1798.0,False,1.0,1798.0,16371.0,9500.0,9516.0,1.0,27842.0,-1.0,True,9516.0
1747,edgar/data/1475233/0001086380-11-000001.txt,0001086380-11-000001.txt,SC 13D,3156.0,3466.0,False,2.0,3364.0,14385.0,8595.0,8932.0,2.0,23651.0,-1.0,True,8932.0
1848,edgar/data/1051470/0001193125-06-207740.txt,0001193125-06-207740.txt,SC 13D,1698.0,1761.0,False,2.0,1716.0,14003.0,2942.0,6692.0,2.0,18254.0,-1.0,True,6692.0


In [96]:
sc_13d_df.loc[sc_13d_df['item_section_start'] - sc_13d_df['cover_page_max_f_end'] > 15000]

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start,success,cover_page_max_f_end
3916,edgar/data/722051/0000722051-06-000202.txt,0000722051-06-000202.txt,SC 13D,2484.0,2556.0,False,2.0,2534.0,26547.0,4749.0,4758.0,9.0,38103.0,-1.0,True,4758.0
4193,edgar/data/919085/0000950157-06-001062.txt,0000950157-06-001062.txt,SC 13D,2466.0,2806.0,False,2.0,2466.0,65952.0,4295.0,4320.0,8.0,75126.0,-1.0,True,4320.0
4625,edgar/data/1041450/0000722051-06-000202.txt,0000722051-06-000202.txt,SC 13D,2484.0,2556.0,False,2.0,2534.0,26547.0,4749.0,4758.0,9.0,38103.0,-1.0,True,4758.0
4685,edgar/data/1064122/0000950133-06-005429.txt,0000950133-06-005429.txt,SC 13D,1862.0,2169.0,False,2.0,2078.0,22240.0,6493.0,6534.0,6.0,52561.0,-1.0,True,6534.0
5161,edgar/data/709878/0000950157-06-001062.txt,0000950157-06-001062.txt,SC 13D,2466.0,2806.0,False,2.0,2466.0,65952.0,4295.0,4320.0,8.0,75126.0,-1.0,True,4320.0
5333,edgar/data/1185854/0000950133-06-005429.txt,0000950133-06-005429.txt,SC 13D,1862.0,2169.0,False,2.0,2078.0,22240.0,6493.0,6534.0,6.0,52561.0,-1.0,True,6534.0
14750,edgar/data/1173514/0001193125-12-418696.txt,0001193125-12-418696.txt,SC 13D,1758.0,1866.0,False,3.0,1793.0,96214.0,75978.0,75998.0,81.0,136777.0,-1.0,True,75998.0
14773,edgar/data/1403853/0001193125-12-497096.txt,0001193125-12-497096.txt,SC 13D,1716.0,1751.0,False,2.0,1719.0,18808.0,2739.0,3181.0,3.0,23179.0,-1.0,True,3181.0
14808,edgar/data/1563359/0001193125-12-497096.txt,0001193125-12-497096.txt,SC 13D,1716.0,1751.0,False,2.0,1719.0,18808.0,2739.0,3181.0,3.0,23179.0,-1.0,True,3181.0
15071,edgar/data/904532/0001193125-12-418696.txt,0001193125-12-418696.txt,SC 13D,1758.0,1866.0,False,3.0,1793.0,96214.0,75978.0,75998.0,81.0,136777.0,-1.0,True,75998.0


In [95]:
sc_13d_df.loc[sc_13d_df['item_section_start'] - sc_13d_df['cover_page_max_f_end'] > 25000]

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start,success,cover_page_max_f_end
4193,edgar/data/919085/0000950157-06-001062.txt,0000950157-06-001062.txt,SC 13D,2466.0,2806.0,False,2.0,2466.0,65952.0,4295.0,4320.0,8.0,75126.0,-1.0,True,4320.0
5161,edgar/data/709878/0000950157-06-001062.txt,0000950157-06-001062.txt,SC 13D,2466.0,2806.0,False,2.0,2466.0,65952.0,4295.0,4320.0,8.0,75126.0,-1.0,True,4320.0


In [94]:
sc_13d_df.loc[sc_13d_df['item_section_start'] - sc_13d_df['cover_page_max_f_end'] > 50000]

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start,success,cover_page_max_f_end
4193,edgar/data/919085/0000950157-06-001062.txt,0000950157-06-001062.txt,SC 13D,2466.0,2806.0,False,2.0,2466.0,65952.0,4295.0,4320.0,8.0,75126.0,-1.0,True,4320.0
5161,edgar/data/709878/0000950157-06-001062.txt,0000950157-06-001062.txt,SC 13D,2466.0,2806.0,False,2.0,2466.0,65952.0,4295.0,4320.0,8.0,75126.0,-1.0,True,4320.0


In [182]:
i = 4193
get_key_indices(sc_13d_df.loc[i, 'file_name'], sc_13d_df.loc[i, 'document'], \
                sc_13d_df.loc[i, 'form_type'], directory)

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start
0,edgar/data/919085/0000950157-06-001062.txt,0000950157-06-001062.txt,SC 13D,3729,4228,False,2,3729,12195,11648,11699,4,34410,-1


In [185]:
t = get_main_doc_text(sc_13d_df.loc[i, 'file_name'], sc_13d_df.loc[i, 'document'], directory)
t[11648:34410]

'\n               \n              IN,\n                HC\n            \n          \n\n      \n    \n    \n    \n      \n        \n        \n      \n      \n        \n        \n        \n          \n        \n      \n      \n        \n        \n        CUSIP\n          No.\n          913377107                                                                                                                                                                                                                                                             Page 4\n          of 20 Pages\n      \n    \n    ITEM\n      1.    SECURITY\n      AND ISSUER\n     \n    The\n      title\n      of the class of equity security to which this statement on Schedule 13D relates\n      is the common stock, par value $.01 per share (the “Shares”), of Universal\n      American Financial Corp., a New York corporation (the “Issuer”). The address of\n      the Issuer’s principal executive offices is Six International Drive

In [187]:
t[12195:34410]

'\n      \n    \n    ITEM\n      1.    SECURITY\n      AND ISSUER\n     \n    The\n      title\n      of the class of equity security to which this statement on Schedule 13D relates\n      is the common stock, par value $.01 per share (the “Shares”), of Universal\n      American Financial Corp., a New York corporation (the “Issuer”). The address of\n      the Issuer’s principal executive offices is Six International Drive, Suite 190,\n      Rye Brook, NY 10573.\n     \n    The\n      Reporting Persons may no longer be deemed to hold the Shares in the ordinary\n      course of business and not with the purpose or with the effect of changing\n      or\n      influencing the control of the Issuer, or in connection with or as a participant\n      in any transaction having such purpose or effect. Prior to the event date,\n      the\n      Reporting Persons relied on Rule 13d-1(b)(1) under the Act with respect to\n      the\n      reporting of their ownership of the Shares.\n     \n    ITEM\

In [193]:
get_index_url('edgar/data/101507/0001051042-11-000006.txt')

'https://www.sec.gov/Archives/edgar/data/101507/000105104211000006/0001051042-11-000006-index.htm'

In [188]:
t[34410:]

'\n      \n    \n     \n     \n    SIGNATURE\n     \n    After\n      reasonable inquiry and to the best of my knowledge and belief, the undersigned\n      certifies that the information set forth in this statement is true, complete\n      and\n      correct.\n     \n    \n      \n\n          \n             \n             \n             \n          \n          \n             Dated: \n              October 25, 2006\n            PERRY\n              CORP.\n          \n          \n              \n             \n             \n          \n          \n             \n            By:  \n            /s/ \n              Richard C. Perry\n          \n          \n             \n            \n              Name:\n              Richard C. Perry\n          \n          \n             \n            Title:   \n              President\n          \n\n      \n    \n     \n    \n      \n        \n\n            \n               \n               \n               \n            \n            \n               D

In [186]:
t[3729:12195]

'\n            \n          \n\n      \n    \n    \n    \n    \n    \n    \n      \n        \n        \n      \n      \n        \n        \n        \n          \n        \n      \n      \n        \n        \n        CUSIP\n          No.\n          913377107                                                                                                                                                                                                                                                           \n Page 2 of 20 Pages\n      \n    \n    \n      \n        \n\n            \n              \n                1.\n              \n              \n                Names\n                  of Reporting Persons.\n                I.R.S.\n                  Identification Nos. of above persons (entities only).\n                 \n                Perry\n                  Corp.\n              \n            \n            \n              \n                2.\n              \n              \n        

In [108]:
get_main_doc_text(sc_13d_df.loc[4193, 'file_name'], sc_13d_df.loc[4193, 'document'], directory)

'\nSC 13D\n1\nsc13d.htm\nSCHEDULE 13D\n\n\n  \n    \n      Schedule 13D\n\n\n\n\n\n\n  \n    \n    \n      \n        \n      \n    \n    \n      \n\n          \n            \n              UNITED\n                STATES\n              SECURITIES\n                AND EXCHANGE COMMISSION\n              Washington,\n                D.C. 20549\n              \n            \n          \n          \n            \n              SCHEDULE\n                13D\n            \n          \n          \n            \n              Under\n                the Securities Exchange Act of 1934\n            \n          \n          \n            \n          \n          \n            \n              UNIVERSAL\n                AMERICAN FINANCIAL CORP.\n            \n          \n          \n            \n              (Name\n                of Issuer)\n              \n            \n          \n          \n            \n              Common\n                Stock (par value $.01 per share)\n            \n      

In [100]:
get_file_text_from_directory(sc_13d_df.loc[4193, 'file_name'], sc_13d_df.loc[4193, 'document'], directory)

'-----BEGIN PRIVACY-ENHANCED MESSAGE-----\nProc-Type: 2001,MIC-CLEAR\nOriginator-Name: webmaster@www.sec.gov\nOriginator-Key-Asymmetric:\n MFgwCgYEVQgBAQICAf8DSgAwRwJAW2sNKK9AVtBzYZmr6aGjlWyK3XmZv3dTINen\n TWSM7vrzLADbmYQaionwg5sDW3P6oaM5D3tdezXMm7z1T+B+twIDAQAB\nMIC-Info: RSA-MD5,RSA,\n Qur5OiBiKW8t3RxGQ8jEryljWwQOFRjbs5OuzYL4Qz7hw5aGWnGh+hsLHPoeBT5/\n DCam4jVdwJxkvHIma4vAgA==\n\n<SEC-DOCUMENT>0000950157-06-001062.txt : 20061025\n<SEC-HEADER>0000950157-06-001062.hdr.sgml : 20061025\n<ACCEPTANCE-DATETIME>20061025143729\nACCESSION NUMBER:\t\t0000950157-06-001062\nCONFORMED SUBMISSION TYPE:\tSC 13D\nPUBLIC DOCUMENT COUNT:\t\t2\nFILED AS OF DATE:\t\t20061025\nDATE AS OF CHANGE:\t\t20061025\nGROUP MEMBERS:\t\tRICHARD C. PERRY\n\nSUBJECT COMPANY:\t\n\n\tCOMPANY DATA:\t\n\t\tCOMPANY CONFORMED NAME:\t\t\tUNIVERSAL AMERICAN FINANCIAL CORP\n\t\tCENTRAL INDEX KEY:\t\t\t0000709878\n\t\tSTANDARD INDUSTRIAL CLASSIFICATION:\tLIFE INSURANCE [6311]\n\t\tIRS NUMBER:\t\t\t\t112580136\n\t\tSTATE OF INCOR

In [73]:
sc_13d_df.loc[sc_13d_df['item_section_start'] - sc_13d_df['cover_page_last_q_end'] > 10000]

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start,success
1440,edgar/data/1100969/0001193125-13-219579.txt,0001193125-13-219579.txt,SC 13D,2374.0,2419.0,False,2.0,2396.0,27349.0,15618.0,15629.0,8.0,44918.0,-1.0,True
1848,edgar/data/1051470/0001193125-06-207740.txt,0001193125-06-207740.txt,SC 13D,1698.0,1761.0,False,2.0,1716.0,14003.0,2942.0,6692.0,2.0,18254.0,-1.0,True
2561,edgar/data/1278382/0001193125-06-207740.txt,0001193125-06-207740.txt,SC 13D,1698.0,1761.0,False,2.0,1716.0,14003.0,2942.0,6692.0,2.0,18254.0,-1.0,True
3679,edgar/data/1134725/0001193125-13-219579.txt,0001193125-13-219579.txt,SC 13D,2374.0,2419.0,False,2.0,2396.0,27349.0,15618.0,15629.0,8.0,44918.0,-1.0,True
3916,edgar/data/722051/0000722051-06-000202.txt,0000722051-06-000202.txt,SC 13D,2484.0,2556.0,False,2.0,2534.0,26547.0,4749.0,4758.0,9.0,38103.0,-1.0,True
4193,edgar/data/919085/0000950157-06-001062.txt,0000950157-06-001062.txt,SC 13D,2466.0,2806.0,False,2.0,2466.0,65952.0,4295.0,4320.0,8.0,75126.0,-1.0,True
4625,edgar/data/1041450/0000722051-06-000202.txt,0000722051-06-000202.txt,SC 13D,2484.0,2556.0,False,2.0,2534.0,26547.0,4749.0,4758.0,9.0,38103.0,-1.0,True
4685,edgar/data/1064122/0000950133-06-005429.txt,0000950133-06-005429.txt,SC 13D,1862.0,2169.0,False,2.0,2078.0,22240.0,6493.0,6534.0,6.0,52561.0,-1.0,True
5161,edgar/data/709878/0000950157-06-001062.txt,0000950157-06-001062.txt,SC 13D,2466.0,2806.0,False,2.0,2466.0,65952.0,4295.0,4320.0,8.0,75126.0,-1.0,True
5333,edgar/data/1185854/0000950133-06-005429.txt,0000950133-06-005429.txt,SC 13D,1862.0,2169.0,False,2.0,2078.0,22240.0,6493.0,6534.0,6.0,52561.0,-1.0,True


In [72]:
t = get_main_doc_text('edgar/data/1100969/0001193125-13-219579.txt', '0001193125-13-219579.txt', directory)
t[15629:27349]

'(1)\nIncludes 104,904 shares of Common Stock of the Company and 131,250 shares issuable upon the exercise of vested options. \n\n\n(2)\nMr.\xa0Zhang is a party to certain agreements described in Item\xa03 and Item\xa04 below, which agreements contain, among other things, voting agreements and\nlimitations on the sale of Common Stock of the Company owned by the Reporting Persons and the Other Voting Shareholders. As a result, Mr.\xa0Zhang may be deemed to be a member of a “group,” within the meaning of Section\xa013(d)(3)\nof the Act. Shares of Common Stock listed as beneficially owned by Mr.\xa0Zhang exclude those held by the Other Voting Shareholders and the Reporting Persons other than Mr.\xa0Zhang, in each case as to which Mr.\xa0Zhang expressly disclaims\nbeneficial ownership. \n\n\n(3)\nPercentage calculated based on 72,925,679 shares of Common Stock, including 72,794,429 shares outstanding as of May\xa09, 2013 (as provided by the Company) and\n131,250 shares issuable upon the ex

In [40]:
(sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] > 50000).sum()

5

In [41]:
sc_13d_df.loc[sc_13d_df['item_section_start'] - sc_13d_df['cover_page_end'] > 50000]

,file_name,document,form_type,title_page_end_lower_bound,cover_page_q1_start,is_rep_q_as_items,num_cusip_sedol_b_q1,cover_page_start,item_section_start,cover_page_last_q_end,cover_page_end,num_cusip_b_items,signature_start,amendment_statement_start,success
2891,edgar/data/1179822/0001144204-06-053486.txt,0001144204-06-053486.txt,SC 13D,2105.0,2388.0,False,2.0,2282.0,87276.0,86217.0,-1.0,23.0,126668.0,-1.0,True
3625,edgar/data/745448/0001144204-06-053486.txt,0001144204-06-053486.txt,SC 13D,2105.0,2388.0,False,2.0,2282.0,87276.0,86217.0,-1.0,23.0,126668.0,-1.0,True
4193,edgar/data/919085/0000950157-06-001062.txt,0000950157-06-001062.txt,SC 13D,2466.0,2806.0,False,2.0,2466.0,65952.0,4295.0,4320.0,8.0,75126.0,-1.0,True
5161,edgar/data/709878/0000950157-06-001062.txt,0000950157-06-001062.txt,SC 13D,2466.0,2806.0,False,2.0,2466.0,65952.0,4295.0,4320.0,8.0,75126.0,-1.0,True
16998,edgar/data/1206133/0001493152-13-000486.txt,0001493152-13-000486.txt,SC 13D,1654.0,1654.0,False,1.0,1654.0,53439.0,53437.0,-1.0,1.0,61678.0,-1.0,True


In [42]:
get_index_url('edgar/data/919085/0000950157-06-001062.txt')

'https://www.sec.gov/Archives/edgar/data/919085/000095015706001062/0000950157-06-001062-index.htm'

In [43]:
directory = '/media/igow/6TB/data/edgar'

In [54]:
full_text = get_file_text_from_directory('edgar/data/709878/0000950157-06-001062.txt', '0000950157-06-001062.txt', directory)

In [67]:
soup = BeautifulSoup(full_text, 'html5lib')

In [68]:
soup.find(["document", "DOCUMENT"])

<document>
<type>SC 13D
<sequence>1
<filename>sc13d.htm
<description>SCHEDULE 13D
<text>

  
    <title>
      Schedule 13D
</title>
<!-- Licensed to: Cravath Swaine & Moore LLP-->
<!-- Document Created using EDGARizer HTML 3.0.4.0 -->
<!-- Copyright 2006 EDGARfilings, Ltd., an IEC company.-->
<!-- All rights reserved EDGARfilings.com -->

  
    <div> </div>
    <div>
      <div>
        <hr align="left" noshade="" size="2" style="COLOR: black" width="100%"/>
      </div>
    </div>
    <div>
      <table cellpadding="0" cellspacing="0" width="100%">

          <tbody><tr>
            <td colspan="3" valign="top" width="77%">
              <div align="center" style="DISPLAY: block; MARGIN-LEFT: 0pt; TEXT-INDENT: 0pt; LINE-HEIGHT: 1.25; MARGIN-RIGHT: 0pt"><font style="DISPLAY: inline; FONT-WEIGHT: bold; FONT-SIZE: 12pt; COLOR: #000000; FONT-FAMILY: Times New Roman"><strong>UNITED
                STATES</strong></font></div>
              <div align="center" style="DISPLAY: block; MARGI

In [69]:
print(soup.find(["document", "DOCUMENT"]).get_text())


SC 13D
1
sc13d.htm
SCHEDULE 13D


  
    
      Schedule 13D






  
     
    
      
        
      
    
    
      

          
            
              UNITED
                STATES
              SECURITIES
                AND EXCHANGE COMMISSION
              Washington,
                D.C. 20549
               
            
          
          
            
              SCHEDULE
                13D
            
          
          
            
              Under
                the Securities Exchange Act of 1934
            
          
          
             
          
          
            
              UNIVERSAL
                AMERICAN FINANCIAL CORP.
            
          
          
            
              (Name
                of Issuer)
               
            
          
          
            
              Common
                Stock (par value $.01 per share)
            
             
            
              913377107
            
         

In [63]:
soup

<html><body><p>-----BEGIN PRIVACY-ENHANCED MESSAGE-----
Proc-Type: 2001,MIC-CLEAR
Originator-Name: webmaster@www.sec.gov
Originator-Key-Asymmetric:
 MFgwCgYEVQgBAQICAf8DSgAwRwJAW2sNKK9AVtBzYZmr6aGjlWyK3XmZv3dTINen
 TWSM7vrzLADbmYQaionwg5sDW3P6oaM5D3tdezXMm7z1T+B+twIDAQAB
MIC-Info: RSA-MD5,RSA,
 Qur5OiBiKW8t3RxGQ8jEryljWwQOFRjbs5OuzYL4Qz7hw5aGWnGh+hsLHPoeBT5/
 DCam4jVdwJxkvHIma4vAgA==

<sec-document>0000950157-06-001062.txt : 20061025
<sec-header>0000950157-06-001062.hdr.sgml : 20061025
<acceptance-datetime>20061025143729
ACCESSION NUMBER:		0000950157-06-001062
CONFORMED SUBMISSION TYPE:	SC 13D
PUBLIC DOCUMENT COUNT:		2
FILED AS OF DATE:		20061025
DATE AS OF CHANGE:		20061025
GROUP MEMBERS:		RICHARD C. PERRY

SUBJECT COMPANY:	

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			UNIVERSAL AMERICAN FINANCIAL CORP
		CENTRAL INDEX KEY:			0000709878
		STANDARD INDUSTRIAL CLASSIFICATION:	LIFE INSURANCE [6311]
		IRS NUMBER:				112580136
		STATE OF INCORPORATION:			NY
		FISCAL YEAR END:			1231

	FILIN

In [57]:
soup.find(["document", "DOCUMENT"])

<document>
<type>SC 13D
<sequence>1
<filename>sc13d.htm
<description>SCHEDULE 13D
<text>
<html>
<head>
<title>
      Schedule 13D
</title>
<!-- Licensed to: Cravath Swaine & Moore LLP-->
<!-- Document Created using EDGARizer HTML 3.0.4.0 -->
<!-- Copyright 2006 EDGARfilings, Ltd., an IEC company.-->
<!-- All rights reserved EDGARfilings.com -->
</head>
<body bgcolor="#ffffff">
<div> </div>
<div>
<div>
<hr align="left" noshade="" size="2" style="COLOR: black" width="100%"/>
</div>
</div>
<div>
<table cellpadding="0" cellspacing="0" width="100%">
<tr>
<td colspan="3" valign="top" width="77%">
<div align="center" style="DISPLAY: block; MARGIN-LEFT: 0pt; TEXT-INDENT: 0pt; LINE-HEIGHT: 1.25; MARGIN-RIGHT: 0pt"><font style="DISPLAY: inline; FONT-WEIGHT: bold; FONT-SIZE: 12pt; COLOR: #000000; FONT-FAMILY: Times New Roman"><strong>UNITED
                STATES</strong></font></div>
<div align="center" style="DISPLAY: block; MARGIN-LEFT: 0pt; TEXT-INDENT: 0pt; LINE-HEIGHT: 1.25; MARGIN-RIGHT: 0

In [59]:
soup.get_text()

'-----BEGIN PRIVACY-ENHANCED MESSAGE-----\nProc-Type: 2001,MIC-CLEAR\nOriginator-Name: webmaster@www.sec.gov\nOriginator-Key-Asymmetric:\n MFgwCgYEVQgBAQICAf8DSgAwRwJAW2sNKK9AVtBzYZmr6aGjlWyK3XmZv3dTINen\n TWSM7vrzLADbmYQaionwg5sDW3P6oaM5D3tdezXMm7z1T+B+twIDAQAB\nMIC-Info: RSA-MD5,RSA,\n Qur5OiBiKW8t3RxGQ8jEryljWwQOFRjbs5OuzYL4Qz7hw5aGWnGh+hsLHPoeBT5/\n DCam4jVdwJxkvHIma4vAgA==\n\n0000950157-06-001062.txt : 20061025\n0000950157-06-001062.hdr.sgml : 20061025\n20061025143729\nACCESSION NUMBER:\t\t0000950157-06-001062\nCONFORMED SUBMISSION TYPE:\tSC 13D\nPUBLIC DOCUMENT COUNT:\t\t2\nFILED AS OF DATE:\t\t20061025\nDATE AS OF CHANGE:\t\t20061025\nGROUP MEMBERS:\t\tRICHARD C. PERRY\n\nSUBJECT COMPANY:\t\n\n\tCOMPANY DATA:\t\n\t\tCOMPANY CONFORMED NAME:\t\t\tUNIVERSAL AMERICAN FINANCIAL CORP\n\t\tCENTRAL INDEX KEY:\t\t\t0000709878\n\t\tSTANDARD INDUSTRIAL CLASSIFICATION:\tLIFE INSURANCE [6311]\n\t\tIRS NUMBER:\t\t\t\t112580136\n\t\tSTATE OF INCORPORATION:\t\t\tNY\n\t\tFISCAL YEAR END:\t\t\t12

In [46]:
t = get_main_doc_text('edgar/data/709878/0000950157-06-001062.txt', '0000950157-06-001062.txt', directory)
t[4320:65952]

'CO\n\n\n\n\n\n\xa0\n\xa0\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCUSIP\n          No.\n          913377107\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0&#\n160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#16

In [107]:
def get_main_doc_text(file_name, document, directory):
  
    full_text = clean_text(get_file_text_from_directory(file_name, document, directory))
    soup = BeautifulSoup(full_text, 'html5lib')
    main_doc_text = soup.find(["document", "DOCUMENT"]).get_text()
    
    return(main_doc_text)

In [148]:
def clean_text(text):
    
    # this function gets rid of unwanted characters and spaces
    
    result = text
    
    char_list = ['\xa0', '&#160;', '&#\n160;']
    
    for c in char_list:
        
        result = re.sub(c, ' ', result) 
        
    return(result)

In [149]:
get_main_doc_text(sc_13d_df.loc[4193, 'file_name'], sc_13d_df.loc[4193, 'document'], directory)

'\nSC 13D\n1\nsc13d.htm\nSCHEDULE 13D\n\n\n  \n    \n      Schedule 13D\n\n\n\n\n\n\n  \n     \n    \n      \n        \n      \n    \n    \n      \n\n          \n            \n              UNITED\n                STATES\n              SECURITIES\n                AND EXCHANGE COMMISSION\n              Washington,\n                D.C. 20549\n               \n            \n          \n          \n            \n              SCHEDULE\n                13D\n            \n          \n          \n            \n              Under\n                the Securities Exchange Act of 1934\n            \n          \n          \n             \n          \n          \n            \n              UNIVERSAL\n                AMERICAN FINANCIAL CORP.\n            \n          \n          \n            \n              (Name\n                of Issuer)\n               \n            \n          \n          \n            \n              Common\n                Stock (par value $.01 per share)\n            \n  

In [150]:
print(get_main_doc_text(sc_13d_df.loc[4193, 'file_name'], sc_13d_df.loc[4193, 'document'], directory))


SC 13D
1
sc13d.htm
SCHEDULE 13D


  
    
      Schedule 13D






  
     
    
      
        
      
    
    
      

          
            
              UNITED
                STATES
              SECURITIES
                AND EXCHANGE COMMISSION
              Washington,
                D.C. 20549
               
            
          
          
            
              SCHEDULE
                13D
            
          
          
            
              Under
                the Securities Exchange Act of 1934
            
          
          
             
          
          
            
              UNIVERSAL
                AMERICAN FINANCIAL CORP.
            
          
          
            
              (Name
                of Issuer)
               
            
          
          
            
              Common
                Stock (par value $.01 per share)
            
             
            
              913377107
            
         

In [127]:
re.sub('\n{1}')

'a\nc'

In [47]:
t

'\nSC 13D\n1\nsc13d.htm\nSCHEDULE 13D\n\n\n\n\n      Schedule 13D\n\n\n\n\n\n\n\n\xa0\n\n\n\n\n\n\n\n\n\nUNITED\n                STATES\nSECURITIES\n                AND EXCHANGE COMMISSION\nWashington,\n                D.C. 20549\n\xa0\n\n\n\n\nSCHEDULE\n                13D\n\n\n\n\nUnder\n                the Securities Exchange Act of 1934\n\n\n\n\xa0\n\n\n\nUNIVERSAL\n                AMERICAN FINANCIAL CORP.\n\n\n\n\n(Name\n                of Issuer)\n\xa0\n\n\n\n\nCommon\n                Stock (par value $.01 per share)\n\n\xa0\n\n913377107\n\n\n\n\n(Title\n                of class of securities)\n\n\xa0\n\n(CUSIP\n                number)\n\n\n\n\nMichael\n                C. Neus\nPerry\n                Corp.\n767\n                Fifth Avenue\n19th\n                Floor\nNew\n                York, NY 10153\nTelephone:\n                (212)\n                583-4000\n\n\n\n\n(Name,\n                address and telephone number of person authorized to receive notices\n             

In [51]:
print(t)


SC 13D
1
sc13d.htm
SCHEDULE 13D




      Schedule 13D







 









UNITED
                STATES
SECURITIES
                AND EXCHANGE COMMISSION
Washington,
                D.C. 20549
 




SCHEDULE
                13D




Under
                the Securities Exchange Act of 1934



 



UNIVERSAL
                AMERICAN FINANCIAL CORP.




(Name
                of Issuer)
 




Common
                Stock (par value $.01 per share)

 

913377107




(Title
                of class of securities)

 

(CUSIP
                number)




Michael
                C. Neus
Perry
                Corp.
767
                Fifth Avenue
19th
                Floor
New
                York, NY 10153
Telephone:
                (212)
                583-4000




(Name,
                address and telephone number of person authorized to receive notices
                and
                communications)




 
October
                24, 2006




(Date
                of event which requir

In [48]:
soup = BeautifulSoup(t, 'html.parser')

In [50]:
soup.get_text()

'\nSC 13D\n1\nsc13d.htm\nSCHEDULE 13D\n\n\n\n\n      Schedule 13D\n\n\n\n\n\n\n\n\xa0\n\n\n\n\n\n\n\n\n\nUNITED\n                STATES\nSECURITIES\n                AND EXCHANGE COMMISSION\nWashington,\n                D.C. 20549\n\xa0\n\n\n\n\nSCHEDULE\n                13D\n\n\n\n\nUnder\n                the Securities Exchange Act of 1934\n\n\n\n\xa0\n\n\n\nUNIVERSAL\n                AMERICAN FINANCIAL CORP.\n\n\n\n\n(Name\n                of Issuer)\n\xa0\n\n\n\n\nCommon\n                Stock (par value $.01 per share)\n\n\xa0\n\n913377107\n\n\n\n\n(Title\n                of class of securities)\n\n\xa0\n\n(CUSIP\n                number)\n\n\n\n\nMichael\n                C. Neus\nPerry\n                Corp.\n767\n                Fifth Avenue\n19th\n                Floor\nNew\n                York, NY 10153\nTelephone:\n                (212)\n                583-4000\n\n\n\n\n(Name,\n                address and telephone number of person authorized to receive notices\n             

In [58]:
full_text

'-----BEGIN PRIVACY-ENHANCED MESSAGE-----\nProc-Type: 2001,MIC-CLEAR\nOriginator-Name: webmaster@www.sec.gov\nOriginator-Key-Asymmetric:\n MFgwCgYEVQgBAQICAf8DSgAwRwJAW2sNKK9AVtBzYZmr6aGjlWyK3XmZv3dTINen\n TWSM7vrzLADbmYQaionwg5sDW3P6oaM5D3tdezXMm7z1T+B+twIDAQAB\nMIC-Info: RSA-MD5,RSA,\n Qur5OiBiKW8t3RxGQ8jEryljWwQOFRjbs5OuzYL4Qz7hw5aGWnGh+hsLHPoeBT5/\n DCam4jVdwJxkvHIma4vAgA==\n\n<SEC-DOCUMENT>0000950157-06-001062.txt : 20061025\n<SEC-HEADER>0000950157-06-001062.hdr.sgml : 20061025\n<ACCEPTANCE-DATETIME>20061025143729\nACCESSION NUMBER:\t\t0000950157-06-001062\nCONFORMED SUBMISSION TYPE:\tSC 13D\nPUBLIC DOCUMENT COUNT:\t\t2\nFILED AS OF DATE:\t\t20061025\nDATE AS OF CHANGE:\t\t20061025\nGROUP MEMBERS:\t\tRICHARD C. PERRY\n\nSUBJECT COMPANY:\t\n\n\tCOMPANY DATA:\t\n\t\tCOMPANY CONFORMED NAME:\t\t\tUNIVERSAL AMERICAN FINANCIAL CORP\n\t\tCENTRAL INDEX KEY:\t\t\t0000709878\n\t\tSTANDARD INDUSTRIAL CLASSIFICATION:\tLIFE INSURANCE [6311]\n\t\tIRS NUMBER:\t\t\t\t112580136\n\t\tSTATE OF INCOR

In [203]:
from multiprocess import Pool

In [ ]:
p = Pool(12)